In [1]:
#Jump to: Maximizing/Matching CONTROL
 #or: Maximizing/Matching SOCIAL  

import csv
import pandas as pd
import numpy as np
import glob
import matplotlib
import scipy as sp
from matplotlib import pyplot as plt
from itertools import cycle, islice
from scipy import stats
from datetime import datetime

import seaborn as sns

-plot score for 50 and 20 intervals 


-removing 1st trials?
-RT
-questionnaire
-JSD for max/match
-win-stay, lose-switch

9/25
need to:

1. look into c07 (key error 399): need to code first 3 trials
2. compare 100, 200, 300 timestamps
4. write script to add probabilities to hand coded df 
5. work on reaction time analyses
    import timestamps from hand coded files 
7. hand code first 3 trials of c07 and add to analyses 

This notebook is for analyzing behavior data in the social multi-armed bandit (SMAB) paradigm.
It uses Pandas dataframes (https://pandas.pydata.org/docs/getting_started/index.html#getting-started).
The 's' datasets are single-player sessions from SMAB3 and the 'j' datasets are multi-player sessions from SMAB3.
The 'c' datasets are single-player sessions from SMAB4 and the 'm' datasets are multi-player sessions from SMAB4.

Updated by Bar Y. 8/4/20

In [40]:
'''
Here, we load all of the csv files into pandas dataframes using organize_trial_by_trial and get the game 1 and 2 scores using 
call_get_score.

This function is called separately for each group of data (SMAB3 control, SMAB3 social, SMAB4 control, SMAB4 social)'''
def imports(sessions,condition,types='default'):
    #create dictionary
    dicti = {}
    
    #loop through the list of sessions
    for i in sessions:
        
        print(i)
        
        #set path based on session
        path = 'data/' + i + '_gamedata.csv'

        #call function to make dataframe
        df = organize_trial_by_trial(path,types)
        
        #add dataframe to dictionary
        dicti[i] = df
        
        #call get score function and add scores to dictionary, with the relevant keys
        if condition == 's' or condition == 'c':
            dicti[i + '_p1_g1_score'],dicti[i + '_p1_g2_score'],dicti[i + '_score_diff'],dicti[i + '_p1_g1_50'],dicti[i + '_p1_g2_50'] = call_get_score(df, condition)
        elif condition == 'm' or condition == 'j':
            dicti[i + '_p1_g1_score'],dicti[i + '_p1_g2_score'],dicti[i + '_p2_g1_score'],dicti[i + '_p2_g2_score'],dicti[i + '_p1_diff'],dicti[i + '_p2_diff'],dicti[i + '_p1_g1_50'],dicti[i + '_p1_g2_50'],dicti[i + '_p2_g1_50'],dicti[i + '_p2_g2_50']= call_get_score(df, condition)
    return dicti

In [2]:
'''
These functions import csv files into pandas dataframes and rename the columns.
input: filename of the csv file to import; the path needs to be included, as well
output: pandas dataframe

There are multiple iterations of this function because the data is in slightly different formats. 
'''


def organize_trial_by_trial(txt_filename, types):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    df = df.T
    if types == 'default':
        df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward", 6:"time", 7:"P1 score", 8:"P2 score"})
    elif types == 'three':
        df = df.rename(columns = {0:"type", 1:"game number", 2:"trial", 3:"player", 4:"arm", 5:"probability", 6:"reward", 7:"time", 8:"P1 score", 9:"P2 score"})
    elif types == 'floatclick':
        df = df.rename(columns = {0:"type", 1:"game number", 2:"trial", 3:"player", 4:"arm", 5:"probability", 6:"P1 score", 7:"P2 score", 8:""})
    #drops first 20 trials of game 1
    #remove if you want to analyze the whole session
    #df = df[40:]
    #df = df.drop(columns = 'time') #can comment this out if want to look at time
    #df = df.drop(['reward'], axis=1)
    df = df.apply(pd.to_numeric)
    df.head()
    
    return df


#used before changes were made to template
def organize_trial_by_trial_hand_code(txt_filename):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    #remove first row with column headings
    df = df.drop(df.index[0])
    #convert values to numbers (NEED TO FIX)
    df.reset_index(drop=True,inplace=True)
    df = df.drop(range(0, 8))
    df.reset_index(drop=True,inplace=True)
    #df = df.apply(pd.to_numeric)
    #add column headings
    df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward",6:"time",7:"P1 score", 8:"P2 score"})
    df.head()
    
    return df

#use this function for hand coded game data files 
def organize_trial_by_trial_hand_code2(txt_filename):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    #remove first row with column headings
    df = df.drop(df.index[0])
    #convert values to numbers (NEED TO FIX)
    df.reset_index(drop=True,inplace=True)
    df = df.drop(range(0, 8))
    df.reset_index(drop=True,inplace=True)
    #drop comment columns
    #df = df.drop(range(7,9), axis = 1)
    #add p1 and p2 score columns
    df.insert
    #add column headings
    df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward",6:"time",7:"comment 1", 8:"comment 2", 9:"P1 score", 10:"P2 score"})
    
    #remove the time and comment columns (we'll add them back in a moment)
    #we are removing them bc we are converting all the values to numbeic and this can't be done to these columns
    time = df['time']
    df = df.drop('time', axis = 1 )
    
    comment_1 = df['comment 1']
    comment_2 = df['comment 2']
    df = df.drop('comment 1', axis = 1 )
    df = df.drop('comment 2', axis = 1 )
    
    df = df.apply(pd.to_numeric)
    
    #add dropped columns back in
    df['time'] = time
    df['comment 1'] = comment_1
    df['comment 2'] = comment_2
    
    #convert time and comment 1 to date time objects
    df['time']= pd.to_datetime(df['time'], errors = 'coerce', format = '%M:%S.%f')
    df['comment 1']= pd.to_datetime(df['comment 1'], errors = 'coerce', format = '%M:%S.%f')

    df.head()
    
    return df

In [3]:
'''
This function returns the score for game 1 and game 2. call_get_score can be used to call this function.
'''
def get_score(df, player_number, game_number):
    if game_number == 1:
        row = 199
    elif game_number == 2:
        row = 399
    if player_number == 1:
        column = 'P1 score'
    elif player_number == 2:
        column = 'P2 score'
    score = df.loc[row, column]
    return score

'''Here we get the score at trial 50 of both games'''
def get_score_at_50(df, player_number, game_number):
    if game_number == 1:
        row = 99
    elif game_number == 2:
        row = 300
    if player_number == 1:
        column = 'P1 score'
    elif player_number == 2:
        column = 'P2 score'
    score = df.loc[row, column]
    return score

In [4]:
'''This function calls get_score.
input: dataframe with game data and version*
output: scores for each player and each game**
*version is 's' for single player SMAB3, 'c' for single player SMAB4, 'j' for multiplayer SMAB3, 
'm' for multiplayer SMAB4
**only 1 player score is outputted for each game in the single player sessions, while scores for both players are 
outputted for multiplayer sessions'''
def call_get_score(df, version):
    p1_g1 = get_score(df, 1, 1)
    p1_g2 = get_score(df, 1, 2)
    p1_score_diff = p1_g2 - p1_g1
    
    p1_g1_50 = get_score_at_50(df, 1, 1)
    p1_g2_50 = get_score_at_50(df, 1, 2)
    
    if version == 'j' or version == 'm':
        p1_g1 = get_score(df, 1, 1)
        p1_g2 = get_score(df, 1, 2)
        p1_score_diff = p1_g2 - p1_g1
        
        p1_g1_50 = get_score_at_50(df, 1, 1)
        p1_g2_50 = get_score_at_50(df, 1, 2)
        
        p2_g1 = get_score(df, 2, 1)
        p2_g2 = get_score(df, 2, 2)
        p2_score_diff = p2_g2 - p2_g1
        
        p2_g1_50 = get_score_at_50(df, 2, 1)
        p2_g2_50 = get_score_at_50(df, 2, 2)
        return p1_g1, p1_g2, p2_g1, p2_g2, p1_score_diff, p2_score_diff, p1_g1_50,p1_g2_50, p2_g1_50, p2_g2_50

    return p1_g1, p1_g2, p1_score_diff, p1_g1_50, p1_g2_50

In [5]:
def remove_first_n_trial(df, n):
    return df.drop(df.head(n).index,inplace=True)

In [6]:
'''This functions separates the dataframes by player and game.
input: dataframe with gamedata and version*
output: four dataframes separated by player and game
*same as call_get_score above'''
def separate_df(df, version):
    df_p1 = df[df['player'] == 1]
    df_p1_g1 = df_p1[df['game number'] == 1]
    df_p1_g2 = df_p1[df['game number'] == 2]
    
    #reset index
    df_p1_g1 = df_p1_g1.reset_index(drop=True)
    df_p1_g2 = df_p1_g2.reset_index(drop=True)
    
    #remove 1st 20 trials
    #df_p1_g1.drop(df_p1_g1.head(20).index,inplace=True)
    #df_p1_g2.drop(df_p1_g2.head(20).index,inplace=True)
    
    if version == 's':
        return df_p1_g1, df_p1_g2
    if version == 'c':
        return df_p1_g1, df_p1_g2
    df_p2 = df[df['player'] ==2 ]
    df_p2_g1 = df_p2[df['game number'] == 1]
    df_p2_g2 = df_p2[df['game number'] == 2]
    
    #reset index
    df_p1_g2.reset_index(drop=True)
    df_p2_g2.reset_index(drop=True)
    
    #remove 1st 20 trials
    #df_p2_g1.drop(df_p2_g1.head(20).index,inplace=True)
    #df_p2_g2.drop(df_p2_g2.head(20).index,inplace=True)
    
    return df_p1_g1, df_p1_g2, df_p2_g1, df_p2_g2

In [7]:
#ignore for now
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [8]:
'''This function separates reaction times into post-win and post-loss arrays.
NOTE that it is intended to be called with dataframes separated by separate_df (ex. s01_p1_g1)'''
def reaction_time(df, time):
    df.reset_index(drop=True, inplace=True)
    post_win_time = []
    post_loss_time = []
    for index, row in df.iterrows():
        if index > 0 and index < 101:
            prev_rew = df.loc[index - 1, 'reward']
            if prev_rew == 1:
                post_win_time.append(df.loc[index, 'time'])
            elif prev_rew == 0:
                post_loss_time.append(df.loc[index, 'time'])
    post_win_mean = np.mean(post_win_time)
    post_loss_mean = np.mean(post_loss_time)
    name = str(get_df_name(df))
    time = time.append({'session' : name , 'post win' : post_win_mean, 'post loss': post_loss_mean} , ignore_index=True)
    return(time)

In [9]:
'''This functionfixes the reward column in the files imported using the '300' function. 
This function works for dataframes that have already been split up by game and player by separate_df (ex. c12_p1_g1)
NOTE that this function had to be written for a subset of the data that had a bug that caused the reward column to be
0 for all trials.'''
def fix_reward(df):
    df.reset_index(drop=True, inplace=True)
    for index, row in df.iterrows():
        if df.loc[index, 'player'] == 1:
                score_column = 'P1 score'
        elif df.loc[index, 'player'] == 2:
                score_column = 'P2 score'
        if index == 0:
            if df.loc[index, score_column] == 1:
                df.loc[index, 'reward'] = 1
            #not necessary here because the values are all 0
            else:
                df.loc[index, 'reward'] = 0
        elif index > 0:
            prev_score = df.loc[index - 1, score_column]
            curr_score = df.loc[index, score_column]
            score_diff = curr_score - prev_score
            df.loc[index, 'reward'] = score_diff
    return(df)

In [10]:
'''This function outputs the hole-probability mappings in each game. 
input: df of session data
output: df with the hole-probability mappings
g = game
p = player
h = hole
r = reward (1 means yes and 0 means no)
'''

def hole_prob_map(df):
    h1_prob = df.loc[df['arm'] == 1, 'probability'].iloc[0]
    h2_prob = df.loc[df['arm'] == 2, 'probability'].iloc[0]
    h3_prob = df.loc[df['arm'] == 3, 'probability'].iloc[0]
    h_probs = pd.DataFrame({'h1_prob': [h1_prob], 'h2_prob': [h2_prob], 'h3_prob': [h3_prob]})
    
    return h_probs


In [11]:
'''Here we compare the choice distributions to idealized models of two decision-making strategies: 
matching and maximizing. Matching would be choosing each each hole proportionally to its reward rate, 
and maximizing would be always choosing the high reward hole.'''

#JSD: Jensen Shannon Divergence. Measures the similarity between two probability distributions. Taken from 
#https://urldefense.com/v3/__https://gist.github.com/zhiyzuo/f80e2b1cfb493a5711330d271a228a3d__;!!Mih3wA!SwZbl3yG75UWaB_c9Pq_T5wxVHgFZMbUZ5HHf5pZDf119g1JHaZr-dX94Xg7JRI$ . *Note: probability distributions need to be
#normalized.'''
#parameters: p and q are the two probability distributions to be compared.

def jsd(p, q, base=np.e):
    
    
    #Implementation of pairwise `jsd` based on https://urldefense.com/v3/__https://en.wikipedia.org/wiki/Jensen**BShannon_divergence__;4oCT!!Mih3wA!SwZbl3yG75UWaB_c9Pq_T5wxVHgFZMbUZ5HHf5pZDf119g1JHaZr-dX9SSBIvMc$ 
    # convert to np.array
    p, q = np.asarray(p), np.asarray(q)
    # normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return sp.stats.entropy(p,m, base=base)/2. +  sp.stats.entropy(q, m, base=base)/2.


#This function calls jsd. 
#Input: counts- a player's choice distributions in a particular game (ex. s01_p1_g1_counts)
#Output: maxi- JSD score for the maximizing strategy
        #matchi- JSD score for the matching strategy
def call_jsd(counts,total=100):
    
    #Idealized probability distributions to use for comparison. You can change these or add more models.

    maximize = [0,0,total]
    match = [30/180 * total,60/180 * total,90/180 * total]
    random = [30/90 * total,30/90 * total,30/90 * total]
    social = [0,total,0]

    maxi = jsd(counts,maximize) #(p, q)
    matchi = jsd(counts,match)
    randi = jsd(counts,random)
    soci = jsd(counts,social)
    return maxi,matchi,randi, soci

# Hand Coding Functions
These functions can be used for game data files that were created by hand coding.

In [12]:
'''This function calculates the number of times each hole was chosen and the 
probability with which each hole rewarded each player in each game.
input: hand coded game data file in a df
output: df with the number of times each hole was chosen by each player in each game and the percentage of choices that were
rewarded
g = game
p = player
h = hole
r = reward (1 means yes and 0 means no)'''

def handcode_calc_prob(df):
    
    #NOTE: turn into function with parameters for game and player
    #get counts for g1, p1
    g1_p1_h1_r1_count,g1_p1_h1_r0_count,g1_p1_h2_r1_count,g1_p1_h2_r0_count,g1_p1_h3_r1_count,g1_p1_h3_r0_count = get_count_hand(df, 1,1)
    
    g1_p2_h1_r1_count,g1_p2_h1_r0_count,g1_p2_h2_r1_count,g1_p2_h2_r0_count,g1_p2_h3_r1_count,g1_p2_h3_r0_count = get_count_hand(df, 2,1)
    
    g2_p1_h1_r1_count,g2_p1_h1_r0_count,g2_p1_h2_r1_count,g2_p1_h2_r0_count,g2_p1_h3_r1_count,g2_p1_h3_r0_count = get_count_hand(df, 1,2)
    
    g2_p2_h1_r1_count,g2_p2_h1_r0_count,g2_p2_h2_r1_count,g2_p2_h2_r0_count,g2_p2_h3_r1_count,g2_p2_h3_r0_count = get_count_hand(df, 2,2)
    
    #calculate reward rate of each hole in each condition
    
    g1_p1_h1 = calc_prob(g1_p1_h1_r1_count,g1_p1_h1_r0_count)
    g1_p1_h2 = calc_prob(g1_p1_h2_r1_count,g1_p1_h2_r0_count)
    g1_p1_h3 = calc_prob(g1_p1_h3_r1_count,g1_p1_h3_r0_count)
    
    g1_p2_h1 = calc_prob(g1_p2_h1_r1_count,g1_p2_h1_r0_count)
    g1_p2_h2 = calc_prob(g1_p2_h2_r1_count,g1_p2_h2_r0_count)
    g1_p2_h3 = calc_prob(g1_p2_h3_r1_count,g1_p2_h3_r0_count)
    
    g2_p1_h1 = calc_prob(g2_p1_h1_r1_count,g2_p1_h1_r0_count)
    g2_p1_h2 = calc_prob(g2_p1_h2_r1_count,g2_p1_h2_r0_count)
    g2_p1_h3 = calc_prob(g2_p1_h3_r1_count,g2_p1_h3_r0_count)
    
    g2_p2_h1 = calc_prob(g2_p2_h1_r1_count,g2_p2_h1_r0_count)
    g2_p2_h2 = calc_prob(g2_p2_h2_r1_count,g2_p2_h2_r0_count)
    g2_p2_h3 = calc_prob(g2_p2_h3_r1_count,g2_p2_h3_r0_count)
    
    
    #NEED to add indices to these
    p1_counts_d = {'g1_p1_h1_count': [g1_p1_h1_r1_count, g1_p1_h1_r0_count], 'g1_p1_h2_count': [g1_p1_h2_r1_count, g1_p1_h2_r0_count],
        'g1_p1_h3_count': [g1_p1_h3_r1_count, g1_p1_h3_r0_count], 'g2_p1_h1_count': [g2_p1_h1_r1_count, g2_p1_h1_r0_count], 
         'g2_p1_h2_count': [g2_p1_h2_r1_count, g2_p1_h2_r0_count],
        'g2_p1_h3_count': [g2_p1_h3_r1_count, g2_p1_h3_r0_count] }
    p1_counts_df = pd.DataFrame(p1_counts_d)
    
    p2_counts_d = {'g1_p2_h1_count': [g1_p2_h1_r1_count, g1_p2_h1_r0_count], 'g1_p2_h2_count': [g1_p2_h2_r1_count, g1_p2_h2_r0_count],
        'g1_p2_h3_count': [g1_p2_h3_r1_count, g1_p2_h3_r0_count], 'g2_p2_h1_count': [g2_p2_h1_r1_count, g2_p2_h1_r0_count], 
         'g2_p2_h2_count': [g2_p2_h2_r1_count, g2_p2_h2_r0_count],
        'g2_p2_h3_count': [g2_p2_h3_r1_count, g2_p2_h3_r0_count] }
    p2_counts_df = pd.DataFrame(p2_counts_d)
    
    p1_counts_df.index = ['rewarded', 'non-rewarded']
    p2_counts_df.index = ['rewarded','non-rewarded']
    
    p1_probs_d = {'p1_h1':[g1_p1_h1, g2_p1_h1], 'p1_h2':[g1_p1_h2, g2_p1_h2], 'p1_h3':[g1_p1_h3, g2_p1_h3]}
    p1_probs_df = pd.DataFrame(p1_probs_d)
    
    p2_probs_d = {'p2_h1':[g1_p2_h1, g2_p2_h1], 'p1_h2':[g1_p2_h2, g2_p2_h2], 'p1_h3':[g1_p2_h3, g2_p2_h3]}
    p2_probs_df = pd.DataFrame(p2_probs_d)
    
    p1_probs_df.index = ['game 1', 'game 2'] 
    p2_probs_df.index = ['game 1','game 2']
    
    return p1_counts_df, p2_counts_df, p1_probs_df, p2_probs_df


In [13]:
'''This function is called in handcode_calc_prob and calculates the number of times each hole was chosen and rewarded/not rewarded
input: df, player and game numbers
output: counts for each hole and outcome (rewarded/not rewarded)
'''
def get_count_hand(df,player,game):
    h1_r1_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==1) & (df['reward']==1)])
    h1_r0_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==1) & (df['reward']==0)])
    h2_r1_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==2) & (df['reward']==1)])
    h2_r0_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==2) & (df['reward']==0)])
    h3_r1_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==3) & (df['reward']==1)])
    h3_r0_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==3) & (df['reward']==0)])
    return h1_r1_count, h1_r0_count, h2_r1_count, h2_r0_count, h3_r1_count, h2_r0_count

In [14]:
'''This function is called in handcode_calc_prob and calculates the reward probability of a hole.
input: number of rewarded trials, number of un-rewarded trials
output: percentage of rewarded trials for that hole'''
def calc_prob(r1,r0):
    try:
        return (r1/(r1+r0) * 100)
    #if the hole was never chosen and the denominator is 0
    except ZeroDivisionError:
        return 0

In [15]:
'''This function fills in the dataframe with the probabilities (30, 60, or 90) to each hole for each player in each
game.
input: dataframe with gamedata, probabilities for each player/game number/hole*
output: dataframe with the probabilities filled in
*You must put the inputs in the correct order as defined in the parameters of the function.
'''
def fill_in_prob(df, p1_g1_h1, p1_g1_h2, p1_g1_h3, p1_g2_h1, p1_g2_h2, p1_g2_h3, p2_g1_h1, p2_g1_h2, p2_g1_h3, 
                 p2_g2_h1, p2_g2_h2, p2_g2_h3):
    for index, row in df.iterrows():
        if df.loc[index,'game number'] == 1:
            if df.loc[index,'player'] == 1:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p1_g1_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p1_g1_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p1_g1_h3
            elif df.loc[index,'player'] == 2:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p2_g1_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p2_g1_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p2_g1_h3
        elif df.loc[index,'game number'] == 2:
            if df.loc[index,'player'] == 1:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p1_g2_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p1_g2_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p1_g2_h3
            elif df.loc[index,'player'] == 2:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p2_g2_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p2_g2_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p2_g2_h3
    return df

In [16]:
from scipy.stats.mstats import winsorize

In [17]:
def winsorize_column(s):
    #90% winsorization
    return winsorize(s, limits=[0.05, 0.05])

# SMAB3 Single Player Analysis

In [18]:
s_trial_by_trial = ['s01','s03','s05','s06','s07','s08','s09']
s_three = ['s11','s12','s15','s16','s17','s19']
s_floatclick = ['s20']

In [20]:
s_trial_by_trial_dict = imports(sessions = s_trial_by_trial, condition = 's')

s01
s03
s05
s06
s07
s08
s09


In [21]:
s_three_dict = imports(sessions = s_three,condition = 's', types = 'three')

s11
s12
s15
s16
s17
s19


In [22]:
s_floatclick_dict = imports(sessions = s_floatclick, condition = 's', types = 'floatclick')

s20


In [23]:
#combine dictionaries into one
s_dict = {**s_trial_by_trial_dict, **s_three_dict}

In [24]:
s_dict

{'s01':      game number  trial  player  arm  probability  reward       time  \
 0              1      1       1    2           30       1  11.220698   
 1              1      1       2    1           90       1   2.742137   
 2              1      2       1    3           60       1   5.362458   
 3              1      2       2    1           90       1   3.699948   
 4              1      3       1    2           30       0   3.489552   
 ..           ...    ...     ...  ...          ...     ...        ...   
 395            2     98       2    1           90       1   2.578115   
 396            2     99       1    1           60       1   2.876227   
 397            2     99       2    1           90       1   2.675604   
 398            2    100       1    1           60       1   2.678057   
 399            2    100       2    1           90       1   2.680739   
 
      P1 score  P2 score  
 0           1         0  
 1           1         1  
 2           2         1  
 3     

In [25]:
'''
The hand coded sessions are done individually.'''
#in the s files, p2 is the confederate
df = organize_trial_by_trial_hand_code2('data/s14_gamedata_handcode.csv')
s14 = df
#s12_p1_g1_score,s12_p1_g2_score,s12_score_diff, s12_p1_g1_50,s12_p1_g2_50 = call_get_score(s12, 's')
s14.head()

,game number,trial,player,arm,probability,reward,time,comment 1,comment 2
0,1,1,1,2.0,NaN,0.0,1900-01-01 00:08:05.158,NaT,
1,1,2,2,NaN,NaN,NaN,NaT,NaT,
2,1,3,1,2.0,NaN,1.0,1900-01-01 00:08:11.966,NaT,
3,1,4,2,NaN,NaN,NaN,NaT,NaT,
4,1,5,1,2.0,NaN,1.0,1900-01-01 00:08:17.495,NaT,


In [26]:
s14_p1_counts, s14_p2_counts, s14_p1_probs, s14_p2_probs = handcode_calc_prob(s14)

In [27]:
s14_p1_probs

,p1_h1,p1_h2,p1_h3
game 1,0.0,88.659794,0.0
game 2,0.0,87.755102,0.0


In [28]:
'''The Hand Code Functions are called for s14'''
s14 = fill_in_prob(s14, 60,90,30,60,90,30,60,90,30,60,90,30)
s14_p1_g1, s14_p1_g2 = separate_df(s14, 'c')
s14_p1_g1_score = s14_p1_g1['reward'].sum()
s14_p1_g2_score = s14_p1_g2['reward'].sum()

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [29]:
s_dict['s14'] = s14
s_dict['s14_p1_g1_score'] = s14_p1_g1_score
s_dict['s14_p1_g2_score'] = s14_p1_g2_score

In [30]:
#TO DO: write function to create s_scores_1, etc.

In [31]:
'''
Here, we create a dataframe of all of the scores and score differences between game 1 and 2'''

#list out all the scores in game 1 by participant
s_scores_1 = pd.DataFrame({'participant':['s01','s03', 's05', 
                               's06', 's07', 's08', 's09', 's11', 's12', 's14','s15', 's16', 's17', 's19', 's20'], 'score':
     [s01_p1_g1_score, s03_p1_g1_score, s05_p1_g1_score,
      s06_p1_g1_score, s07_p1_g1_score, s08_p1_g1_score, s09_p1_g1_score, s11_p1_g1_score, s12_p1_g1_score, s14_p1_g1_score, s15_p1_g1_score, s16_p1_g1_score, s17_p1_g1_score, s19_p1_g1_score, s20_p1_g1_score]})
#to prevent an extra column that numbers each row:
s_scores_1.set_index('participant', inplace=True, drop=True)

#game 2 data is split by same (s) and different (d) contingencies in g1 and g2
#list out all the scores in game 2 by participant
s_scores_2_d = pd.DataFrame({'participant':['s01', 's03', 's05', 
                               's06', 's07', 's08', 's09',], 'score':
    [s01_p1_g2_score, s03_p1_g2_score, s05_p1_g2_score, 
s06_p1_g2_score, s07_p1_g2_score, s08_p1_g2_score, s09_p1_g2_score]})

#to prevent an extra column that numbers each row:
s_scores_2_d.set_index('participant', inplace=True, drop=True)

s_scores_2_s = pd.DataFrame({'participant':['s11', 's12', 's14','s15', 's16', 's17', 's19', 's20'], 'score':
    [s11_p1_g2_score, s12_p1_g2_score, s14_p1_g2_score, s15_p1_g2_score, s16_p1_g2_score, s17_p1_g2_score, s19_p1_g2_score, s20_p1_g2_score]})

#to prevent an extra column that numbers each row:
s_scores_2_s.set_index('participant', inplace=True, drop=True)


#list out all the scores in game 1 by participant
s_scores_1_50 = pd.DataFrame({'participant':['s01','s03', 's05', 
                               's06', 's07', 's08', 's09', 's11', 's12', 's15', 's16', 's17', 's19', 's20'], 'score_1_50':
     [s01_p1_g1_50, s03_p1_g1_50, s05_p1_g1_50,
      s06_p1_g1_50, s07_p1_g1_50, s08_p1_g1_50, s09_p1_g1_50, s11_p1_g1_50, s12_p1_g1_50, s15_p1_g1_50, s16_p1_g1_50, s17_p1_g1_50, s19_p1_g1_50, s20_p1_g1_50]})
#to prevent an extra column that numbers each row:
s_scores_1_50.set_index('participant', inplace=True, drop=True)

#list out all the scores in game 2 by participant
s_scores_2_50 = pd.DataFrame({'participant':['s01', 's03', 's05', 
                               's06', 's07', 's08', 's09', 's11', 's12', 's15', 's16', 's17', 's19', 's20'], 'score_2_50':
    [s01_p1_g2_50, s03_p1_g2_50, s05_p1_g2_50, 
s06_p1_g2_50, s07_p1_g2_50, s08_p1_g2_50, s09_p1_g2_50, s11_p1_g2_50, s12_p1_g2_50, s15_p1_g2_50, s16_p1_g2_50, s17_p1_g2_50, s19_p1_g2_50, s20_p1_g2_50]})
s_scores_2_50.set_index('participant', inplace=True, drop=True)

#frames = [s_scores_1_50,s_scores_2_50,s_scores_1,s_scores_2]'''

NameError: name 's01_p1_g1_score' is not defined

In [ ]:
'''Here we plot the scores.'''

s_scores_1_bar = s_scores_1.plot.bar()

s_scores_2_s_bar = s_scores_2_s.plot.bar()

s_scores_2_d_bar = s_scores_2_d.plot.bar()

'''
#merge both games by participant:
s_scores = pd.merge(s_scores_1, s_scores_2,on='participant')

#label columns
s_scores.columns=['Game 1','Game 2']

s_scores'''

In [ ]:
#make a boxplot of the scores in game 1 and game 2 
#s_scores_box = s_scores.boxplot(labels=['Game 1','Game 2'])
#plt.grid(b=None)
#plt.yticks(np.arange(50, 105, step=10))

s_scores1_box = sns.boxplot( 
                 data=s_scores_1,
                 width=0.5,
                 palette="pastel")
s_scores1_box.axes.set_title("SMAB3 Control Scores in Game 1",
                    fontsize=16)

plot_file_name="s_score1_box.jpg"
 
plt.ylim(0, 100)
s_scores1_box.axes.set(ylabel='Score')

# save as jpeg
s_scores1_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
#make a boxplot of the scores in game 1 and game 2 
#s_scores_box = s_scores.boxplot(labels=['Game 1','Game 2'])
#plt.grid(b=None)
#plt.yticks(np.arange(50, 105, step=10))

s_scores2s_box = sns.boxplot( 
                 data=s_scores_2_s,
                 width=0.5,
                 palette="pastel")
s_scores2s_box.axes.set_title("SMAB3 Control Scores in Game 2-Same",
                    fontsize=16)

plot_file_name="s_score_2s_box.jpg"
 
plt.ylim(0, 100)
s_scores2s_box.axes.set(ylabel='Score')

# save as jpeg
s_scores2s_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
#make a boxplot of the scores in game 1 and game 2 
#s_scores_box = s_scores.boxplot(labels=['Game 1','Game 2'])
#plt.grid(b=None)
#plt.yticks(np.arange(50, 105, step=10))

s_scores2d_box = sns.boxplot( 
                 data=s_scores_2_d,
                 width=0.5,
                 palette="pastel")
s_scores2d_box.axes.set_title("SMAB3 Control Scores in Game 2-Different",
                    fontsize=16)

plot_file_name="s_score_2d_box.jpg"
 
plt.ylim(0, 100)
s_scores2d_box.axes.set(ylabel='Score')

# save as jpeg
s_scores2d_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
'''Here we look at the means, standard deviations, and medians of the scores in games 1 and 2.'''

s_scores_1_mean = s_scores_1.mean()

In [ ]:
s_scores_1_std = s_scores_1.std()

In [ ]:
s_scores_1_mean, s_scores_1_std

In [ ]:
s_scores_2s_mean = s_scores_2_s.mean()

In [ ]:
s_scores_2s_std = s_scores_2_s.std()

In [ ]:
s_scores_2d_mean = s_scores_2_d.mean()

In [ ]:
s_scores_2d_std = s_scores_2_d.std()

In [ ]:
s_scores_2s_mean, s_scores_2s_std

In [ ]:
s_scores_2d_mean, s_scores_2d_std

In [ ]:
s_scores_2_d

In [ ]:
sp.stats.ttest_ind(s_scores_2_s, s_scores_2_d)

In [ ]:
'''Here we separate the dataframes of game 1 and game 2 for each subject.'''


s01_p1_g1, s01_p1_g2 = separate_df(s01, 's')
s03_p1_g1, s03_p1_g2 = separate_df(s03, 's')
s05_p1_g1, s05_p1_g2 = separate_df(s05, 's')
s06_p1_g1, s06_p1_g2 = separate_df(s06, 's')
s07_p1_g1, s07_p1_g2 = separate_df(s07, 's')
s08_p1_g1, s08_p1_g2 = separate_df(s08, 's')
s09_p1_g1, s09_p1_g2 = separate_df(s09, 's')
s11_p1_g1, s11_p1_g2 = separate_df(s11, 's')
s12_p1_g1, s12_p1_g2 = separate_df(s12, 's')
s15_p1_g1, s15_p1_g2 = separate_df(s15, 's')
s16_p1_g1, s16_p1_g2 = separate_df(s16, 's')
s17_p1_g1, s17_p1_g2 = separate_df(s17, 's')
s19_p1_g1, s19_p1_g2 = separate_df(s19, 's')
s20_p1_g1, s20_p1_g2 = separate_df(s20, 's')

In [ ]:
s20_p1_g1

In [ ]:
'''Here we check the arm-probability mappings in each game'''
s01_p1_g1_map = hole_prob_map(s01_p1_g1)
s01_p1_g2_map = hole_prob_map(s01_p1_g2)

s03_p1_g1_map = hole_prob_map(s03_p1_g1)
s03_p1_g2_map = hole_prob_map(s03_p1_g2)

#s05_p1_g1_map = hole_prob_map(s05_p1_g1)
#s05_p1_g2_map = hole_prob_map(s05_p1_g2)

s07_p1_g1_map = hole_prob_map(s07_p1_g1)
s07_p1_g2_map = hole_prob_map(s07_p1_g2)

s08_p1_g1_map = hole_prob_map(s08_p1_g1)
s08_p1_g2_map = hole_prob_map(s08_p1_g2)

s09_p1_g1_map = hole_prob_map(s09_p1_g1)
s09_p1_g2_map = hole_prob_map(s09_p1_g2)

s11_p1_g1_map = hole_prob_map(s11_p1_g1)
s11_p1_g2_map = hole_prob_map(s11_p1_g2)

s12_p1_g1_map = hole_prob_map(s12_p1_g1)
s12_p1_g2_map = hole_prob_map(s12_p1_g2)

s15_p1_g1_map = hole_prob_map(s15_p1_g1)
s15_p1_g2_map = hole_prob_map(s15_p1_g2)

s16_p1_g1_map = hole_prob_map(s16_p1_g1)
s16_p1_g2_map = hole_prob_map(s16_p1_g2)

#s17_p1_g1_map = hole_prob_map(s17_p1_g1)
#s17_p1_g2_map = hole_prob_map(s17_p1_g2)

In [ ]:
s07_p1_g1_map

In [ ]:
s01_p1_g1_score

In [ ]:
'''Here we count the number of times that each hole was selected by each subject in each game by calling value_counts'''


s01_p1_g1_count = s01_p1_g1['probability'].value_counts(sort=False)
s01_p1_g2_count = s01_p1_g2['probability'].value_counts(sort=False)
s03_p1_g1_count = s03_p1_g1['probability'].value_counts(sort=False)
s03_p1_g2_count = s03_p1_g2['probability'].value_counts(sort=False)
s05_p1_g1_count = s05_p1_g1['probability'].value_counts(sort=False)
s05_p1_g2_count = s05_p1_g2['probability'].value_counts(sort=False)
s06_p1_g1_count = s06_p1_g1['probability'].value_counts(sort=False)
s06_p1_g2_count = s06_p1_g2['probability'].value_counts(sort=False)
s07_p1_g1_count = s07_p1_g1['probability'].value_counts(sort=False)
s07_p1_g2_count = s07_p1_g2['probability'].value_counts(sort=False)
s08_p1_g1_count = s08_p1_g1['probability'].value_counts(sort=False)
s08_p1_g2_count = s08_p1_g2['probability'].value_counts(sort=False)
s09_p1_g1_count = s09_p1_g1['probability'].value_counts(sort=False)
s09_p1_g2_count = s09_p1_g2['probability'].value_counts(sort=False)
s11_p1_g1_count = s11_p1_g1['probability'].value_counts(sort=False)
s11_p1_g2_count = s11_p1_g2['probability'].value_counts(sort=False)
s12_p1_g1_count = s12_p1_g1['probability'].value_counts(sort=False)
s12_p1_g2_count = s12_p1_g2['probability'].value_counts(sort=False)
s14_p1_g1_count = s14_p1_g1['probability'].value_counts(sort=False)
s14_p1_g2_count = s14_p1_g2['probability'].value_counts(sort=False)
s15_p1_g1_count = s15_p1_g1['probability'].value_counts(sort=False)
s15_p1_g2_count = s15_p1_g2['probability'].value_counts(sort=False)
s16_p1_g1_count = s16_p1_g1['probability'].value_counts(sort=False)
s16_p1_g2_count = s16_p1_g2['probability'].value_counts(sort=False)
s17_p1_g1_count = s17_p1_g1['probability'].value_counts(sort=False)
s17_p1_g2_count = s17_p1_g2['probability'].value_counts(sort=False)
s19_p1_g1_count = s19_p1_g1['probability'].value_counts(sort=False)
s19_p1_g2_count = s19_p1_g2['probability'].value_counts(sort=False)
s20_p1_g1_count = s20_p1_g1['probability'].value_counts(sort=False)
s20_p1_g2_count = s20_p1_g2['probability'].value_counts(sort=False)

In [ ]:
s05_p1_g1_count

In [ ]:
s05_p1_g2_count

In [ ]:
'''Here we create dataframes with the counts for game 1.'''

s_g1_counts = pd.DataFrame([s01_p1_g1_count, s03_p1_g1_count, s05_p1_g1_count, s06_p1_g1_count, s07_p1_g1_count, 
                s08_p1_g1_count, s09_p1_g1_count, s11_p1_g1_count, s12_p1_g1_count,s14_p1_g1_count, s15_p1_g1_count, s16_p1_g1_count,
                    s17_p1_g1_count, s19_p1_g1_count, s20_p1_g1_count],
                          index=['s01_p1_g1_count', 's03_p1_g1_count', 
                            's05_p1_g1_count', 's06_p1_g1_count', 's07_p1_g1_count', 's08_p1_g1_count', 
                                 's09_p1_g1_count', 's11_p1_g1_count', 's12_p1_g1_count','s14_p1_g1_count', 's15_p1_g1_count', 
                                 's16_p1_g1_count','s17_p1_g1_count', 's19_p1_g1_count', 's20_p1_g1_count'])



#How to prevent NaN: df = df.fillna(0). This makes the value 0 if a hole was never selected by a subject.
s_g1_counts = s_g1_counts.fillna(0)

#print
s_g1_counts

In [ ]:
#Here we plot the value counts

s_g1_counts.sum(axis=1)


s_g1_counts_bar = s_g1_counts.plot.bar()

s_g1_counts_bar.axes.set_title("SMAB3 Control Choice Distributions in Game 1",
                    fontsize=16)

# save as jpeg
s_g1_counts_bar.figure.savefig("s_g1_counts_bar.jpg",
                    format='jpeg')

In [ ]:
s_g1_counts_box = sns.boxplot( 
                 data=s_g1_counts, 
                 width=0.5,
                 palette="pastel")
s_g1_counts_box.axes.set_title("SMAB3 Control Choice Distributions in Game 1",
                    fontsize=16)

plot_file_name="s_g1_counts_box.jpg"
 
plt.ylim(0, 100)
s_g1_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
s_g1_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
'''Here we get the mean and standard deviation of the number of selections of each hole where low is the 0th column and the 30% hole, 
med is the 1st column and the 60% hole, and high is the 2nd column and the 90% hole.'''


s_g1_low = np.mean(s_g1_counts.iloc[:,0])
s_g1_med = np.mean(s_g1_counts.iloc[:,1])
s_g1_high = np.mean(s_g1_counts.iloc[:,2])


s_g1_low_std = np.std(s_g1_counts.iloc[:,0])
s_g1_med_std = np.std(s_g1_counts.iloc[:,1])
s_g1_high_std = np.std(s_g1_counts.iloc[:,2])

In [ ]:
s_g1_low, s_g1_med, s_g1_high

In [ ]:
 s_g1_low_std, s_g1_med_std, s_g1_high_std

In [ ]:
'''Here we create dataframes with the counts for game 2.'''


s_g2_s_counts = pd.DataFrame([s01_p1_g2_count, s03_p1_g2_count, 
                            s05_p1_g2_count, s06_p1_g2_count, s07_p1_g2_count, s08_p1_g2_count, s09_p1_g2_count],
                             index= ['s01_p1_g2_count', 's03_p1_g2_count', 's05_p1_g2_count', 's06_p1_g2_count',
                                 's07_p1_g2_count', 's08_p1_g2_count', 's09_p1_g2_count'])


#How to prevent NaN: df = df.fillna(0)
s_g2_s_counts = s_g2_s_counts.fillna(0)

#print
s_g2_s_counts

s_g2_s_counts.sum(axis=1)

In [ ]:
s_g2_d_counts = pd.DataFrame([s11_p1_g2_count, s12_p1_g2_count, s14_p1_g2_count, s15_p1_g2_count, s16_p1_g2_count,
                    s17_p1_g2_count, s19_p1_g2_count, s20_p1_g2_count],
index= ['s11_p1_g2_count', 's12_p1_g2_count', 's14_p1_g2_count', 's15_p1_g2_count',
                                 's16_p1_g2_count', 's17_p1_g2_count', 's19_p1_g2_count', 's20_p1_g2_count'])


#How to prevent NaN: df = df.fillna(0)
s_g2_d_counts = s_g2_d_counts.fillna(0)


s_g2_d_counts.sum(axis=1)

In [ ]:
s_g2_d_counts

In [ ]:
'''Here we plot the counts in game 2.'''

s_g2_s_counts_bar = s_g2_s_counts.plot.bar()

# save as jpeg
s_g2_s_counts_bar.figure.savefig(plot_file_name,
                    format='jpeg',
                    dpi=100)

In [ ]:
s_g2_s_counts_box = sns.boxplot( 
                 data=s_g2_s_counts, 
                 width=0.5,
                 palette="pastel")
s_g2_s_counts_box.axes.set_title("SMAB3 Control Choice Distributions in Game 2-Same",
                    fontsize=16)

plot_file_name="s_g2_s_counts_box.jpg"

plt.ylim(0, 100)
s_g2_s_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
s_g2_s_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
s_g2_d_counts_box = sns.boxplot( 
                 data=s_g2_d_counts, 
                 width=0.5,
                 palette="pastel")
s_g2_d_counts_box.axes.set_title("SMAB3 Control Choice Distributions in Game 2-Different",
                    fontsize=16)

plot_file_name="s_g2_d_counts_box.jpg"

plt.ylim(0, 100)
s_g2_d_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
s_g2_d_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
'''Here we get the mean and standard deviation of the number of times participants chose each hole.'''


s_g2_s_low = np.mean(s_g2_s_counts.iloc[:,0])
s_g2_s_med = np.mean(s_g2_s_counts.iloc[:,1])
s_g2_s_high = np.mean(s_g2_s_counts.iloc[:,2])


In [ ]:
'''Here we get the mean and standard deviation of the number of times participants chose each hole.'''


s_g2_d_low = np.mean(s_g2_d_counts.iloc[:,0])
s_g2_d_med = np.mean(s_g2_d_counts.iloc[:,1])
s_g2_d_high = np.mean(s_g2_d_counts.iloc[:,2])


In [ ]:
s_g2_s_low_std = np.std(s_g2_s_counts.iloc[:,0])
s_g2_s_med_std = np.std(s_g2_s_counts.iloc[:,1])
s_g2_s_high_std = np.std(s_g2_s_counts.iloc[:,2])

In [ ]:
s_g2_d_low_std = np.std(s_g2_d_counts.iloc[:,0])
s_g2_d_med_std = np.std(s_g2_d_counts.iloc[:,1])
s_g2_d_high_std = np.std(s_g2_d_counts.iloc[:,2])

In [ ]:
'''Here we place the value counts for each hole for each game into separate numy arrays and compare them statistically.


import researchpy as rp


s_90_1 = np.array(s_g1_counts.iloc[:,2])
s_90_2 = np.array(s_g2_counts.iloc[:,2])

s_60_1 = np.array(s_g1_counts.iloc[:,1])
s_60_2 = np.array(s_g2_counts.iloc[:,1])


s_30_1 = np.array(s_g1_counts.iloc[:,0])
s_30_2 = np.array(s_g2_counts.iloc[:,0])

s_90_p = sp.stats.ttest_rel(s_90_1,s_90_2)
s_60_p = sp.stats.ttest_rel(s_60_1,s_60_2)
s_30_p = sp.stats.ttest_rel(s_30_1,s_30_2)'''

In [ ]:
s01_p1_g1.reset_index(drop=True, inplace=True)

# Reaction Time Analyses

In [ ]:
'Here we get the reaction times'
time_s_1 = pd.DataFrame(columns = ['session','post win','post loss'])
time_s_2 = pd.DataFrame(columns = ['session','post win','post loss'])

In [ ]:
time_s_1 = reaction_time(s01_p1_g1, time_s_1)

In [ ]:
time_s_1 = reaction_time(s03_p1_g1, time_s_1)
time_s_1 = reaction_time(s05_p1_g1, time_s_1)
time_s_1 = reaction_time(s06_p1_g1, time_s_1)
time_s_1 = reaction_time(s07_p1_g1, time_s_1)
time_s_1 = reaction_time(s08_p1_g1, time_s_1)
time_s_1 = reaction_time(s09_p1_g1, time_s_1)
time_s_1 = reaction_time(s11_p1_g1, time_s_1)
time_s_1 = reaction_time(s12_p1_g1, time_s_1)
time_s_1 = reaction_time(s15_p1_g1, time_s_1)
time_s_1 = reaction_time(s16_p1_g1, time_s_1)
time_s_1 = reaction_time(s17_p1_g1, time_s_1)
time_s_1 = reaction_time(s19_p1_g1, time_s_1)
#time_s_1 = reaction_time(s20_p1_g1, time_s_1)

In [ ]:
time_s_2 = reaction_time(s03_p1_g2, time_s_2)
time_s_2 = reaction_time(s05_p1_g2, time_s_2)
time_s_2 = reaction_time(s06_p1_g2, time_s_2)
time_s_2 = reaction_time(s07_p1_g2, time_s_2)
time_s_2 = reaction_time(s08_p1_g2, time_s_2)
time_s_2 = reaction_time(s09_p1_g2, time_s_2)
time_s_2 = reaction_time(s11_p1_g2, time_s_2)
time_s_2 = reaction_time(s12_p1_g2, time_s_2)
time_s_2 = reaction_time(s15_p1_g2, time_s_2)
time_s_2 = reaction_time(s16_p1_g2, time_s_2)
time_s_2 = reaction_time(s17_p1_g2, time_s_2)
time_s_2 = reaction_time(s19_p1_g2, time_s_2)
#time_s_2 = reaction_time(s20_p1_g2, time_s_2)

In [ ]:
sp.stats.ttest_rel(time_s_1['post win'], time_s_1['post loss'])

In [ ]:
sp.stats.ttest_rel(time_s_2['post win'], time_s_2['post loss'])

In [ ]:
#sp.stats.ttest_rel(time_s_1['post loss'], time_s_2['post loss'])

In [ ]:
s01_p1_g1.loc[0,'time']

In [ ]:
'''Here we get the average reaction times for each subject in games 1 and 2'''
s01_p1_g1_t = s01_p1_g1['time'].mean()
s01_p1_g2_t = s01_p1_g2['time'].mean()

# SMAB4 Single Player Analysis

In [46]:
c_trial_by_trial = ['c02','c03','c05','c06','c09','c10','c11']
c_three = ['c12','c13','c14','c15','c16','c17','c18']

In [47]:
c_trial_by_trial_dict = imports(sessions = c_trial_by_trial, condition = 'c')

c02
c03
c05
c06
c09
c10
c11


In [ ]:
c_three_dict = imports(sessions = c_three, condition = 'c', types = 'three')

c12
c13
c14
c15
c16
c17
c18


In [ ]:
'''Here we load the csv files into pandas dataframes by calling organize_trial_by_trial and get the scores in games
1 and 2 by calling call_get_score.'''

df = organize_trial_by_trial_hand_code2('data/c01_gamedata_handcode.csv')
c01 = df
#c02_p1_g1_score,c02_p1_g2_score,c02_score_diff, c02_p1_g1_50,c02_p1_g2_50 = call_get_score(c02, 'c')
c01.head()

#missing trials! excluded
df = organize_trial_by_trial_hand_code2('data/c04_gamedata_handcode.csv')
c04 = df
#c04_p1_g1_score,c04_p1_g2_score,c04_score_diff, c04_p1_g1_50,c04_p1_g2_50 = call_get_score(c04, 'c')
c04.head()

#in the s files, p2 is the confederate
df = organize_trial_by_trial('data/c07_gamedata.csv',types='trial_by_trial')
c07 = df
c07_p1_g1_score,c07_p1_g2_score,c07_score_diff, c07_p1_g1_50,c07_p1_g2_50 = 78, 86, 8, 34, 42 
#c07.head()

In [ ]:
#c01 has 9 extra trials in game 1
#drop these trials
c01 = c01.drop(c01.index[200:210])
c01.reset_index(drop=True)

In [ ]:
c01_p1_counts, c01_p2_counts, c01_p1_probs, c01_p2_probs = handcode_calc_prob(c01)
c04_p1_counts, c04_p2_counts, c04_p1_probs, c04_p2_probs = handcode_calc_prob(c04)

In [ ]:
c01_p1_counts

In [ ]:
c04_p1_counts

In [ ]:
c01_p1_probs

In [ ]:
c04_p1_probs

In [ ]:
#NOT SURE
c01 = fill_in_prob(c01, 90,60,30,90,60,30,90,60,30,90,60,30)
c04 = fill_in_prob(c04, 90,60,30,90,60,30,90,60,30,90,60,30)

In [ ]:
c01_p1_g1, c01_p1_g2 = separate_df(c01, 'c')
c04_p1_g1, c04_p1_g2 = separate_df(c04, 'c')

In [ ]:
c01_p1_g1_score = c01_p1_g1['reward'].sum()
c01_p1_g2_score = c01_p1_g2['reward'].sum()
c04_p1_g1_score = c04_p1_g1['reward'].sum()
c04_p1_g2_score = c04_p1_g2['reward'].sum()

In [ ]:
c_dict = {**c_trial_by_trial_dict, **c_three_dict}

In [ ]:
c_dict['c07'] = c07
c_dict['c01'] = c01
c_dict['c04'] = c04
c_dict['c01_p1_g1_score'] = c01_p1_g1_score
c_dict['c01_p1_g2_score'] = c01_p1_g2_score
c_dict['c04_p1_g1_score'] = c04_p1_g1_score
c_dict['c04_p1_g2_score'] = c04_p1_g2_score
c_dict['c07_p1_g1_score'] = c07_p1_g1_score
c_dict['c07_p1_g2_score'] = c07_p1_g2_score

In [ ]:
c_dict

In [ ]:
'''Here we place the scores in games 1 and 2 into dataframes.'''


#list out all the scores in game 1 by participant
c_scores_1 = pd.DataFrame({'participant':['c01','c02', 'c03', 
                               'c05', 'c06', 'c07','c09', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18'], 'score':
     [c01_p1_g1_score, c02_p1_g1_score, c03_p1_g1_score, c05_p1_g1_score, c06_p1_g1_score, c07_p1_g1_score,
      c09_p1_g1_score, c10_p1_g1_score, c11_p1_g1_score, c12_p1_g1_score, c13_p1_g1_score, c14_p1_g1_score, 
      c15_p1_g1_score, c16_p1_g1_score, c17_p1_g1_score, c18_p1_g1_score]})
#to prevent an extra column that numbers each row:
c_scores_1.set_index('participant', inplace=True, drop=True)

#print
c_scores_1


#list out all the scores in game 2 by participant
c_scores_2_d = pd.DataFrame({'participant':['c02', 'c03', 
                              'c05', 'c06', 'c07','c09', 'c10', 'c11'], 'score':
                           [c02_p1_g2_score, c03_p1_g2_score, c05_p1_g2_score, c06_p1_g2_score, c07_p1_g2_score, c09_p1_g2_score, c10_p1_g2_score, c11_p1_g2_score]})

#to prevent an extra column that numbers each row:
c_scores_2_d.set_index('participant', inplace=True, drop=True)

c_scores_2_s = pd.DataFrame({'participant':['c01','c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18'], 'score':
                           [c01_p1_g2_score, c12_p1_g2_score, c13_p1_g2_score, c14_p1_g2_score, c15_p1_g2_score, c16_p1_g2_score, c17_p1_g2_score, c18_p1_g2_score]})

#to prevent an extra column that numbers each row:
c_scores_2_s.set_index('participant', inplace=True, drop=True)

'''
#list out all the scores in game 1 by participant
c_scores_1_50 = pd.DataFrame({'participant':['c02', 'c03', 
                              'c05', 'c06', 'c09', 'c10', 'c11', 'c12', 'c13', 'c15', 'c14', 'c16', 'c17', 'c18'], 'score_1_50':
     [c02_p1_g1_50, c03_p1_g1_50, c05_p1_g1_50,
      c06_p1_g1_50, c09_p1_g1_50, c10_p1_g1_50, c11_p1_g1_50, c12_p1_g1_50, c13_p1_g1_50, c14_p1_g1_50, c15_p1_g1_50,
      c16_p1_g1_50, c17_p1_g1_50, c18_p1_g1_50]})
#to prevent an extra column that numbers each row:
c_scores_1_50.set_index('participant', inplace=True, drop=True)

#list out all the scores in game 2 by participant
c_scores_2_50 = pd.DataFrame({'participant':['c02', 'c03', 
                              'c05', 'c06', 'c09', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17','c18'], 'score_1_50':
     [c02_p1_g2_50, c03_p1_g2_50, c05_p1_g2_50,
      c06_p1_g2_50, c09_p1_g2_50, c10_p1_g2_50, c11_p1_g2_50, c12_p1_g2_50, c13_p1_g2_50, c14_p1_g2_50, 
      c15_p1_g2_50,c16_p1_g2_50, c17_p1_g2_50, c18_p1_g2_50]})
c_scores_2_50.set_index('participant', inplace=True, drop=True)'''

In [ ]:
#c_scores1_box = c_scores_1.boxplot()
#plt.grid(b=None)
#plt.yticks(np.arange(50, 105, step=10))


c_scores1_box = sns.boxplot( 
                 data=c_scores_1, 
                 width=0.5,
                 palette="pastel")
c_scores1_box.axes.set_title("SMAB4 Control Scores in Game 1",
                    fontsize=16)

plot_file_name="c_g1_score_box.jpg"
 
plt.ylim(0, 100)
c_scores1_box.axes.set(ylabel='Score')

# save as jpeg
c_scores1_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
c_scores_2d_box = sns.boxplot( 
                 data=c_scores_2_d, 
                 width=0.5,
                 palette="pastel")
c_scores_2d_box.axes.set_title("SMAB4 Control Scores in Game 1-Different",
                    fontsize=16)

plot_file_name="c_g2d_score_box.jpg"
 
plt.ylim(0, 100)
c_scores_2d_box.axes.set(ylabel='Score')

# save as jpeg
c_scores_2d_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
c_scores_2s_box = sns.boxplot( 
                 data=c_scores_2_s, 
                 width=0.5,
                 palette="pastel")
c_scores_2s_box.axes.set_title("SMAB4 Control Scores in Game 1-Same",
                    fontsize=16)

plot_file_name="c_g2s_score_box.jpg"
 
plt.ylim(0, 100)
c_scores_2s_box.axes.set(ylabel='Score')

# save as jpeg
c_scores_2s_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
c_scores_2s_mean = c_scores_2_s.mean()

In [ ]:
c_scores_2s_std = c_scores_2_s.std()

In [ ]:
c_scores_2d_mean = c_scores_2_d.mean()

In [ ]:
c_scores_2d_std = c_scores_2_d.std()

In [ ]:
c_scores_2s_mean, c_scores_2s_std

In [ ]:
c_scores_2d_mean, c_scores_2d_std

In [ ]:
sp.stats.ttest_ind(c_scores_2_s, c_scores_2_d)

In [ ]:
'''Here we get the mean, median, and standard deviation of the scores in games 1 and 2.'''

c_scores_1_mean = c_scores_1.mean()


c_scores_1_mean

In [ ]:
c_scores_1_std = c_scores_1.std()


c_scores_1_std

In [ ]:
c_scores_1_mean, c_scores_1_std

In [ ]:
'''Here we separate the dataframes into games 1 and 2.'''

#s01_p1_g1, s01_p1_g2 = separate_df(s01, 's')
c02_p1_g1, c02_p1_g2 = separate_df(c02, 'c')
c03_p1_g1, c03_p1_g2 = separate_df(c03, 'c')
c05_p1_g1, c05_p1_g2 = separate_df(c05, 'c')
c06_p1_g1, c06_p1_g2 = separate_df(c06, 'c')
c07_p1_g1, c07_p1_g2 = separate_df(c07, 'c')
c09_p1_g1, c09_p1_g2 = separate_df(c09, 'c')
c10_p1_g1, c10_p1_g2 = separate_df(c10, 'c')
c11_p1_g1, c11_p1_g2 = separate_df(c11, 'c')
c12_p1_g1, c12_p1_g2 = separate_df(c12, 'c')
c13_p1_g1, c13_p1_g2 = separate_df(c13, 'c')
c14_p1_g1, c14_p1_g2 = separate_df(c14, 'c')
c15_p1_g1, c15_p1_g2 = separate_df(c15, 'c')
c16_p1_g1, c16_p1_g2 = separate_df(c16, 'c')
c17_p1_g1, c17_p1_g2 = separate_df(c17, 'c')
c18_p1_g1, c18_p1_g2 = separate_df(c18, 'c')

In [ ]:
c02_p1_g1_map = hole_prob_map(c02_p1_g1)
c02_p1_g2_map = hole_prob_map(c02_p1_g2)

c03_p1_g1_map = hole_prob_map(c03_p1_g1)
c03_p1_g2_map = hole_prob_map(c03_p1_g2)

c07_p1_g1_map = hole_prob_map(c07_p1_g1)
c07_p1_g2_map = hole_prob_map(c07_p1_g2)

c05_p1_g1_map = hole_prob_map(c05_p1_g1)
c05_p1_g2_map = hole_prob_map(c05_p1_g2)

c06_p1_g1_map = hole_prob_map(c06_p1_g1)
c06_p1_g2_map = hole_prob_map(c06_p1_g2)

c09_p1_g1_map = hole_prob_map(c09_p1_g1)
c09_p1_g2_map = hole_prob_map(c09_p1_g2)

c10_p1_g1_map = hole_prob_map(c10_p1_g1)
c10_p1_g2_map = hole_prob_map(c10_p1_g2)

c11_p1_g1_map = hole_prob_map(c11_p1_g1)
c11_p1_g2_map = hole_prob_map(c11_p1_g2)

c12_p1_g1_map = hole_prob_map(c12_p1_g1)
c12_p1_g2_map = hole_prob_map(c12_p1_g2)

c13_p1_g1_map = hole_prob_map(c13_p1_g1)
c13_p1_g2_map = hole_prob_map(c13_p1_g2)

c14_p1_g1_map = hole_prob_map(c14_p1_g1)
c14_p1_g2_map = hole_prob_map(c14_p1_g2)

c15_p1_g1_map = hole_prob_map(c15_p1_g1)
c15_p1_g2_map = hole_prob_map(c15_p1_g2)

c16_p1_g1_map = hole_prob_map(c16_p1_g1)
c16_p1_g2_map = hole_prob_map(c16_p1_g2)

c17_p1_g1_map = hole_prob_map(c17_p1_g1)
c17_p1_g2_map = hole_prob_map(c17_p1_g2)

c18_p1_g1_map = hole_prob_map(c18_p1_g1)
c18_p1_g2_map = hole_prob_map(c18_p1_g2)

In [ ]:
#call fix_reward on all of the files imported with '300' to fix the reward column
c12_p1_g1 = fix_reward(c12_p1_g1)
c12_p1_g2 = fix_reward(c12_p1_g2)
c13_p1_g1 = fix_reward(c13_p1_g1)
c13_p1_g2 = fix_reward(c13_p1_g2)
c14_p1_g1 = fix_reward(c14_p1_g1)
c14_p1_g2 = fix_reward(c14_p1_g2)
c15_p1_g1 = fix_reward(c15_p1_g1)
c15_p1_g2 = fix_reward(c15_p1_g2)
c16_p1_g1 = fix_reward(c16_p1_g1)
c16_p1_g2 = fix_reward(c16_p1_g2)
c17_p1_g1 = fix_reward(c17_p1_g1)
c17_p1_g2 = fix_reward(c17_p1_g2)
c18_p1_g1 = fix_reward(c18_p1_g1)
c18_p1_g2 = fix_reward(c18_p1_g2)


In [ ]:
'''Here we count the number of times each hole was chosen in each game and place it in an array.'''

c01_p1_g1_count = c01_p1_g1['probability'].value_counts(sort=False)
c01_p1_g2_count = c01_p1_g2['probability'].value_counts(sort=False)
c02_p1_g1_count = c02_p1_g1['probability'].value_counts(sort=False)
c02_p1_g2_count = c02_p1_g2['probability'].value_counts(sort=False)
c03_p1_g1_count = c03_p1_g1['probability'].value_counts(sort=False)
c03_p1_g2_count = c03_p1_g2['probability'].value_counts(sort=False)
c04_p1_g1_count = c04_p1_g1['probability'].value_counts(sort=False)
c04_p1_g2_count = c04_p1_g2['probability'].value_counts(sort=False)
c05_p1_g1_count = c05_p1_g1['probability'].value_counts(sort=False)
c05_p1_g2_count = c05_p1_g2['probability'].value_counts(sort=False)
c06_p1_g1_count = c06_p1_g1['probability'].value_counts(sort=False)
c06_p1_g2_count = c06_p1_g2['probability'].value_counts(sort=False)
c07_p1_g1_count = c07_p1_g1['probability'].value_counts(sort=False)
c07_p1_g2_count = c07_p1_g2['probability'].value_counts(sort=False)
c09_p1_g1_count = c09_p1_g1['probability'].value_counts(sort=False)
c09_p1_g2_count = c09_p1_g2['probability'].value_counts(sort=False)
c10_p1_g1_count = c10_p1_g1['probability'].value_counts(sort=False)
c10_p1_g2_count = c10_p1_g2['probability'].value_counts(sort=False)
c11_p1_g1_count = c11_p1_g1['probability'].value_counts(sort=False)
c11_p1_g2_count = c11_p1_g2['probability'].value_counts(sort=False)
c12_p1_g1_count = c12_p1_g1['probability'].value_counts(sort=False)
c12_p1_g2_count = c12_p1_g2['probability'].value_counts(sort=False)
c13_p1_g1_count = c13_p1_g1['probability'].value_counts(sort=False)
c13_p1_g2_count = c13_p1_g2['probability'].value_counts(sort=False)
c14_p1_g1_count = c14_p1_g1['probability'].value_counts(sort=False)
c14_p1_g2_count = c14_p1_g2['probability'].value_counts(sort=False)
c15_p1_g1_count = c15_p1_g1['probability'].value_counts(sort=False)
c15_p1_g2_count = c15_p1_g2['probability'].value_counts(sort=False)
c16_p1_g1_count = c16_p1_g1['probability'].value_counts(sort=False)
c16_p1_g2_count = c16_p1_g2['probability'].value_counts(sort=False)
c17_p1_g1_count = c17_p1_g1['probability'].value_counts(sort=False)
c17_p1_g2_count = c17_p1_g2['probability'].value_counts(sort=False)
c18_p1_g1_count = c18_p1_g1['probability'].value_counts(sort=False)
c18_p1_g2_count = c18_p1_g2['probability'].value_counts(sort=False)

In [ ]:
'''Here we place the number of times each hole was chosen by each subject in game 1 and into a dataframe.
We also plot this data and get the means and standard deviations of it.'''


c_g1_counts = pd.DataFrame([c01_p1_g1_count, c02_p1_g1_count, c03_p1_g1_count,
                            c05_p1_g1_count, c06_p1_g1_count, c07_p1_g1_count, c09_p1_g1_count, c10_p1_g1_count, c11_p1_g1_count, c12_p1_g1_count, c13_p1_g1_count, c14_p1_g1_count, c15_p1_g1_count,
                            c16_p1_g1_count, c17_p1_g1_count, c18_p1_g1_count],
         index=['c01_p1_g1_count', 'c02_p1_g1_count', 'c03_p1_g1_count', 'c05_p1_g1_count', 'c06_p1_g1_count', 
                'c07_p1_g1_count','c09_p1_g1_count', 'c10_p1_g1_count', 'c11_p1_g1_count', 'c12_p1_g1_count', 'c13_p1_g1_count', 'c14_p1_g1_count', 
                'c15_p1_g1_count', 'c16_p1_g1_count', 'c17_p1_g1_count', 'c18_p1_g1_count'])


#How to prevent NaN: df = df.fillna(0)
c_g1_counts = c_g1_counts.fillna(0)

#print
c_g1_counts


c_g1_counts.sum(axis=1)


c_g1_counts_bar = c_g1_counts.plot.bar()

c_g1_counts_bar.axes.set_title("SMAB4 Control Choice Distributions in Game 1",
                    fontsize=16)

# save as jpeg
c_g1_counts_bar.figure.savefig('c_g1_counts_bar.jpg',
                    format='jpeg',
                    dpi=100)

In [ ]:
c_g1_counts_box = sns.boxplot( 
                 data=c_g1_counts, 
                 width=0.5,
                 palette="pastel")
c_g1_counts_box.axes.set_title("SMAB4 Control Choice Distributions in Game 1",
                    fontsize=16)

plot_file_name="c_g1_counts_box.jpg"
 
plt.ylim(0, 100)
c_g1_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
c_g1_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
# In[158]:


#0 is low (30), 1 is med (60), 2 is high (90)
c_g1_low = np.mean(c_g1_counts.iloc[:,0])
c_g1_med = np.mean(c_g1_counts.iloc[:,1])
c_g1_high = np.mean(c_g1_counts.iloc[:,2])

In [ ]:
c_g1_low_std = np.std(c_g1_counts.iloc[:,0])
c_g1_med_std = np.std(c_g1_counts.iloc[:,1])
c_g1_high_std = np.std(c_g1_counts.iloc[:,2])

In [ ]:
c_g1_low, c_g1_med, c_g1_high

In [ ]:
c_g1_low_std, c_g1_med_std, c_g1_high_std

In [ ]:
'''Here we place the number of times each hole was chosen by each subject in game 2 and into a dataframe.
We also plot this data and get the means and standard deviations of it.'''

c_g2_d_counts = pd.DataFrame([c02_p1_g2_count, c03_p1_g2_count,
                            c05_p1_g2_count, c06_p1_g2_count, c07_p1_g2_count, c09_p1_g2_count, c10_p1_g2_count, c11_p1_g2_count],
         index=['c02_p1_g2_count', 'c03_p1_g2_count', 'c05_p1_g2_count', 'c06_p1_g2_count', 'c07_p1_g2_count',
                'c09_p1_g2_count', 'c10_p1_g2_count', 'c11_p1_g2_count'])


#How to prevent NaN: df = df.fillna(0)
c_g2_d_counts = c_g2_d_counts.fillna(0)

#print
c_g2_d_counts


c_g2_d_counts.sum(axis=1)


c_g2_d_counts_bar = c_g2_d_counts.plot.bar()

# save as jpeg
c_g2_d_counts_bar.figure.savefig(plot_file_name,
                    format='jpeg',
                    dpi=100)

In [ ]:
'''Here we place the number of times each hole was chosen by each subject in game 2 and into a dataframe.
We also plot this data and get the means and standard deviations of it.'''

c_g2_s_counts = pd.DataFrame([c01_p1_g2_count, c12_p1_g2_count, c13_p1_g2_count, c14_p1_g2_count, c15_p1_g2_count, c16_p1_g2_count, c17_p1_g2_count, c18_p1_g2_count],
         index=['c01_p1_g2_count', 'c12_p1_g2_count', 'c13_p1_g2_count', 
                'c14_p1_g2_count', 'c15_p1_g2_count',
                'c16_p1_g2_count', 'c17_p1_g2_count', 'c18_p1_g2_count'])


#How to prevent NaN: df = df.fillna(0)
c_g2_s_counts = c_g2_s_counts.fillna(0)

#print
c_g2_s_counts


c_g2_s_counts.sum(axis=1)


c_g2_s_counts_bar = c_g2_s_counts.plot.bar()

# save as jpeg
c_g2_s_counts_bar.figure.savefig(plot_file_name,
                    format='jpeg',
                    dpi=100)

In [ ]:
c_g2_s_counts_box = sns.boxplot( 
                 data=c_g2_s_counts, 
                 width=0.5,
                 palette="pastel")
c_g2_s_counts_box.axes.set_title("SMAB4 Control Choice Distributions in Game 2-Same",
                    fontsize=16)

plot_file_name="c_g2_s_counts_box.jpg"
 
plt.ylim(0, 100)
c_g2_s_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
c_g2_s_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
c_g2_d_counts_box = sns.boxplot( 
                 data=c_g2_d_counts, 
                 width=0.5,
                 palette="pastel")
c_g2_d_counts_box.axes.set_title("SMAB4 Control Choice Distributions in Game 2-Different",
                    fontsize=16)

plot_file_name="c_g2_d_counts_box.jpg"
 
plt.ylim(0, 100)
c_g2_d_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
c_g2_d_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
c_g2_s_low = np.mean(c_g2_s_counts.iloc[:,0])
c_g2_s_med = np.mean(c_g2_s_counts.iloc[:,1])
c_g2_s_high = np.mean(c_g2_s_counts.iloc[:,2])

c_g2_d_low = np.mean(c_g2_d_counts.iloc[:,0])
c_g2_d_med = np.mean(c_g2_d_counts.iloc[:,1])
c_g2_d_high = np.mean(c_g2_d_counts.iloc[:,2])

In [ ]:
c_g2_s_low_std = np.std(c_g2_s_counts.iloc[:,0])
c_g2_s_med_std = np.std(c_g2_s_counts.iloc[:,1])
c_g2_s_high_std = np.std(c_g2_s_counts.iloc[:,2])

c_g2_d_low_std = np.std(c_g2_d_counts.iloc[:,0])
c_g2_d_med_std = np.std(c_g2_d_counts.iloc[:,1])
c_g2_d_high_std = np.std(c_g2_d_counts.iloc[:,2])

In [ ]:
sp.stats.ttest_ind(c_g1_counts, c_g2_s_counts)

In [ ]:
sp.stats.ttest_ind(c_g1_counts, c_g2_d_counts)

In [ ]:
sp.stats.ttest_ind(c_g2_s_counts, c_g2_d_counts)

In [ ]:
c_g2_s_low_std, c_g2_s_med_std, c_g2_s_high_std, c_g2_d_low_std, c_g2_d_med_std, c_g2_d_high_std

# Reaction Time Analyses

In [ ]:
'Here we get the reaction times'
'NEED to fix rewards for SMAB4'
time_c_1 = pd.DataFrame(columns = ['session','post win','post loss'])
time_c_2 = pd.DataFrame(columns = ['session','post win','post loss'])

In [ ]:
time_c_1 = reaction_time(c02_p1_g1, time_c_1)
time_c_1 = reaction_time(c03_p1_g1, time_c_1)
time_c_1 = reaction_time(c05_p1_g1, time_c_1)
time_c_1 = reaction_time(c06_p1_g1, time_c_1)
time_c_1 = reaction_time(c09_p1_g1, time_c_1)
time_c_1 = reaction_time(c10_p1_g1, time_c_1)
time_c_1 = reaction_time(c11_p1_g1, time_c_1)
time_c_1 = reaction_time(c12_p1_g1, time_c_1)
time_c_1 = reaction_time(c13_p1_g1, time_c_1)
time_c_1 = reaction_time(c14_p1_g1, time_c_1)
time_c_1 = reaction_time(c15_p1_g1, time_c_1)
time_c_1 = reaction_time(c16_p1_g1, time_c_1)
time_c_1 = reaction_time(c17_p1_g1, time_c_1)
time_c_1 = reaction_time(c18_p1_g1, time_c_1)

In [ ]:
time_c_1

In [ ]:
time_c_2 = reaction_time(c02_p1_g2, time_c_2)
time_c_2 = reaction_time(c03_p1_g2, time_c_2)
time_c_2 = reaction_time(c05_p1_g2, time_c_2)
time_c_2 = reaction_time(c06_p1_g2, time_c_2)
time_c_2 = reaction_time(c09_p1_g2, time_c_2)
time_c_2 = reaction_time(c10_p1_g2, time_c_2)
time_c_2 = reaction_time(c11_p1_g2, time_c_2)
time_c_2 = reaction_time(c12_p1_g2, time_c_2)
time_c_2 = reaction_time(c13_p1_g2, time_c_2)
time_c_2 = reaction_time(c14_p1_g2, time_c_2)
time_c_2 = reaction_time(c15_p1_g2, time_c_2)
time_c_2 = reaction_time(c16_p1_g2, time_c_2)
time_c_2 = reaction_time(c17_p1_g2, time_c_2)
time_c_2 = reaction_time(c18_p1_g2, time_c_2)

In [ ]:
times_c_g1_box = sns.boxplot( 
                 data=time_c_1, 
                 width=0.5,
                 palette="pastel")
times_c_g1_box.axes.set_title("SMAB4 Control Reaction Times in Game 1",
                    fontsize=16)

plot_file_name="times_c_g1_box.jpg"
 
plt.ylim(0, 5)
times_c_g1_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_c_g1_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
times_c_g2_box = sns.boxplot( 
                 data=time_c_2, 
                 width=0.5,
                 palette="pastel")
times_c_g2_box.axes.set_title("SMAB4 Control Reaction Times in Game 2",
                    fontsize=16)

plot_file_name="times_c_g2_box.jpg"
 
plt.ylim(0, 5)
times_c_g2_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_c_g2_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
np.mean(time_c_1)

In [ ]:
np.mean(time_c_2)

In [ ]:
np.std(time_c_1)

In [ ]:
np.std(time_c_2)

In [ ]:
sp.stats.ttest_rel(time_c_1['post win'],time_c_1['post loss'])

In [ ]:
sp.stats.ttest_rel(time_c_2['post win'],time_c_2['post loss'])

In [ ]:
#not working
#sp.stats.ttest_rel(time_c_1['post win'],time_c_2['post win'])

In [ ]:
#not working
#sp.stats.ttest_rel(time_c_1['post loss'],time_c_2['post loss'])

# Maximizing vs Matching Analysis for Confederate Data

In [ ]:
import math

In [ ]:
'''
#TEST- JSD for multiple distributions
def entropy(prob_dist, base=math.e):
        return -sum([p * math.log(p,base) for p in prob_dist if p != 0])

def jsd(prob_dists, base=math.e):
    weight = 1/len(prob_dists) #all same weight
    js_left = [0,0,0]
    js_right = 0    
    for pd in prob_dists:
        js_left[0] += pd[0]*weight
        js_left[1] += pd[1]*weight
        js_left[2] += pd[2]*weight
        js_right += weight*entropy(pd,base)
    return entropy(js_left)-js_right'''

In [ ]:
#NOTE: rewrite using pandas dataframe
# In[239]:

#Call 'call_jsd' for each participant for each game in the single player sessions.
#with 0, put in actual probabilities in brackets

s01_g1_max,s01_g1_match,s01_g1_rand,s01_g1_soc = call_jsd(s01_p1_g1_count)
s01_g2_max,s01_g2_match,s01_g2_rand,s01_g2_soc = call_jsd(s01_p1_g2_count)
s03_g1_max,s03_g1_match,s03_g1_rand,s03_g1_soc = call_jsd(s03_p1_g1_count)
s03_g2_max,s03_g2_match,s03_g2_rand,s03_g2_soc = call_jsd(s03_p1_g2_count)
s05_g1_max,s05_g1_match,s05_g1_rand,s05_g1_soc= call_jsd(s05_p1_g1_count)
s05_g2_max,s05_g2_match,s05_g2_rand,s05_g2_soc = call_jsd([11, 0, 89])
s06_g1_max,s06_g1_match,s06_g1_rand,s06_g1_soc = call_jsd(s06_p1_g1_count)
s06_g2_max,s06_g2_match,s06_g2_rand,s06_g2_soc = call_jsd(s06_p1_g2_count)
s07_g1_max,s07_g1_match,s07_g1_rand,s07_g1_soc = call_jsd(s07_p1_g1_count)
s07_g2_max,s07_g2_match,s07_g2_rand,s07_g2_soc = call_jsd(s07_p1_g2_count)
s08_g1_max,s08_g1_match,s08_g1_rand,s08_g1_soc = call_jsd(s08_p1_g1_count)
s08_g2_max,s08_g2_match,s08_g2_rand,s08_g2_soc = call_jsd(s08_p1_g2_count)
s09_g1_max,s09_g1_match,s09_g1_rand,s09_g1_soc = call_jsd(s09_p1_g1_count)
s09_g2_max,s09_g2_match,s09_g2_rand,s09_g2_soc = call_jsd(s09_p1_g2_count)
s11_g1_max,s11_g1_match,s11_g1_rand,s11_g1_soc = call_jsd(s11_p1_g1_count)
s11_g2_max,s11_g2_match,s11_g2_rand,s11_g2_soc = call_jsd(s11_p1_g2_count)
s12_g1_max,s12_g1_match,s12_g1_rand,s12_g1_soc = call_jsd(s12_p1_g1_count)
s12_g2_max,s12_g2_match,s12_g2_rand,s12_g2_soc = call_jsd(s12_p1_g2_count)
s14_g1_max,s14_g1_match,s14_g1_rand,s14_g1_soc = call_jsd(s14_p1_g1_count)
s14_g2_max,s14_g2_match,s14_g2_rand,s14_g2_soc = call_jsd([2,0,98])
s15_g1_max,s15_g1_match,s15_g1_rand,s15_g1_soc = call_jsd(s15_p1_g1_count)
s15_g2_max,s15_g2_match,s15_g2_rand,s15_g2_soc = call_jsd(s15_p1_g2_count)
s16_g1_max,s16_g1_match,s16_g1_rand,s16_g1_soc = call_jsd(s16_p1_g1_count)
s16_g2_max,s16_g2_match,s16_g2_rand,s16_g2_soc = call_jsd(s16_p1_g2_count)
s17_g1_max,s17_g1_match,s17_g1_rand,s17_g1_soc = call_jsd(s17_p1_g1_count)
s17_g2_max,s17_g2_match,s17_g2_rand,s17_g2_soc = call_jsd([0,22,78])
s19_g1_max,s19_g1_match,s19_g1_rand,s19_g1_soc = call_jsd(s19_p1_g1_count)
s19_g2_max,s19_g2_match,s19_g2_rand,s19_g2_soc = call_jsd(s19_p1_g2_count)
s20_g1_max,s20_g1_match,s20_g1_rand,s20_g1_soc = call_jsd(s20_p1_g1_count)
s20_g2_max,s20_g2_match,s20_g2_rand,s20_g2_soc = call_jsd(s20_p1_g2_count)
c01_g1_max,c01_g1_match,c01_g1_rand,c01_g1_soc = call_jsd(c01_p1_g1_count)
c01_g2_max,c01_g2_match,c01_g2_rand,c01_g2_soc = call_jsd(c01_p1_g2_count)
c02_g1_max,c02_g1_match,c02_g1_rand,c02_g1_soc = call_jsd(c02_p1_g1_count)
c02_g2_max,c02_g2_match,c02_g2_rand,c02_g2_soc = call_jsd(c02_p1_g2_count)
c03_g1_max,c03_g1_match,c03_g1_rand,c03_g1_soc = call_jsd(c03_p1_g1_count)
c03_g2_max,c03_g2_match,c03_g2_rand,c03_g2_soc = call_jsd(c03_p1_g2_count)
c04_g1_max,c04_g1_match,c04_g1_rand,c04_g1_soc = call_jsd(c04_p1_g1_count)
c04_g2_max,c04_g2_match,c04_g2_rand,c04_g2_soc = call_jsd(c04_p1_g2_count)
c05_g1_max,c05_g1_match,c05_g1_rand,c05_g1_soc = call_jsd(c05_p1_g1_count)
c05_g2_max,c05_g2_match,c05_g2_rand,c05_g2_soc = call_jsd(c05_p1_g2_count)
c06_g1_max,c06_g1_match,c06_g1_rand,c06_g1_soc = call_jsd(c06_p1_g1_count)
c06_g2_max,c06_g2_match,c06_g2_rand,c06_g2_soc = call_jsd(c06_p1_g1_count)
c07_g1_max,c07_g1_match,c07_g1_rand,c07_g1_soc = call_jsd(c07_p1_g1_count)
c07_g2_max,c07_g2_match,c07_g2_rand,c07_g2_soc = call_jsd(c07_p1_g1_count)
c09_g1_max,c09_g1_match,c09_g1_rand,c09_g1_soc = call_jsd(c09_p1_g1_count)
c09_g2_max,c09_g2_match,c09_g2_rand,c09_g2_soc = call_jsd(c09_p1_g2_count)
c10_g1_max,c10_g1_match,c10_g1_rand,c10_g1_soc = call_jsd(c10_p1_g1_count)
c10_g2_max,c10_g2_match,c10_g2_rand,c10_g2_soc = call_jsd(c10_p1_g2_count)
c11_g1_max,c11_g1_match,c11_g1_rand,c11_g1_soc = call_jsd(c11_p1_g1_count)
c11_g2_max,c11_g2_match,c11_g2_rand,c11_g2_soc= call_jsd(c11_p1_g2_count)
c12_g1_max,c12_g1_match,c12_g1_rand,c12_g1_soc = call_jsd(c12_p1_g1_count)
c12_g2_max,c12_g2_match,c12_g2_rand,c12_g2_soc = call_jsd(c12_p1_g2_count)
c13_g1_max,c13_g1_match,c13_g1_rand,c13_g1_soc  = call_jsd(c13_p1_g1_count)
c13_g2_max,c13_g2_match,c13_g2_rand,c13_g2_soc = call_jsd(c13_p1_g2_count)
c14_g1_max,c14_g1_match,c14_g1_rand,c14_g1_soc = call_jsd(c14_p1_g1_count)
c14_g2_max,c14_g2_match,c14_g2_rand,c14_g2_soc = call_jsd(c14_p1_g2_count)
c16_g1_max,c16_g1_match,c16_g1_rand,c16_g1_soc = call_jsd(c16_p1_g1_count)
c16_g2_max,c16_g2_match,c16_g2_rand,c16_g2_soc = call_jsd(c16_p1_g2_count)
c17_g1_max,c17_g1_match,c17_g1_rand,c17_g1_soc = call_jsd(c17_p1_g1_count)
c17_g2_max,c17_g2_match,c17_g2_rand,c17_g2_soc = call_jsd(c17_p1_g2_count)
c18_g1_max,c18_g1_match,c18_g1_rand,c18_g1_soc = call_jsd(c18_p1_g1_count)
c18_g2_max,c18_g2_match,c18_g2_rand,c18_g2_soc = call_jsd(c18_p1_g2_count)

In [ ]:
s01_g1_max

In [ ]:
s01_g1_match

In [ ]:
s01_p1_g2_count

In [ ]:
s01_g2_max

In [ ]:
# In[ ]:


#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
control_jsd_g1 = pd.DataFrame({'max':[s01_g1_max,s03_g1_max,s05_g1_max,s06_g1_max,s07_g1_max,s08_g1_max,
                                s09_g1_max, s11_g1_max, s12_g1_max, s14_g1_max, s15_g1_max, s16_g1_max, s17_g1_max, 
                                      s19_g1_max, s20_g1_max, c01_g1_max, c02_g1_max, c03_g1_max, c05_g1_max, c06_g1_max, 
                                      c07_g1_max, c09_g1_max, c10_g1_max, c11_g1_max, c12_g1_max, c13_g1_max, c14_g1_max, 
                                      c16_g1_max, c17_g1_max, c18_g1_max],
                      'match':[s01_g1_match,s03_g1_match,s05_g1_match
                               ,s06_g1_match,s07_g1_match,s08_g1_match,s09_g1_match, s11_g1_match, 
                               s12_g1_match, s14_g1_match, s15_g1_match, s16_g1_match, s17_g1_match, s19_g1_match, 
                               s20_g1_match, c01_g1_match, c02_g1_match, c03_g1_match, c05_g1_match, c06_g1_match, 
                                      c07_g1_match, c09_g1_match, c10_g1_match, c11_g1_match, c12_g1_match, c13_g1_match, c14_g1_match, 
                                      c16_g1_match, c17_g1_match, c18_g1_match],
                    'rand':[s01_g1_rand,s03_g1_rand,s05_g1_rand,s06_g1_rand,s07_g1_rand,s08_g1_rand,
                                s09_g1_rand, s11_g1_rand, s12_g1_rand, s14_g1_rand, s15_g1_rand, s16_g1_rand, s17_g1_rand, 
                                      s19_g1_rand, s20_g1_rand, c01_g1_rand, c02_g1_rand, c03_g1_rand, c05_g1_rand, c06_g1_rand, 
                                      c07_g1_rand, c09_g1_rand, c10_g1_rand, c11_g1_rand, c12_g1_rand, c13_g1_rand, c14_g1_rand, 
                                      c16_g1_rand, c17_g1_rand, c18_g1_rand],
                        'soc':[s01_g1_soc,s03_g1_soc,s05_g1_soc,s06_g1_soc,s07_g1_soc,s08_g1_soc,
                                s09_g1_soc, s11_g1_soc, s12_g1_soc, s14_g1_soc, s15_g1_soc, s16_g1_soc, s17_g1_soc, 
                                      s19_g1_soc, s20_g1_soc, c01_g1_soc, c02_g1_soc, c03_g1_soc, c05_g1_soc, c06_g1_soc, 
                                      c07_g1_soc, c09_g1_soc, c10_g1_soc, c11_g1_soc, c12_g1_soc, c13_g1_soc, c14_g1_soc, 
                                      c16_g1_soc, c17_g1_soc, c18_g1_soc]},
                              
                        index=['s01', 's03','s05','s06','s07','s08','s09', 's11', 's12', 's14', 's15', 's16', 's17', 's19', 's20', 'c01', 'c02', 'c03', 
                               'c05', 'c06', 'c07','c09', 'c10', 'c11', 'c12', 'c13', 'c14', 'c16', 'c17', 'c18'])

In [ ]:
control_jsd_g1

In [ ]:
# In[ ]:


#closer to 0 = closer to that pure strategy
control_jsd_g1



# get the column name of min values in every row
strategy_control_jsd_g1 = control_jsd_g1.idxmin(axis=1)
 
print("min values of row are at following columns :")
print(strategy_control_jsd_g1)


# In[ ]:


strategy_control_jsd_g1.value_counts()

In [ ]:
#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
control_jsd_g2d = pd.DataFrame({'max':[s01_g2_max,s03_g2_max,s05_g2_max,s06_g2_max,s07_g2_max,s08_g2_max,
                                s09_g2_max, c02_g2_max, c03_g2_max, c05_g2_max, c06_g2_max, c07_g2_max,
                                      c09_g2_max, c10_g2_max, c11_g2_max],
                      'match':[s01_g2_match,s03_g2_match,s05_g2_match,
                               s06_g2_match,s07_g2_match,s08_g2_match,s09_g2_match, 
                                c02_g2_match, c03_g2_match, c05_g2_match, c06_g2_match, c07_g2_match,
                                      c09_g2_match, c10_g2_match, c11_g2_match],
                    'rand':[s01_g2_rand,s03_g2_rand,s05_g2_rand,s06_g2_rand,s07_g2_rand,s08_g2_rand,
                                s09_g2_rand, c02_g2_rand, c03_g2_rand, c05_g2_rand, c06_g2_rand, c07_g2_rand, 
                                      c09_g2_rand, c10_g2_rand, c11_g2_rand],
                        'soc':[s01_g2_soc,s03_g2_soc,s05_g2_soc,s06_g2_soc,s07_g2_soc,s08_g2_soc,
                                s09_g2_soc, c02_g2_soc, c03_g2_soc, c05_g2_soc, c06_g2_soc, c07_g2_soc,
                                      c09_g2_soc, c10_g2_soc, c11_g2_soc]},    
                        index=['s01', 's03','s05','s06','s07','s08','s09', 'c02', 'c03', 
                               'c05', 'c06', 'c07', 'c09', 'c10', 'c11'])

In [ ]:
#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
control_jsd_g2s = pd.DataFrame({'max':[s11_g2_max, s12_g2_max, s14_g2_max, s15_g2_max, s16_g2_max, s17_g2_max, 
                                      s19_g2_max, s20_g2_max, c01_g2_max, c12_g2_max, c13_g2_max, c14_g2_max, 
                                      c16_g2_max, c17_g2_max, c18_g2_max],
                      'match':[s11_g2_match, s12_g2_match, s14_g2_match, s15_g2_match, s16_g2_match, s17_g2_match, s19_g2_match, 
                               s20_g2_match, c01_g2_match, c12_g2_match, c13_g2_match, c14_g2_match, 
                                      c16_g2_match, c17_g2_match, c18_g2_match],
                    'rand':[s11_g2_rand, s12_g2_rand, s14_g2_rand, s15_g2_rand, s16_g2_rand, s17_g2_rand, 
                                      s19_g2_rand, s20_g2_rand, c01_g2_rand, c12_g2_rand, c13_g2_rand, c14_g2_rand, 
                                      c16_g2_rand, c17_g2_rand, c18_g2_rand],
                        'soc':[s11_g2_soc, s12_g2_soc, s14_g2_soc, s15_g2_soc, s16_g2_soc, s17_g2_soc, 
                                      s19_g2_soc, s20_g2_soc, c01_g2_soc, c12_g2_soc, c13_g2_soc, c14_g2_soc, 
                                      c16_g2_soc, c17_g2_soc, c18_g2_soc]},
                              
                        index=['s11', 's12', 's14','s15', 's16', 's17', 's19', 's20', 'c01', 'c12', 'c13', 'c14', 'c16', 'c17', 'c18'])

In [ ]:
control_jsd_g2d

# get the column name of min values in every row
strategy_control_jsd_g2d = control_jsd_g2d.idxmin(axis=1)
 
print("min values of row are at following columns :")
print(strategy_control_jsd_g2d)



minValues_control_jsd_g2d = control_jsd_g2d.min()
 
print('minimum value in each column : ')
print(minValues_control_jsd_g2d)


# In[ ]:


maxValues_control_jsd_g2d = control_jsd_g2d.max()
 
print('maximum value in each column : ')
print(maxValues_control_jsd_g2d)


# In[ ]:


strategy_control_jsd_g2d.value_counts()


In [ ]:
control_jsd_g2s

# get the column name of min values in every row
strategy_control_jsd_g2s = control_jsd_g2s.idxmin(axis=1)
 
print("min values of row are at following columns :")
print(strategy_control_jsd_g2s)



minValues_control_jsd_g2s = control_jsd_g2s.min()
 
print('minimum value in each column : ')
print(minValues_control_jsd_g2s)


# In[ ]:


maxValues_control_jsd_g2s = control_jsd_g2s.max()
 
print('maximum value in each column : ')
print(maxValues_control_jsd_g2s)


# In[ ]:


strategy_control_jsd_g2s.value_counts()


In [ ]:
# In[ ]:


#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g1['max'], control_jsd_g1['match'], 1)

control_ego_jsd_g1_plot = control_jsd_g1.plot.scatter(x='max',y='match',s=50)
plt.title('Control Egocentric Strategies Game 1', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('control_ego_jsd_1.jpeg',format='jpeg')
plt.show()

In [ ]:
# In[ ]:


#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g2d['max'], control_jsd_g2d['match'], 1)

control_ego_jsd_g2d_plot = control_jsd_g2d.plot.scatter(x='max',y='match',s=50)
plt.title('Control Egocentric Strategies Game 2- Different', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('control_ego_jsd_2d.jpg',format='jpg')
plt.show()

In [ ]:
#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g2s['max'], control_jsd_g2s['match'], 1)

control_ego_jsd_g2s_plot = control_jsd_g2s.plot.scatter(x='max',y='match',s=50)
plt.title('Control Egocentric Strategies Game 2- Same', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('control_ego_jsd_2s.jpg',format='jpg')
plt.show()

In [ ]:
control_jsd_g2 = control_jsd_g2s.append(control_jsd_g2d)
#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g2['max'], control_jsd_g2['match'], 1)

control_ego_jsd_g2_plot = sns.scatterplot(data=control_jsd_g2,x='max',y='match',s=50,alpha=0.5)
plt.title('Control Egocentric Strategies Game 2- Combined', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('control_ego_jsd_2.jpg',format='jpg')

#add g1 on same plot
sns.scatterplot(data=control_jsd_g1, x='max',y='match',s=50,alpha=0.5)
plt.show()

In [ ]:
sp.stats.ttest_ind(control_jsd_g1, control_jsd_g2s)

In [ ]:
control_jsd_g2 = control_jsd_g2s.append(control_jsd_g2d)

In [ ]:
sp.stats.ttest_ind(control_jsd_g2s,control_jsd_g2d)

In [ ]:
sp.stats.ttest_ind(control_jsd_g1,control_jsd_g2)

# SMAB3 Joint Analysis

In [33]:
j_trial_by_trial = ['j01','j02','j03','j04','j05','j06','j07','j08']
j_three = ['j10','j11']

In [34]:
j_trial_by_trial_dict = imports(sessions = j_trial_by_trial, condition = 'j')

j01
j02
j03
j04
j05
j06
j07
j08


In [36]:
j_three_dict = imports(sessions = j_three, condition = 'j', types='three')

j10
j11


In [37]:
#combine dictionaries into one
j_dict = {**j_trial_by_trial_dict, **j_three_dict}

In [ ]:
'''Here we separate the dataframes by suject and game by calling separate_df. We also create arrays with the number of
times that each subject chose each hole by calling value_counts.'''

j01_p1_g1, j01_p1_g2, j01_p2_g1, j01_p2_g2 = separate_df(j01, 'j')
j02_p1_g1, j02_p1_g2, j02_p2_g1, j02_p2_g2 = separate_df(j02, 'j')
j03_p1_g1, j03_p1_g2, j03_p2_g1, j03_p2_g2 = separate_df(j03, 'j')
j04_p1_g1, j04_p1_g2, j04_p2_g1, j04_p2_g2 = separate_df(j04, 'j')
j05_p1_g1, j05_p1_g2, j05_p2_g1, j05_p2_g2 = separate_df(j05, 'j')
j06_p1_g1, j06_p1_g2, j06_p2_g1, j06_p2_g2 = separate_df(j06, 'j')
j07_p1_g1, j07_p1_g2, j07_p2_g1, j07_p2_g2 = separate_df(j07, 'j')
j08_p1_g1, j08_p1_g2, j08_p2_g1, j08_p2_g2 = separate_df(j08, 'j')
j10_p1_g1, j10_p1_g2, j10_p2_g1, j10_p2_g2 = separate_df(j10, 'j')
j11_p1_g1, j11_p1_g2, j11_p2_g1, j11_p2_g2 = separate_df(j11, 'j')



j01_p1_g1_count = j01_p1_g1['probability'].value_counts(sort=False)
j01_p1_g2_count = j01_p1_g2['probability'].value_counts(sort=False)
j01_p2_g1_count = j01_p2_g1['probability'].value_counts(sort=False)
j01_p2_g2_count = j01_p2_g2['probability'].value_counts(sort=False)
j02_p1_g1_count = j02_p1_g1['probability'].value_counts(sort=False)
j02_p1_g2_count = j02_p1_g2['probability'].value_counts(sort=False)
j02_p2_g1_count = j02_p2_g1['probability'].value_counts(sort=False)
j02_p2_g2_count = j02_p2_g2['probability'].value_counts(sort=False)
j03_p1_g1_count = j03_p1_g1['probability'].value_counts(sort=False)
j03_p1_g2_count = j03_p1_g2['probability'].value_counts(sort=False)
j03_p2_g1_count = j03_p2_g1['probability'].value_counts(sort=False)
j03_p2_g2_count = j03_p2_g2['probability'].value_counts(sort=False)
j04_p1_g1_count = j04_p1_g1['probability'].value_counts(sort=False)
j04_p1_g2_count = j04_p1_g2['probability'].value_counts(sort=False)
j04_p2_g1_count = j04_p2_g1['probability'].value_counts(sort=False)
j04_p2_g2_count = j04_p2_g2['probability'].value_counts(sort=False)
j05_p1_g1_count = j05_p1_g1['probability'].value_counts(sort=False)
j05_p1_g2_count = j05_p1_g2['probability'].value_counts(sort=False)
j05_p2_g1_count = j05_p2_g1['probability'].value_counts(sort=False)
j05_p2_g2_count = j05_p2_g2['probability'].value_counts(sort=False)
j06_p1_g1_count = j06_p1_g1['probability'].value_counts(sort=False)
j06_p1_g2_count = j06_p1_g2['probability'].value_counts(sort=False)
j06_p2_g1_count = j06_p2_g1['probability'].value_counts(sort=False)
j06_p2_g2_count = j06_p2_g2['probability'].value_counts(sort=False)
j07_p1_g1_count = j07_p1_g1['probability'].value_counts(sort=False)
j07_p1_g2_count = j07_p1_g2['probability'].value_counts(sort=False)
j07_p2_g1_count = j07_p2_g1['probability'].value_counts(sort=False)
j07_p2_g2_count = j07_p2_g2['probability'].value_counts(sort=False)
j08_p1_g1_count = j08_p1_g1['probability'].value_counts(sort=False)
j08_p1_g2_count = j08_p1_g2['probability'].value_counts(sort=False)
j08_p2_g1_count = j08_p2_g1['probability'].value_counts(sort=False)
j08_p2_g2_count = j08_p2_g2['probability'].value_counts(sort=False)
j10_p1_g1_count = j10_p1_g1['probability'].value_counts(sort=False)
j10_p1_g2_count = j10_p1_g2['probability'].value_counts(sort=False)
j10_p2_g1_count = j10_p2_g1['probability'].value_counts(sort=False)
j10_p2_g2_count = j10_p2_g2['probability'].value_counts(sort=False)
j11_p1_g1_count = j11_p1_g1['probability'].value_counts(sort=False)
j11_p1_g2_count = j11_p1_g2['probability'].value_counts(sort=False)
j11_p2_g1_count = j11_p2_g1['probability'].value_counts(sort=False)
j11_p2_g2_count = j11_p2_g2['probability'].value_counts(sort=False)

In [ ]:
'''Here we mech the hole-probability mappings in each game for each player'''
j01_p1_g1_map = hole_prob_map(j01_p1_g1)
j01_p1_g2_map = hole_prob_map(j01_p1_g2)
j01_p2_g1_map = hole_prob_map(j01_p2_g1)
j01_p2_g2_map = hole_prob_map(j01_p2_g2)

j02_p1_g1_map = hole_prob_map(j02_p1_g1)
j02_p1_g2_map = hole_prob_map(j02_p1_g2)
j02_p2_g1_map = hole_prob_map(j02_p2_g1)
j02_p2_g2_map = hole_prob_map(j02_p2_g2)

j03_p1_g1_map = hole_prob_map(j03_p1_g1)
j03_p1_g2_map = hole_prob_map(j03_p1_g2)
j03_p2_g1_map = hole_prob_map(j03_p2_g1)
j03_p2_g2_map = hole_prob_map(j03_p2_g2)

j04_p1_g1_map = hole_prob_map(j04_p1_g1)
j04_p1_g2_map = hole_prob_map(j04_p1_g2)
j04_p2_g1_map = hole_prob_map(j04_p2_g1)
j04_p2_g2_map = hole_prob_map(j04_p2_g2)

j05_p1_g1_map = hole_prob_map(j05_p1_g1)
j05_p1_g2_map = hole_prob_map(j05_p1_g2)
j05_p2_g1_map = hole_prob_map(j05_p2_g1)
j05_p2_g2_map = hole_prob_map(j05_p2_g2)

j06_p1_g1_map = hole_prob_map(j06_p1_g1)
j06_p1_g2_map = hole_prob_map(j06_p1_g2)
j06_p2_g1_map = hole_prob_map(j06_p2_g1)
j06_p2_g2_map = hole_prob_map(j06_p2_g2)

j07_p1_g1_map = hole_prob_map(j07_p1_g1)
j07_p1_g2_map = hole_prob_map(j07_p1_g2)
j07_p2_g1_map = hole_prob_map(j07_p2_g1)
j07_p2_g2_map = hole_prob_map(j07_p2_g2)

j08_p1_g1_map = hole_prob_map(j08_p1_g1)
j08_p1_g2_map = hole_prob_map(j08_p1_g2)
j08_p2_g1_map = hole_prob_map(j08_p2_g1)
j08_p2_g2_map = hole_prob_map(j08_p2_g2)

j10_p1_g1_map = hole_prob_map(j10_p1_g1)
j10_p1_g2_map = hole_prob_map(j10_p1_g2)
j10_p2_g1_map = hole_prob_map(j10_p2_g1)
j10_p2_g2_map = hole_prob_map(j10_p2_g2)

#j11_p1_g1_map = hole_prob_map(j11_p1_g1)
#j11_p1_g2_map = hole_prob_map(j11_p1_g2)
#j11_p2_g1_map = hole_prob_map(j11_p2_g1)
#j11_p2_g2_map = hole_prob_map(j11_p2_g2)

In [ ]:
j10_p2_g1_map

In [ ]:
'''Here we create dataframes with all of the scores of each subject in games 1 and 2. 50 refers to the score half-way
through the games'''


# # Combined Joint scores G1

j_scores_1 = pd.DataFrame({'participant':['j01_p1', 'j01_p2','j02_p1', 'j02_p2','j03_p1', 'j03_p2','j04_p1', 'j04_p2','j05_p1', 'j05_p2','j06_p1','j06_p2','j07_p1', 'j07_p2','j08_p1', 'j08_p2','j10_p1', 'j10_p2','j11_p1', 'j11_p2'], 'score_1':
     [j01_p1_g1_score, j01_p2_g1_score, j02_p1_g1_score, j02_p2_g1_score, j03_p1_g1_score, j03_p2_g1_score,
      j04_p1_g1_score, j04_p2_g1_score, j05_p1_g1_score, j05_p2_g1_score, j06_p1_g1_score, j06_p2_g1_score,
      j07_p1_g1_score, j07_p2_g1_score, j08_p1_g1_score, j08_p2_g1_score, j10_p1_g1_score, j10_p2_g1_score, j11_p1_g1_score, j11_p2_g1_score]})
j_scores_1.set_index('participant', inplace=True, drop=True)
j_scores_1


# # Combined Joint scores G2

# In[ ]:


j_scores_2_d = pd.DataFrame({'participant':['j01_p1', 'j01_p2','j02_p1', 'j02_p2','j03_p1', 'j03_p2','j04_p1', 'j04_p2','j05_p1', 'j05_p2','j06_p1','j06_p2','j07_p1', 'j07_p2','j08_p1', 'j08_p2'], 'score_2':
     [j01_p1_g2_score, j01_p2_g2_score, j02_p1_g2_score, j02_p2_g2_score, j03_p1_g2_score, j03_p2_g2_score,
      j04_p1_g2_score, j04_p2_g2_score, j05_p1_g2_score, j05_p2_g2_score, j06_p1_g2_score, j06_p2_g2_score,
      j07_p1_g2_score, j07_p2_g2_score, j08_p1_g2_score, j08_p2_g2_score]})
j_scores_2_d.set_index('participant', inplace=True, drop=True)

j_scores_2_s = pd.DataFrame({'participant':['j10_p1', 'j10_p2','j11_p1', 'j11_p2'], 'score_2':
     [j10_p1_g2_score, j10_p2_g2_score, j11_p1_g2_score, j11_p2_g2_score]})
j_scores_2_s.set_index('participant', inplace=True, drop=True)

j_scores_1_50 = pd.DataFrame({'participant':['j01_p1', 'j01_p2','j02_p1', 'j02_p2','j03_p1', 'j03_p2','j04_p1', 'j04_p2','j05_p1', 'j05_p2','j06_p1','j06_p2','j07_p1', 'j07_p2','j08_p1', 'j08_p2','j10_p1', 'j10_p2','j11_p1', 'j11_p2'], 'score_1_50':
     [j01_p1_g1_50, j01_p2_g1_50, j02_p1_g1_50, j02_p2_g1_50, j03_p1_g1_50, j03_p2_g1_50,
      j04_p1_g1_50, j04_p2_g1_50, j05_p1_g1_50, j05_p2_g1_50, j06_p1_g1_50, j06_p2_g1_50,
      j07_p1_g1_50, j07_p2_g1_50, j08_p1_g1_50, j08_p2_g1_50, j10_p1_g1_50, j10_p2_g1_50, j11_p1_g1_50, j11_p2_g1_50]})
j_scores_1_50.set_index('participant', inplace=True, drop=True)




j_scores_2_50 = pd.DataFrame({'participant':['j01_p1', 'j01_p2','j02_p1', 'j02_p2','j03_p1', 'j03_p2','j04_p1', 'j04_p2','j05_p1', 'j05_p2','j06_p1','j06_p2','j07_p1', 'j07_p2','j08_p1', 'j08_p2','j10_p1', 'j10_p2','j11_p1', 'j11_p2'], 'score_2_50':
     [j01_p1_g2_50, j01_p2_g2_50, j02_p1_g2_50, j02_p2_g2_50, j03_p1_g2_50, j03_p2_g2_50,
      j04_p1_g2_50, j04_p2_g2_50, j05_p1_g2_50, j05_p2_g2_50, j06_p1_g2_50, j06_p2_g2_50,
      j07_p1_g2_50, j07_p2_g2_50, j08_p1_g2_50, j08_p2_g2_50, j10_p1_g2_50, j10_p2_g2_50, j11_p1_g2_50, j11_p2_g2_50]})
j_scores_2_50.set_index('participant', inplace=True, drop=True)



In [ ]:
j_scores_1_box = sns.boxplot( 
                 data=j_scores_1, 
                 width=0.5,
                 palette="pastel")
j_scores_1_box.axes.set_title("SMAB3 Social Scores in Game 1",
                    fontsize=16)

plot_file_name="j_score_1_box.jpg"
 
plt.ylim(0, 100)
j_scores_1_box.axes.set(ylabel='Score')

# save as jpeg
j_scores_1_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
j_scores_2s_box = sns.boxplot( 
                 data=j_scores_2_s, 
                 width=0.5,
                 palette="pastel")
j_scores_2s_box.axes.set_title("SMAB3 Social Scores in Game 1-Same",
                    fontsize=16)

plot_file_name="j_score_2s_box.jpg"
 
plt.ylim(0, 100)
j_scores_2s_box.axes.set(ylabel='Score')

# save as jpeg
j_scores_2s_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
j_scores_2d_box = sns.boxplot( 
                 data=j_scores_2_d, 
                 width=0.5,
                 palette="pastel")
j_scores_2d_box.axes.set_title("SMAB3 Social Scores in Game 1-Different",
                    fontsize=16)

plot_file_name="j_score_2d_box.jpg"
 
plt.ylim(0, 100)
j_scores_2d_box.axes.set(ylabel='Score')

# save as jpeg
j_scores_2d_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
j_scores_2s_mean = j_scores_2_s.mean()

In [ ]:
j_scores_2d_mean = j_scores_2_d.mean()

In [ ]:
j_scores_2s_std = j_scores_2_s.std()

In [ ]:
j_scores_2d_std = j_scores_2_d.std()

In [ ]:
j_scores_2s_mean, j_scores_2s_std

In [ ]:
j_scores_2d_mean, j_scores_2d_std

In [ ]:
sp.stats.ttest_ind(j_scores_2_s, j_scores_2_d)

In [ ]:
s_scores_2_s.size

In [ ]:
'''Here we get the mean, median, and standard deviation of the scores. We also run a t-test comparing the scores in games 1 and 2
and plot the data.'''

j_scores_1_mean = j_scores_1.mean()


# In[ ]:


j_scores_1_mean


# In[ ]:


j_scores_1_std = j_scores_1.std()


# In[ ]:


j_scores_1_std

In [ ]:
#j_scores_p = sp.stats.ttest_rel(j_scores['Game 1'],j_scores['Game 2'])

In [ ]:
len(j_scores_1)

In [ ]:
'''
j_scores_box = sns.boxplot( 
                 data=j_scores, 
                 width=0.5,
                 palette="pastel")
j_scores_box.axes.set_title("SMAB3 Social Scores",
                    fontsize=16)

plot_file_name="j_score_box.jpg"
 
plt.ylim(0, 100)
j_scores_box.axes.set(ylabel='Score')

# save as jpeg
j_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')'''

In [ ]:
'''Here we place all of the data into the number of times each subject chose each hole into dataframes for games 1 
and 2. We plot this data and run statistics on it.'''

# # Joint combined choices in G1



j_g1_counts = pd.DataFrame([j01_p1_g1_count, j01_p2_g1_count, j02_p1_g1_count, j02_p2_g1_count, j03_p1_g1_count, 
              j03_p2_g1_count, j04_p1_g1_count, j04_p2_g1_count, j05_p1_g1_count, j05_p2_g1_count, j06_p1_g1_count, 
              j06_p2_g1_count, j07_p1_g1_count, j07_p2_g1_count, j08_p1_g1_count, j08_p2_g1_count, 
              j10_p1_g1_count, j10_p2_g1_count, j11_p1_g1_count, j11_p2_g1_count],
                index = ['j01_p1_g1_count', 'j01_p2_g1_count', 'j02_p1_g1_count', 'j02_p2_g1_count', 
                 'j03_p1_g1_count', 'j03_p2_g1_count', 'j04_p1_g1_count', 'j04_p2_g1_count', 'j05_p1_g1_count', 
                  'j05_p2_g1_count', 'j06_p1_g1_count', 'j06_p2_g1_count', 'j07_p1_g1_count', 
                    'j07_p2_g1_count', 'j08_p1_g1_count', 'j08_p2_g1_count', 'j10_p1_g1_count', 'j10_p2_g1_count', 
                    'j11_p1_g1_count', 'j11_p2_g1_count'])


# In[ ]:


j_g1_counts = j_g1_counts.fillna(0)


# In[ ]:


j_g1_counts 


# In[ ]:


j_g1_counts.sum(axis=1)


# In[ ]:


j_g1_counts_bar = j_g1_counts.plot.bar()

j_g1_counts_bar.axes.set_title("SMAB3 Social Choice Distributions in Game 1",
                    fontsize=16)

j_g1_counts_bar.figure.savefig("j_g1_counts_bar.jpg",
                    format='jpeg')

In [ ]:
# In[ ]:
sns.set_style("whitegrid")

j_g1_counts_box = sns.boxplot( 
                 data=j_g1_counts, 
                 width=0.5,
                 palette="pastel")
j_g1_counts_box.axes.set_title("SMAB3 Social Choice Distributions in Game 1",
                    fontsize=16)

plot_file_name="j_g1_counts_box.jpg"
 
plt.ylim(0, 100)
j_g1_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
j_g1_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')


# In[ ]:

In [ ]:
j_g1_low = np.mean(j_g1_counts.iloc[:,0])
j_g1_med = np.mean(j_g1_counts.iloc[:,1])
j_g1_high = np.mean(j_g1_counts.iloc[:,2])


# In[ ]:


j_g1_low


# In[ ]:


j_g1_med


# In[ ]:


j_g1_high


# In[ ]:


j_g1_low_std = np.std(j_g1_counts.iloc[:,0])
j_g1_med_std = np.std(j_g1_counts.iloc[:,1])
j_g1_high_std = np.std(j_g1_counts.iloc[:,2])


# In[ ]:


j_g1_low_std


# In[ ]:


j_g1_med_std


# In[ ]:


j_g1_high_std

In [ ]:
# # Joint combined choices in G2


j_g2_d_counts = pd.DataFrame([j01_p1_g2_count, j01_p2_g2_count, j02_p1_g2_count, j02_p2_g2_count, j03_p1_g2_count, 
              j03_p2_g2_count, j04_p1_g2_count, j04_p2_g2_count, j05_p1_g2_count, j05_p2_g2_count, j06_p1_g2_count, 
              j06_p2_g2_count, j07_p1_g2_count, j07_p2_g2_count, j08_p1_g2_count, j08_p2_g2_count],
                index = ['j01_p1_g2_count', 'j01_p2_g2_count', 'j02_p1_g2_count', 'j02_p2_g2_count', 
                 'j03_p1_g2_count', 'j03_p2_g2_count', 'j04_p1_g2_count', 'j04_p2_g2_count', 'j05_p1_g2_count', 
                  'j05_p2_g2_count', 'j06_p1_g2_count', 'j06_p2_g2_count', 'j07_p1_g2_count', 'j07_p2_g2_count', 
                 'j08_p1_g2_count', 'j08_p2_g2_count'])



j_g2_d_counts = j_g2_d_counts.fillna(0)



j_g2_d_counts.sum(axis=1)



j_g2_d_counts_bar = j_g2_d_counts.plot.bar()

In [ ]:
j_g2_s_counts = pd.DataFrame([j10_p1_g2_count, j10_p2_g2_count, j11_p1_g2_count, j11_p2_g2_count],
                index = ['j10_p1_g2_count', 'j10_p2_g2_count', 
                    'j11_p1_g2_count', 'j11_p2_g2_count'])



j_g2_s_counts = j_g2_s_counts.fillna(0)



j_g2_s_counts.sum(axis=1)



j_g2_s_counts_bar = j_g2_s_counts.plot.bar()

In [ ]:
# In[ ]:


j_g2_s_counts_box = sns.boxplot( 
                 data=j_g2_s_counts, 
                 width=0.5,
                 palette="pastel")
j_g2_s_counts_box.axes.set_title("SMAB3 Social Choice Distributions in Game 2- Same",
                    fontsize=16)

plot_file_name="j_g2_s_counts_box.jpg"
 
plt.ylim(0, 100)
j_g2_s_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
j_g2_s_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
j_g2_d_counts_box = sns.boxplot( 
                 data=j_g2_d_counts, 
                 width=0.5,
                 palette="pastel")
j_g2_d_counts_box.axes.set_title("SMAB3 Social Choice Distributions in Game 2- Different",
                    fontsize=16)

plot_file_name="j_g2_d_counts_box.jpg"
 
plt.ylim(0, 100)
j_g2_d_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
j_g2_d_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
j_scores_1

In [ ]:
j_scores_1_mean

In [ ]:
j_scores_1_std

In [ ]:
# In[ ]:

j_g1_low = np.mean(j_g1_counts.iloc[:,0])
j_g1_med = np.mean(j_g1_counts.iloc[:,1])
j_g1_high = np.mean(j_g1_counts.iloc[:,2])

j_g2_s_low = np.mean(j_g2_s_counts.iloc[:,0])
j_g2_s_med = np.mean(j_g2_s_counts.iloc[:,1])
j_g2_s_high = np.mean(j_g2_s_counts.iloc[:,2])

j_g2_d_low = np.mean(j_g2_d_counts.iloc[:,0])
j_g2_d_med = np.mean(j_g2_d_counts.iloc[:,1])
j_g2_d_high = np.mean(j_g2_d_counts.iloc[:,2])

In [ ]:
j_g1_low, j_g1_med, j_g1_high

In [ ]:
j_g1_low_std, j_g1_med_std, j_g1_high_std

In [ ]:
j_g1_counts

# Reaction Time

In [ ]:
'Here we get the reaction times'
time_j_1 = pd.DataFrame(columns = ['session','post win','post loss'])
time_j_2 = pd.DataFrame(columns = ['session','post win','post loss'])

In [ ]:
time_j_1 = reaction_time(j01_p1_g1, time_j_1)
time_j_1 = reaction_time(j02_p1_g1, time_j_1)
time_j_1 = reaction_time(j03_p1_g1, time_j_1)
time_j_1 = reaction_time(j04_p1_g1, time_j_1)
time_j_1 = reaction_time(j05_p1_g1, time_j_1)
time_j_1 = reaction_time(j06_p1_g1, time_j_1)
time_j_1 = reaction_time(j07_p1_g1, time_j_1)
time_j_1 = reaction_time(j08_p1_g1, time_j_1)
time_j_1 = reaction_time(j10_p1_g1, time_j_1)
time_j_1 = reaction_time(j11_p1_g1, time_j_1)

In [ ]:
time_j_2 = reaction_time(j01_p1_g2, time_j_2)
time_j_2 = reaction_time(j02_p1_g2, time_j_2)
time_j_2 = reaction_time(j03_p1_g2, time_j_2)
time_j_2 = reaction_time(j04_p1_g2, time_j_2)
time_j_2 = reaction_time(j05_p1_g2, time_j_2)
time_j_2 = reaction_time(j06_p1_g2, time_j_2)
time_j_2 = reaction_time(j07_p1_g2, time_j_2)
time_j_2 = reaction_time(j08_p1_g2, time_j_2)
time_j_2 = reaction_time(j10_p1_g2, time_j_2)
time_j_2 = reaction_time(j11_p1_g2, time_j_2)

In [ ]:
times_j_g1_box = sns.boxplot( 
                 data=time_j_1, 
                 width=0.5,
                 palette="pastel")
times_j_g1_box.axes.set_title("SMAB3 Joint Reaction Times in Game 1",
                    fontsize=16)

plot_file_name="times_j_g1_box.jpg"
 
plt.ylim(0, 5)
times_j_g1_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_j_g1_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
times_j_g2_box = sns.boxplot( 
                 data=time_j_2, 
                 width=0.5,
                 palette="pastel")
times_j_g2_box.axes.set_title("SMAB3 Joint Reaction Times in Game 2",
                    fontsize=16)

plot_file_name="times_j_g2_box.jpg"
 
plt.ylim(0, 5)
times_j_g2_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_j_g2_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
times_s_g2_box = sns.boxplot( 
                 data=time_s_2, 
                 width=0.5,
                 palette="pastel")
times_s_g2_box.axes.set_title("SMAB3 Control Reaction Times in Game 2",
                    fontsize=16)

plot_file_name="times_s_g2_box.jpg"
 
plt.ylim(0, 5)
times_s_g2_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_s_g2_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
times_s_g1_box = sns.boxplot( 
                 data=time_s_1, 
                 width=0.5,
                 palette="pastel")
times_s_g1_box.axes.set_title("SMAB3 Control Reaction Times in Game 1",
                    fontsize=16)

plot_file_name="times_s_g1_box.jpg"
 
plt.ylim(0, 5)
times_s_g1_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_s_g1_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
np.mean(time_s_1)

In [ ]:
np.mean(time_s_2)

In [ ]:
np.std(time_s_1)

In [ ]:
np.std(time_s_2)

In [ ]:
np.mean(time_j_1)

In [ ]:
np.mean(time_j_2)

In [ ]:
np.std(time_j_1)

In [ ]:
np.std(time_j_2)

In [ ]:
sp.stats.ttest_rel(time_s_1['post win'],time_s_1['post loss'])

In [ ]:
sp.stats.ttest_rel(time_s_2['post win'],time_s_2['post loss'])

In [ ]:
sp.stats.ttest_rel(time_j_1['post win'],time_j_1['post loss'])

In [ ]:
sp.stats.ttest_rel(time_j_2['post win'],time_j_2['post loss'])

# SMAB4 Joint Data 

In [44]:
m_trial_by_trial = ['m05']
m_three = ['m06','m08','m09','m10','m11','m13','m14']

In [39]:
m_trial_by_trial_dict = imports(sessions = m_trial_by_trial, condition = 'm')

m05


In [45]:
'''Here we load the csv files into pandas dataframes by calling organize_trial_by_trial and get the scores in games
1 and 2 by calling call_get_score.
Note: for the files that were hand coded, the scores will be obtained later, as the score column needs to be filled out still.'''

# # Multi Analysis

df = organize_trial_by_trial_hand_code2('data/m01_gamedata_handcode.csv')
m01 = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m01.head()

df = organize_trial_by_trial_hand_code2('data/m03_gamedata_handcode.csv')
m03 = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m03.head()

df = organize_trial_by_trial_hand_code2('data/m07_gamedata_handcode.csv')
m07 = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m07.head()

#NOTE: m09 from LSL needs to be combined with hand coded data because the lsl csv file is missing the first 13 trials
df = organize_trial_by_trial_hand_code('data/m09_gamedata_handcode.csv')
m09_hand = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m09_hand.head()



m05_p1_g1, m05_p1_g2, m05_p2_g1, m05_p2_g2 = separate_df(m05, 'm')
m06_p1_g1, m06_p1_g2, m06_p2_g1, m06_p2_g2 = separate_df(m06, 'm')
m07_p1_g1, m07_p1_g2, m07_p2_g1, m07_p2_g2 = separate_df(m07, 'm')
m08_p1_g1, m08_p1_g2, m08_p2_g1, m08_p2_g2 = separate_df(m08, 'm')
m10_p1_g1, m10_p1_g2, m10_p2_g1, m10_p2_g2 = separate_df(m10, 'm')
m11_p1_g1, m11_p1_g2, m11_p2_g1, m11_p2_g2 = separate_df(m11, 'm')
m13_p1_g1, m13_p1_g2, m13_p2_g1, m13_p2_g2 = separate_df(m13, 'm')

m14_p1_g1, m14_p1_g2, m14_p2_g1, m14_p2_g2 = separate_df(m14, 'm')

NameError: name 'm05' is not defined

In [ ]:
m01_p1_counts, m01_p2_counts, m01_p1_probs, m01_p2_probs = handcode_calc_prob(m01)
m03_p1_counts, m03_p2_counts, m03_p1_probs, m03_p2_probs = handcode_calc_prob(m03)
m07_p1_counts, m07_p2_counts, m07_p1_probs, m07_p2_probs = handcode_calc_prob(m07)

In [ ]:
m01_p1_counts

In [ ]:
m01_p2_counts

In [ ]:
m01_p1_probs

In [ ]:
m01_p2_probs

In [ ]:
m03_p1_counts

In [ ]:
m03_p2_counts

In [ ]:
m03_p1_probs

In [ ]:
m03_p2_probs

In [ ]:
m01 = fill_in_prob(m01,90,60,30,60,30,90,30,60,90,60,90,30)
m03 = fill_in_prob(m03,30,90,60,90,30,60,90,30,60,30,90,60)

In [ ]:
m01_p1_g1, m01_p1_g2, m01_p2_g1, m01_p2_g2 = separate_df(m01, 'm')
m03_p1_g1, m03_p1_g2, m03_p2_g1, m03_p2_g2 = separate_df(m03, 'm')
m07_p1_g1, m07_p1_g2, m07_p2_g1, m07_p2_g2 = separate_df(m07, 'm')

In [ ]:
m01_p1_g1_score = m01_p1_g1['reward'].sum()
m01_p1_g2_score = m01_p1_g2['reward'].sum()
m01_p2_g1_score = m01_p2_g1['reward'].sum()
m01_p2_g2_score = m01_p2_g2['reward'].sum()

m03_p1_g1_score = m03_p1_g1['reward'].sum()
m03_p1_g2_score = m03_p1_g2['reward'].sum()
m03_p2_g1_score = m03_p2_g1['reward'].sum()
m03_p2_g2_score = m03_p2_g2['reward'].sum()

m07_p1_g1_score = m07_p1_g1['reward'].sum()
m07_p1_g2_score = m07_p1_g2['reward'].sum()
m07_p2_g1_score = m07_p2_g1['reward'].sum()
m07_p2_g2_score = m07_p2_g2['reward'].sum()

In [ ]:
# In[ ]:
#Not working for m01 and m07 (hand coded)- need to fill in the probability column
m01_p1_g1_count = m01_p1_g1['probability'].value_counts(sort=False)
m01_p1_g2_count = m01_p1_g2['probability'].value_counts(sort=False)
m01_p2_g1_count = m01_p2_g1['probability'].value_counts(sort=False)
m01_p2_g2_count = m01_p2_g2['probability'].value_counts(sort=False)
m03_p1_g1_count = m03_p1_g1['probability'].value_counts(sort=False)
m03_p1_g2_count = m03_p1_g2['probability'].value_counts(sort=False)
m03_p2_g1_count = m03_p2_g1['probability'].value_counts(sort=False)
m03_p2_g2_count = m03_p2_g2['probability'].value_counts(sort=False)

m05_p1_g1_count = m05_p1_g1['probability'].value_counts(sort=False)
m05_p1_g2_count = m05_p1_g2['probability'].value_counts(sort=False)
m05_p2_g1_count = m05_p2_g1['probability'].value_counts(sort=False)
m05_p2_g2_count = m05_p2_g2['probability'].value_counts(sort=False)

m07_p1_g1_count = m07_p1_g1['probability'].value_counts(sort=False)
m07_p1_g2_count = m07_p1_g2['probability'].value_counts(sort=False)
m07_p2_g1_count = m07_p2_g1['probability'].value_counts(sort=False)
m07_p2_g2_count = m07_p2_g2['probability'].value_counts(sort=False)

In [ ]:
'''Here we check the probability-hole mappings for each player in each game'''
#m07,m06, and m09 are excluded from further analyses because the hole probability matchings are incorrect for those sessions
m01_p1_g1_map = hole_prob_map(m01_p1_g1)
m01_p1_g2_map = hole_prob_map(m01_p1_g2)
m01_p2_g1_map = hole_prob_map(m01_p2_g1)

m03_p2_g2_map = hole_prob_map(m03_p2_g2)
m03_p1_g1_map = hole_prob_map(m03_p1_g1)
m03_p1_g2_map = hole_prob_map(m03_p1_g2)
m03_p2_g1_map = hole_prob_map(m03_p2_g1)
m03_p2_g2_map = hole_prob_map(m03_p2_g2)

m05_p1_g1_map = hole_prob_map(m05_p1_g1)
m05_p1_g2_map = hole_prob_map(m05_p1_g2)
m05_p2_g1_map = hole_prob_map(m05_p2_g1)
m05_p2_g2_map = hole_prob_map(m05_p2_g2)

m06_p1_g1_map = hole_prob_map(m06_p1_g1)
m06_p1_g2_map = hole_prob_map(m06_p1_g2)
m06_p2_g1_map = hole_prob_map(m06_p2_g1)
m06_p2_g2_map = hole_prob_map(m06_p2_g2)

#m07_p1_g1_map = hole_prob_map(m07_p1_g1)
#m07_p1_g2_map = hole_prob_map(m07_p1_g2)
m07_p2_g1_map = hole_prob_map(m07_p2_g1)
#m07_p2_g2_map = hole_prob_map(m07_p2_g2)

m08_p1_g1_map = hole_prob_map(m08_p1_g1)
m08_p1_g2_map = hole_prob_map(m08_p1_g2)
m08_p2_g1_map = hole_prob_map(m08_p2_g1)
#m08_p2_g2_map = hole_prob_map(m08_p2_g2)

m10_p1_g1_map = hole_prob_map(m10_p1_g1)
m10_p1_g2_map = hole_prob_map(m10_p1_g2)
m10_p2_g1_map = hole_prob_map(m10_p2_g1)
m10_p2_g2_map = hole_prob_map(m10_p2_g2)

#m11_p1_g1_map = hole_prob_map(m11_p1_g1)
#m11_p1_g2_map = hole_prob_map(m11_p1_g2)
#m11_p2_g1_map = hole_prob_map(m11_p2_g1)
#m11_p2_g2_map = hole_prob_map(m11_p2_g2)

m13_p1_g1_map = hole_prob_map(m13_p1_g1)
m13_p1_g2_map = hole_prob_map(m13_p1_g2)
m13_p2_g1_map = hole_prob_map(m13_p2_g1)
m13_p2_g2_map = hole_prob_map(m13_p2_g2)

In [ ]:
m08_p1_g1_map, m08_p2_g1_map

In [ ]:
#same_probs_g1_2
m06_p1_g1_count = m06_p1_g1['probability'].value_counts(sort=False)
m06_p1_g2_count = m06_p1_g2['probability'].value_counts(sort=False)
m06_p2_g1_count = m06_p2_g1['probability'].value_counts(sort=False)
m06_p2_g2_count = m06_p2_g2['probability'].value_counts(sort=False)
m07_p1_g1_count = m07_p1_g1['probability'].value_counts(sort=False)
m07_p1_g2_count = m07_p1_g2['probability'].value_counts(sort=False)
m07_p2_g1_count = m07_p2_g1['probability'].value_counts(sort=False)
m07_p2_g2_count = m07_p2_g2['probability'].value_counts(sort=False)
m08_p1_g1_count = m08_p1_g1['probability'].value_counts(sort=False)
m08_p1_g2_count = m08_p1_g2['probability'].value_counts(sort=False)
m08_p2_g1_count = m08_p2_g1['probability'].value_counts(sort=False)
m08_p2_g2_count = m08_p2_g2['probability'].value_counts(sort=False)
m10_p1_g1_count = m10_p1_g1['probability'].value_counts(sort=False)
m10_p1_g2_count = m10_p1_g2['probability'].value_counts(sort=False)
m10_p2_g1_count = m10_p2_g1['probability'].value_counts(sort=False)
m10_p2_g2_count = m10_p2_g2['probability'].value_counts(sort=False)
m11_p1_g1_count = m11_p1_g1['probability'].value_counts(sort=False)
m11_p1_g2_count = m11_p1_g2['probability'].value_counts(sort=False)
m11_p2_g1_count = m11_p2_g1['probability'].value_counts(sort=False)
m11_p2_g2_count = m11_p2_g2['probability'].value_counts(sort=False)
m13_p1_g1_count = m13_p1_g1['probability'].value_counts(sort=False)
m13_p1_g2_count = m13_p1_g2['probability'].value_counts(sort=False)
m13_p2_g1_count = m13_p2_g1['probability'].value_counts(sort=False)
m13_p2_g2_count = m13_p2_g2['probability'].value_counts(sort=False)
m14_p1_g1_count = m14_p1_g1['probability'].value_counts(sort=False)
m14_p1_g2_count = m14_p1_g2['probability'].value_counts(sort=False)
m14_p2_g1_count = m14_p2_g1['probability'].value_counts(sort=False)
m14_p2_g2_count = m14_p2_g2['probability'].value_counts(sort=False)

In [ ]:
#call fix_reward on all of the files imported with '300' to fix the reward column
m06_p1_g1 = fix_reward(m06_p1_g1)
m06_p1_g2 = fix_reward(m06_p1_g2)
m06_p2_g1 = fix_reward(m06_p2_g1)
m06_p2_g2 = fix_reward(m06_p2_g2)
m08_p1_g1 = fix_reward(m08_p1_g1)
m08_p1_g2 = fix_reward(m08_p1_g2)
m08_p2_g1 = fix_reward(m08_p2_g1)
m08_p2_g2 = fix_reward(m08_p2_g2)
m10_p1_g1 = fix_reward(m10_p1_g1)
m10_p1_g2 = fix_reward(m10_p1_g2)
m10_p2_g1 = fix_reward(m10_p2_g1)
m10_p2_g2 = fix_reward(m10_p2_g2)
m11_p1_g1 = fix_reward(m11_p1_g1)
m11_p1_g2 = fix_reward(m11_p1_g2)
m11_p2_g1 = fix_reward(m11_p2_g1)
m11_p2_g2 = fix_reward(m11_p2_g2)
m13_p1_g1 = fix_reward(m13_p1_g1)
m13_p1_g2 = fix_reward(m13_p1_g2)
m13_p2_g1 = fix_reward(m13_p2_g1)
m13_p2_g2 = fix_reward(m13_p2_g2)
m14_p1_g1 = fix_reward(m14_p1_g1)
m14_p1_g2 = fix_reward(m14_p1_g2)
m14_p2_g1 = fix_reward(m14_p2_g1)
m14_p2_g2 = fix_reward(m14_p2_g2)

In [ ]:
# # Combined multi scores G1

# In[ ]:


m_scores_1 = pd.DataFrame({'participant':['m01_p1','m01_p2', 'm03_p1', 'm03_p2','m05_p1', 'm05_p2', 'm08_p1', 'm08_p2', 'm10_p1', 'm10_p2', 'm11_p1', 'm11_p2', 'm13_p1', 'm13_p2', 'm14_p1', 'm14_p2'], 'score_1':
     [m01_p1_g1_score, m01_p2_g1_score, m03_p1_g1_score, m03_p2_g1_score, m05_p1_g1_score, m05_p2_g1_score, m08_p1_g1_score, m08_p2_g1_score,
      m10_p1_g1_score, m10_p2_g1_score, m11_p1_g1_score, m11_p2_g1_score, m13_p1_g1_score, m13_p2_g1_score,
      m14_p1_g1_score, m14_p2_g1_score]})
m_scores_1.set_index('participant', inplace=True, drop=True)
m_scores_1

In [ ]:
# # Combined multi scores G2

# In[ ]:

#NEEDS TO BE SEPARATED
m_scores_2_d = pd.DataFrame({'participant':['m01_p1', 'm01_p2','m03_p1','m03_p2','m05_p1','m05_p2'], 'score_2':
     [m01_p1_g2_score, m01_p2_g2_score, m03_p1_g2_score, m03_p2_g2_score, m05_p1_g2_score, m05_p2_g2_score]})
m_scores_2_d.set_index('participant', inplace=True, drop=True)

m_scores_2_s = pd.DataFrame({'participant':['m08_p1', 'm08_p2', 'm10_p1', 'm10_p2', 'm11_p1', 'm11_p2', 'm13_p1', 'm13_p2', 'm14_p1', 'm14_p2'], 'score_2':
     [m08_p1_g2_score, m08_p2_g2_score,
      m10_p1_g2_score, m10_p2_g2_score, m11_p1_g2_score, m11_p2_g2_score, m13_p1_g2_score, m13_p2_g2_score,
      m14_p1_g2_score, m14_p2_g2_score]})
m_scores_2_s.set_index('participant', inplace=True, drop=True)


#NEEDS TO BE REVISITED, add 50 to hand coded sessions
'''m_scores_1_50 = pd.DataFrame({'participant':['m01_p1','m01_p2', 'm03_p1', 'm03_p2','m05_p1', 'm05_p2','m06_p1', 'm06_p2', 'm07_p1', 'm07_p2', 'm08_p1', 'm08_p2', 'm10_p1', 'm10_p2', 'm11_p1', 'm11_p2', 'm13_p1', 'm13_p2', 'm14_p1', 'm14_p2'], 'score_1_50':
     [m01_p1_g1_50, m01_p2_g1_50, m03_p1_g1_50, m03_p2_g1_50, m05_p1_g1_50, m05_p2_g1_50, m06_p1_g1_50, m06_p2_g1_50, m07_p1_g1_50, m07_p2_g1_50, m08_p1_g1_50, m08_p2_g1_50, m10_p1_g1_50, m10_p2_g1_50, m11_p1_g1_50, m11_p2_g1_50, m13_p1_g1_50, m13_p2_g1_50,
     m14_p1_g1_50, m14_p2_g1_50,]})
m_scores_1_50.set_index('participant', inplace=True, drop=True)

#NEEDS TO BE SEPARATED
m_scores_2_50 = pd.DataFrame({'participant':['m05_p1', 'm05_p2','m06_p1', 'm06_p2', 'm07_p1', 'm07_p2', 'm08_p1', 'm08_p2', 'm10_p1', 'm10_p2', 'm11_p1', 'm11_p2', 'm13_p1', 'm13_p2', 'm14_p1', 'm14_p2'], 'score_1_50':
     [m01_p1_g2_50, m01_p2_g2_50, m03_p1_g2_50, m03_p2_g2_50, m05_p1_g2_50, m05_p2_g2_50, m06_p1_g2_50, m06_p2_g2_50, m07_p1_g2_50, m07_p2_g2_50, m08_p1_g2_50, m08_p2_g2_50, m10_p1_g2_50, m10_p2_g2_50, m11_p1_g2_50, m11_p2_g2_50, m13_p1_g2_50, m13_p2_g2_50,
     m14_p1_g2_50, m14_p2_g2_50,]})
m_scores_2_50.set_index('participant', inplace=True, drop=True)'''

In [ ]:
m_scores_1

In [ ]:
m_scores_1_bar = m_scores_1.plot.bar()

In [ ]:
m_g1_scores_box = sns.boxplot( 
                 data=m_scores_1, 
                 width=0.5,
                 palette="pastel")
m_g1_scores_box.axes.set_title("SMAB4 Social Scores in Game 1",
                    fontsize=16)

plot_file_name="m_g1_score_box.jpg"
 
plt.ylim(0, 100)
m_g1_scores_box.axes.set(ylabel='Score')

# save as jpeg
m_g1_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')


In [ ]:
m_g2d_scores_box = sns.boxplot( 
                 data=m_scores_2_d, 
                 width=0.5,
                 palette="pastel")
m_g2d_scores_box.axes.set_title("SMAB4 Social Scores in Game 2-Different",
                    fontsize=16)

plot_file_name="m_g2d_score_box.jpg"
 
plt.ylim(0, 100)
m_g2d_scores_box.axes.set(ylabel='Score')

# save as jpeg
m_g2d_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
m_g2s_scores_box = sns.boxplot( 
                 data=m_scores_2_s, 
                 width=0.5,
                 palette="pastel")
m_g2s_scores_box.axes.set_title("SMAB4 Social Scores in Game 2-Same",
                    fontsize=16)

plot_file_name="m_g2s_score_box.jpg"
 
plt.ylim(0, 100)
m_g2s_scores_box.axes.set(ylabel='Score')

# save as jpeg
m_g2s_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
m_scores_2_d

In [ ]:
m_scores_2s_mean = m_scores_2_s.mean()

In [ ]:
m_scores_2s_std = m_scores_2_s.std()

In [ ]:
m_scores_2d_mean = m_scores_2_d.mean()

In [ ]:
m_scores_2d_std = m_scores_2_d.std()

In [ ]:
m_scores_2s_mean, m_scores_2s_std

In [ ]:
m_scores_2d_mean, m_scores_2d_std

In [ ]:
sp.stats.ttest_ind(m_scores_2_s, m_scores_2_d)

In [ ]:
'''Here we place the counts of how many times each hole was chosen into dataframes for games 1 and 2. We also 
get statistics on this data and plot it.'''

# # Multi combined choices in G1

# In[ ]:


m_g1_counts = pd.DataFrame([m01_p1_g1_count, m01_p2_g1_count, m03_p1_g1_count, m03_p2_g1_count, m05_p1_g1_count, m05_p2_g1_count, m08_p1_g1_count, m08_p2_g1_count, 
              m10_p1_g1_count, m10_p2_g1_count, m11_p1_g1_count, m11_p2_g1_count, m13_p1_g1_count, m13_p2_g1_count, 
              m14_p1_g1_count, m14_p2_g1_count],
                index = ['m01_p1_g1_count', 'm01_p2_g1_count', 'm03_p1_g1_count', 'm03_p2_g1_count', 'm05_p1_g1_count', 'm05_p2_g1_count', 
                'm08_p1_g1_count', 'm08_p2_g1_count', 'm10_p1_g1_count', 'm10_p2_g1_count', 'm11_p1_g1_count', 
                'm11_p2_g1_count', 'm13_p1_g1_count', 'm13_p2_g1_count', 'm14_p1_g1_count', 'm14_p2_g1_count']) 

# In[ ]:


m_g1_counts = m_g1_counts.fillna(0) 



# In[ ]:


m_g1_counts


# In[ ]:


m_g1_counts.sum(axis=1)


# In[ ]:

# save as jpeg


m_g1_counts_bar = m_g1_counts.plot.bar()

m_g1_counts_bar.axes.set(ylabel='SMAB4 Game 1 Choices')

m_g1_counts_bar.axes.set_title("SMAB4 Social Choice Distributions in Game 1",fontsize=16)

m_g1_counts_bar.figure.savefig("m1_g1_counts_bar.jpg",
                    format='jpeg')

In [ ]:
m01_p1_g1_count

In [ ]:
m_g1_counts

In [ ]:
len(m_g1_counts)

In [ ]:
# In[ ]:


m_g1_counts_box = sns.boxplot( 
                 data = m_g1_counts, 
                 width=0.5,
                 palette="pastel")
m_g1_counts_box.axes.set_title("SMAB4 Social Choice Distributions in Game 1",
                    fontsize=16)

plot_file_name="m_g1_counts_box.jpg"
 
plt.ylim(0, 100)
m_g1_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
m_g1_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
# In[ ]:


m_g1_low = np.mean(m_g1_counts.iloc[:,0])
m_g1_med = np.mean(m_g1_counts.iloc[:,1])
m_g1_high = np.mean(m_g1_counts.iloc[:,2])


# In[ ]:


m_g1_low


# In[ ]:


m_g1_med


# In[ ]:


m_g1_high


# In[ ]:


m_g1_low_std = np.std(m_g1_counts.iloc[:,0])
m_g1_med_std = np.std(m_g1_counts.iloc[:,1])
m_g1_high_std = np.std(m_g1_counts.iloc[:,2])


# In[ ]:


m_g1_low_std


# In[ ]:


m_g1_med_std


# In[ ]:


m_g1_high_std

In [ ]:
# # Multi combined choices in G2


m_g2d_counts = pd.DataFrame([m01_p1_g2_count, m01_p2_g2_count, m03_p1_g2_count, m03_p2_g2_count, m05_p1_g2_count, m05_p2_g2_count],
                index = ['m01_p1_g2_count','m01_p2_g2_count','m03_p1_g2_count', 'm03_p2_g2_count','m05_p1_g2_count', 'm05_p2_g2_count'])



m_g2d_counts = m_g2d_counts.fillna(0) 


m_g2d_counts.sum(axis=1)

In [ ]:
# # Multi combined choices in G2


m_g2s_counts = pd.DataFrame([m08_p1_g2_count, m08_p2_g2_count, 
              m10_p1_g2_count, m10_p2_g2_count, m11_p1_g2_count, m11_p2_g2_count, m13_p1_g2_count, m13_p2_g2_count, 
              m14_p1_g2_count, m14_p2_g2_count],
                index = ['m08_p1_g2_count', 'm08_p2_g2_count', 'm10_p1_g2_count', 'm10_p2_g2_count', 'm11_p1_g2_count', 
                'm11_p2_g2_count', 'm13_p1_g2_count', 'm13_p2_g2_count', 'm14_p1_g2_count', 'm14_p2_g2_count'])


m_g2s_counts = m_g2s_counts.fillna(0) 



m_g2s_counts.sum(axis=1)

In [ ]:
m_scores_1

In [ ]:
m_scores_mean_1 = np.mean(m_scores_1)
m_scores_std_1 = np.std(m_scores_1)

In [ ]:
m_scores_mean_1

In [ ]:
m_scores_std_1

In [ ]:
m_g1_low, m_g1_med, m_g1_high

In [ ]:
m_g1_low_std, m_g1_med_std, m_g1_high_std

In [ ]:
m_g2s_counts_box = sns.boxplot( 
                 data = m_g2s_counts, 
                 width=0.5,
                 palette="pastel")
m_g2s_counts_box.axes.set_title("SMAB4 Social Choice Distributions in Game 2-Same",
                    fontsize=16)

plot_file_name="m_g2s_counts_box.jpg"
 
plt.ylim(0, 100)
m_g2s_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
m_g2s_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
m_g2d_counts_box = sns.boxplot( 
                 data = m_g2d_counts, 
                 width=0.5,
                 palette="pastel")
m_g2d_counts_box.axes.set_title("SMAB4 Social Choice Distributions in Game 2-Different",
                    fontsize=16)

plot_file_name="m_g2d_counts_box.jpg"
 
plt.ylim(0, 100)
m_g2d_counts_box.axes.set(xlabel='Arm',ylabel='Frequency')

# save as jpeg
m_g2d_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
# In[ ]:


m_g2s_low = np.mean(m_g2s_counts.iloc[:,0])
m_g2s_med = np.mean(m_g2s_counts.iloc[:,1])
m_g2s_high = np.mean(m_g2s_counts.iloc[:,2])

m_g2s_low_std = np.std(m_g2s_counts.iloc[:,0])
m_g2s_med_std = np.std(m_g2s_counts.iloc[:,1])
m_g2s_high_std = np.std(m_g2s_counts.iloc[:,2])



import researchpy as rp


# In[ ]:


m_90_1 = np.array(m_g1_counts.iloc[:,2])
m_90_2s = np.array(m_g2s_counts.iloc[:,2])
m_90_2d = np.array(m_g2s_counts.iloc[:,2])

m_60_1 = np.array(m_g1_counts.iloc[:,1])
m_60_2s = np.array(m_g2s_counts.iloc[:,1])
m_60_2d = np.array(m_g2d_counts.iloc[:,1])


m_30_1 = np.array(m_g1_counts.iloc[:,0])
m_30_2s = np.array(m_g2s_counts.iloc[:,0])
m_30_2d = np.array(m_g2d_counts.iloc[:,0])

# Reaction Time

In [ ]:
'Here we get the reaction times'
time_m_1 = pd.DataFrame(columns = ['session','post win','post loss'])
time_m_2 = pd.DataFrame(columns = ['session','post win','post loss'])

time_m_1 = reaction_time(m05_p1_g1, time_m_1)
time_m_1 = reaction_time(m06_p1_g1, time_m_1)
time_m_1 = reaction_time(m08_p1_g1, time_m_1)
time_m_1 = reaction_time(m10_p1_g1, time_m_1)
time_m_1 = reaction_time(m11_p1_g1, time_m_1)
time_m_1 = reaction_time(m13_p1_g1, time_m_1)
time_m_1 = reaction_time(m14_p1_g1, time_m_1)
time_m_1 = reaction_time(m05_p2_g1, time_m_1)
time_m_1 = reaction_time(m06_p2_g1, time_m_1)
time_m_1 = reaction_time(m08_p2_g1, time_m_1)
time_m_1 = reaction_time(m10_p2_g1, time_m_1)
time_m_1 = reaction_time(m11_p2_g1, time_m_1)
time_m_1 = reaction_time(m13_p2_g1, time_m_1)
time_m_1 = reaction_time(m14_p2_g1, time_m_1)

time_m_2 = reaction_time(m05_p1_g2, time_m_2)
time_m_2 = reaction_time(m06_p1_g2, time_m_2)
time_m_2 = reaction_time(m08_p1_g2, time_m_2)
time_m_2 = reaction_time(m10_p1_g2, time_m_2)
time_m_2 = reaction_time(m11_p1_g2, time_m_2)
time_m_2 = reaction_time(m13_p1_g2, time_m_2)
time_m_2 = reaction_time(m14_p1_g2, time_m_2)
time_m_2 = reaction_time(m05_p2_g2, time_m_2)
time_m_2 = reaction_time(m06_p2_g2, time_m_2)
time_m_2 = reaction_time(m08_p2_g2, time_m_2)
time_m_2 = reaction_time(m10_p2_g2, time_m_2)
time_m_2 = reaction_time(m11_p2_g2, time_m_2)
time_m_2 = reaction_time(m13_p2_g2, time_m_2)
time_m_2 = reaction_time(m14_p2_g2, time_m_2)

In [ ]:
times_m_g1_box = sns.boxplot( 
                 data=time_m_1, 
                 width=0.5,
                 palette="pastel")
times_m_g1_box.axes.set_title("SMAB4 Joint Reaction Times in Game 1",
                    fontsize=16)

plot_file_name="times_m_g1_box.jpg"
 
plt.ylim(0, 5)
times_m_g1_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_m_g1_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
times_m_g2_box = sns.boxplot( 
                 data=time_m_2, 
                 width=0.5,
                 palette="pastel")
times_m_g2_box.axes.set_title("SMAB4 Joint Reaction Times in Game 2",
                    fontsize=16)

plot_file_name="times_m_g2_box.jpg"
 
plt.ylim(0, 5)
times_m_g2_box.axes.set(ylabel='Time in sec.')

# save as jpeg
times_m_g2_box.figure.savefig(plot_file_name,format='jpeg')

In [ ]:
np.mean(time_m_1)

In [ ]:
np.mean(time_m_2)

In [ ]:
np.std(time_m_1)

In [ ]:
np.std(time_m_2)

In [ ]:
sp.stats.ttest_rel(time_m_1['post win'], time_m_1['post loss'])

In [ ]:
sp.stats.ttest_rel(time_m_2['post win'], time_m_2['post loss'])

In [ ]:
sp.stats.ttest_rel(time_m_1['post win'], time_m_2['post win'])

In [ ]:
sp.stats.ttest_rel(time_m_1['post loss'], time_m_2['post loss'])

# Test for differences between SMAB3 and SMAB4 in multi-player data

In [ ]:
#simple t-test for score 
sp.stats.ttest_ind(m_scores_1, j_scores_1)

In [ ]:
#combine control_score_2 and control_score_1
#social_scores = social_scores_1.merge(social_scores_2,right_index=True,left_index=True)


#convert from wide-format to long-format
#social_scores = pd.melt(social_scores,ignore_index=False)

# Combining SMAB3 and 4

In [ ]:
#combine m_counts and j_counts
soc_g1_counts_unmelt = m_g1_counts.append(j_g1_counts)

In [ ]:
#combine m_counts and j_counts
soc_g2_s_counts_unmelt = m_g2s_counts.append(j_g2_s_counts)

soc_g2_d_counts_unmelt = m_g2d_counts.append(j_g2_d_counts)

soc_g2_counts_unmelt = soc_g2_s_counts_unmelt.append(soc_g2_d_counts_unmelt)

In [ ]:
confed_g2_d_counts_unmelt = c_g2_d_counts.append(s_g2_d_counts)

confed_g2_s_counts_unmelt = c_g2_s_counts.append(s_g2_s_counts)

confed_g2_counts_unmelt = confed_g2_s_counts_unmelt.append(confed_g2_d_counts_unmelt)

In [ ]:
soc_g2_s_scores_unmelt = m_scores_2_s.append(j_scores_2_s)

soc_g2_d_scores_unmelt = m_scores_2_d.append(j_scores_2_d)

In [ ]:
confed_g2_s_scores_unmelt = c_scores_2_s.append(s_scores_2_s)

confed_g2_d_scores_unmelt = c_scores_2_d.append(s_scores_2_d)

In [ ]:
'''The next few cells are done in order to be able to run ANOVA.'''
soc_g1_counts = soc_g1_counts_unmelt.melt(ignore_index=False)
soc_g2_counts = soc_g2_counts_unmelt.melt(ignore_index=False)

In [ ]:
soc_g2_counts

In [ ]:
confed_g1_counts_unmelt = c_g1_counts.append(s_g1_counts)
confed_g1_counts = confed_g1_counts_unmelt.melt(ignore_index=False)
confed_g2_counts = confed_g2_counts_unmelt.melt(ignore_index=False)

In [ ]:
np.mean(confed_g1_counts_unmelt)

In [ ]:
np.mean(soc_g1_counts_unmelt)

In [ ]:
np.std(confed_g1_counts_unmelt)

In [ ]:
np.std(soc_g1_counts_unmelt)

In [ ]:
soc_g1_counts_unmelt_winsor = soc_g1_counts_unmelt.apply(winsorize_column,axis=0)

In [ ]:
confed_g1_counts_unmelt_winsor = confed_g1_counts_unmelt.apply(winsorize_column,axis=0)

In [ ]:
np.mean(confed_g1_counts_unmelt_winsor)

In [ ]:
np.mean(soc_g1_counts_unmelt_winsor)

In [ ]:
soc_g1_counts_unmelt

In [ ]:
sp.stats.ttest_ind(confed_g1_counts_unmelt,soc_g1_counts_unmelt)

In [ ]:
sp.stats.ttest_ind(confed_g1_counts_unmelt_winsor,soc_g1_counts_unmelt_winsor)

In [ ]:
g1_counts = confed_g1_counts.append(soc_g1_counts)
g2_counts = confed_g2_counts.append(soc_g2_counts)

In [ ]:
g1_counts

In [ ]:
#add another column to scores indicating whether it's the single or multiplayer
#TURN INTO FUNCTION

subjects = g1_counts.index.to_numpy()

condition = np.array([])

for i in subjects:
    if [i][0][0] == 's' or [i][0][0] == 'c':
        condition = np.append(condition,'control')
    elif [i][0][0] == 'm' or [i][0][0] == 'j':
        condition = np.append(condition,'social')

g1_counts['condition'] = condition

In [ ]:
#add column to joint_counts indicating whether it's SMAB3 or 4
subjects = g1_counts.index.to_numpy()

experiment = np.array([])

for i in subjects:
    if [i][0][0] == 'j' or [i][0][0] == 's':
        experiment = np.append(experiment,3)
    elif [i][0][0] == 'm' or [i][0][0] == 'c':
        experiment = np.append(experiment,4)
        
g1_counts['experiment'] = experiment

In [ ]:
#add another column to scores indicating whether it's the single or multiplayer
#TURN INTO FUNCTION

subjects = g2_counts.index.to_numpy()

condition = np.array([])

for i in subjects:
    if [i][0][0] == 's' or [i][0][0] == 'c':
        condition = np.append(condition,'control')
    elif [i][0][0] == 'm' or [i][0][0] == 'j':
        condition = np.append(condition,'social')

g2_counts['condition'] = condition

In [ ]:
#add column to joint_counts indicating whether it's SMAB3 or 4
subjects = g2_counts.index.to_numpy()

experiment = np.array([])

for i in subjects:
    if [i][0][0] == 'j' or [i][0][0] == 's':
        experiment = np.append(experiment,3)
    elif [i][0][0] == 'm' or [i][0][0] == 'c':
        experiment = np.append(experiment,4)
        
g2_counts['experiment'] = experiment

In [ ]:
g2_counts

In [ ]:
#combine g1_counts and g2_counts
counts = g1_counts.append(g2_counts)

#add column to counts indicating whether it's game 1 or 2
subjects = counts.index.to_numpy()

game = np.array([])

for i in subjects:
    if [i][0][8] == '1':
        game = np.append(game,1)
    elif [i][0][8] == '2':
        game = np.append(game,2)
        
counts['game'] = game
        

In [ ]:
#add participant column with subject code 
participant = np.array([])

for i in subjects:
    participant = np.append(participant, [i][0][0:6])
    
counts['participant'] = participant

In [ ]:
g2_counts

In [ ]:
#break up counts dataframe into 30,60,90
grouped = g1_counts.groupby(g1_counts.variable)
g1_counts_30 = grouped.get_group(30.0)
g1_counts_60 = grouped.get_group(60.0)
g1_counts_90 = grouped.get_group(90.0)

#break up counts dataframe into 30,60,90
grouped = g2_counts.groupby(g2_counts.variable)
g2_counts_30 = grouped.get_group(30.0)
g2_counts_60 = grouped.get_group(60.0)
g2_counts_90 = grouped.get_group(90.0)

In [ ]:
#break up counts dataframe into 30,60,90
grouped = counts.groupby(counts.variable)
counts_30 = grouped.get_group(30.0)
counts_60 = grouped.get_group(60.0)
counts_90 = grouped.get_group(90.0)

In [ ]:
#combine c_score and s_score
confed_g1_score = c_scores_1.append(s_scores_1)
confed_g1_score = confed_g1_score.melt(ignore_index=False)

In [ ]:
confed_g1_score

In [ ]:
#combine m_score and j_score
soc_g1_score = m_scores_1.append(j_scores_1)
soc_g1_score = soc_g1_score.melt(ignore_index=False)

In [ ]:
soc_g1_score

In [ ]:
import pingouin as pg

Mixed measures ANOVA with all of the data looking at condition (social vs control) as the between variable and game (1 vs 2) as the within variable

In [ ]:
counts_30

In [ ]:
counts_30.shape

# Testing for differences between SMAB3 and 4

In [ ]:
#run 2x2 between measures ANOVA for the 90 hole
pg.anova(data=g1_counts_90, dv='value', between=['experiment','condition'])

In [ ]:
#run 2x2 between measures ANOVA for the 90 hole
pg.anova(data=g1_counts_30, dv='value', between=['experiment','condition'])

In [ ]:
#run 2x2 between measures ANOVA for the 90 hole
pg.anova(data=g2_counts_90, dv='value', between=['experiment','condition'])

In [ ]:
#run 2x2 between measures ANOVA for the 90 hole
pg.anova(data=g2_counts_30, dv='value', between=['experiment','condition'])

# SMAB3 + 4 Combined Analyses

In [ ]:
pg.mixed_anova(data=counts_30, dv='value', between='condition', within='game', subject='participant')

In [ ]:
pg.mixed_anova(data=counts_90, dv='value', between='condition', within='game', subject='participant')

In [ ]:
pg.mixed_anova(data=counts_60, dv='value', between='condition', within='game', subject='participant')

In [ ]:
pg.qqplot(counts['value'])

In [ ]:
counts 

In [ ]:
#unmelt the counts df
counts = counts.pivot_table(index = counts.index, columns = ['variable'])

In [ ]:
counts

In [ ]:
'''This function calculates a weighted avg of the choice behavior and adds the average to a new column 
called avg for each row.'''
def weight_avg(df):
    avg = np.array([])
    for index, row in df.iterrows():
        value = (counts['value', 30.0][index] * 0.3) + (counts['value', 60.0][index] * 0.6) + (counts['value', 90.0][index] * 0.9) 
        avg = np.append(avg, value)
    df['avg'] = avg
    return df

In [ ]:
counts = weight_avg(counts)

In [ ]:
counts

In [ ]:
counts = counts.drop(['game','value','experiment'], axis = 1)

In [ ]:
#add participant column with subject code 
subjects = counts.index.to_numpy()

participant = np.array([])

for i in subjects:
    participant = np.append(participant, [i][0][0:6])
    
counts['participant'] = participant

game = np.array([])

for i in subjects:
    if [i][0][8] == '1':
        game = np.append(game,1)
    elif [i][0][8] == '2':
        game = np.append(game,2)
        
counts['game'] = game

experiment = np.array([])

for i in subjects:
    if [i][0][0] == 'j' or [i][0][0] == 's':
        experiment = np.append(experiment,3)
    elif [i][0][0] == 'm' or [i][0][0] == 'c':
        experiment = np.append(experiment,4)
        
counts['experiment'] = experiment

condition = np.array([])

for i in subjects:
    if [i][0][0] == 's' or [i][0][0] == 'c':
        condition = np.append(condition,'control')
    elif [i][0][0] == 'm' or [i][0][0] == 'j':
        condition = np.append(condition,'social')

counts['condition'] = condition

In [ ]:
#missing jo7_p2_g2
counts

In [ ]:
counts.shape

In [ ]:
counts = counts.sort_index()

In [ ]:
pg.mixed_anova(data=counts, dv='avg', between='condition', within='game', subject='participant')

In [ ]:
#split data into 2 halves and see if this is still true 
#run ANOVA for g2_s and g2_d 

# Plotting Combined Data

In [ ]:
soc_g1_counts_box = sns.boxplot( 
                 data=soc_g1_counts_unmelt,
                 width=0.5,
                 palette="pastel")
soc_g1_counts_box.axes.set_title("Social Choice Distributions in Game 1",
                    fontsize=16)

plot_file_name="soc_counts1_box.jpg"
 
plt.ylim(0, 100)
soc_g1_counts_box.axes.set(ylabel='Counts')

# save as jpeg
soc_g1_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
confed_g1_counts_box = sns.boxplot( 
                 data=confed_g1_counts_unmelt,
                 width=0.5,
                 palette="pastel")
confed_g1_counts_box.axes.set_title("Confederate Choice Distributions in Game 1",
                    fontsize=16)

plot_file_name="confed_counts1_box.jpg"
 
plt.ylim(0, 100)
confed_g1_counts_box.axes.set(ylabel='Counts')

# save as jpeg
confed_g1_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
sp.stats.ttest_ind(soc_g1_counts_unmelt, confed_g1_counts_unmelt)

In [ ]:
confed_g1_scores_box = sns.boxplot( 
                 data=confed_g1_score['value'],
                 width=0.5,
                 palette="pastel")
confed_g1_scores_box.axes.set_title("Confederate Scores in Game 1",
                    fontsize=16)

plot_file_name="confed_score1_box.jpg"
 
plt.ylim(0, 100)
confed_g1_scores_box.axes.set(ylabel='Score')

# save as jpeg
confed_g1_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
soc_g1_scores_box = sns.boxplot( 
                 data=soc_g1_score['value'],
                 width=0.5,
                 palette="pastel")
soc_g1_scores_box.axes.set_title("Social Scores in Game 1",
                    fontsize=16)

plot_file_name="soc_score1_box.jpg"
 
plt.ylim(0, 100)
soc_g1_scores_box.axes.set(ylabel='Score')

# save as jpeg
soc_g1_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
sp.stats.ttest_ind(soc_g1_score['value'], confed_g1_score['value'])

In [ ]:
soc_g2s_counts_box = sns.boxplot( 
                 data=soc_g2_s_counts_unmelt,
                 width=0.5,
                 palette="pastel")
soc_g2s_counts_box.axes.set_title("Social Choice Distributions in Game 2-Same",
                    fontsize=16)

plot_file_name="soc_counts2s_box.jpg"
 
plt.ylim(0, 100)
soc_g2s_counts_box.axes.set(ylabel='Counts')

# save as jpeg
soc_g2s_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
soc_g2_s_counts_unmelt

In [ ]:
soc_g2d_counts_box = sns.boxplot( 
                 data=soc_g2_d_counts_unmelt,
                 width=0.5,
                 palette="pastel")
soc_g2d_counts_box.axes.set_title("Social Choice Distributions in Game 2-Different",
                    fontsize=16)

plot_file_name="soc_counts2d_box.jpg"
 
plt.ylim(0, 100)
soc_g2d_counts_box.axes.set(ylabel='Counts')

# save as jpeg
soc_g2d_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
sp.stats.ttest_ind(soc_g2_d_counts_unmelt, soc_g2_s_counts_unmelt)

In [ ]:
sp.stats.ttest_ind(soc_g2_d_counts_unmelt, soc_g2_s_counts_unmelt)

In [ ]:
confed_g2d_counts_box = sns.boxplot( 
                 data=confed_g2_d_counts_unmelt,
                 width=0.5,
                 palette="pastel")
confed_g2d_counts_box.axes.set_title("Control Choice Distributions in Game 2-Different",
                    fontsize=16)

plot_file_name="confed_counts2d_box.jpg"
 
plt.ylim(0, 100)
confed_g2d_counts_box.axes.set(ylabel='Counts')

# save as jpeg
confed_g2d_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
confed_g2s_counts_box = sns.boxplot( 
                 data=confed_g2_s_counts_unmelt,
                 width=0.5,
                 palette="pastel")
confed_g2s_counts_box.axes.set_title("Control Choice Distributions in Game 2-Same",
                    fontsize=16)

plot_file_name="confed_counts2s_box.jpg"
 
plt.ylim(0, 100)
confed_g2s_counts_box.axes.set(ylabel='Counts')

# save as jpeg
confed_g2s_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
sp.stats.ttest_ind(confed_g2_d_counts_unmelt, confed_g2_s_counts_unmelt)

In [ ]:
soc_g2_s_scores_box = sns.boxplot( 
                 data=soc_g2_s_scores_unmelt,
                 width=0.5,
                 palette="pastel")
soc_g2_s_scores_box.axes.set_title("Social Scores in Game 2-Same",
                    fontsize=16)

plot_file_name="soc_g2_s_scores_box.jpg"
 
plt.ylim(0, 100)
soc_g2_s_scores_box.axes.set(ylabel='Score')

# save as jpeg
soc_g2_s_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
soc_g2_d_scores_box = sns.boxplot( 
                 data=soc_g2_d_scores_unmelt,
                 width=0.5,
                 palette="pastel")
soc_g2_d_scores_box.axes.set_title("Social Scores in Game 2-Different",
                    fontsize=16)

plot_file_name="soc_g2_d_scores_box.jpg"
 
plt.ylim(0, 100)
soc_g2_d_scores_box.axes.set(ylabel='Score')

# save as jpeg
soc_g2_d_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
sp.stats.ttest_ind(soc_g2_d_scores_unmelt, soc_g2_s_scores_unmelt)

In [ ]:
#NEED TO FIX
confed_g2_d_scores_box = sns.boxplot( 
                 data=confed_g2_d_scores_unmelt,
                 width=0.5,
                 palette="pastel")
confed_g2_d_scores_box.axes.set_title("Control Scores in Game 2-Different",
                    fontsize=16)

plot_file_name="confed_g2_d_scores_box.jpg"
 
plt.ylim(0, 100)
confed_g2_d_scores_box.axes.set(ylabel='Score')

# save as jpeg
confed_g2_d_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
confed_g2_s_scores_box = sns.boxplot( 
                 data=confed_g2_s_scores_unmelt,
                 width=0.5,
                 palette="pastel")
confed_g2_s_scores_box.axes.set_title("Control Scores in Game 2-Same",
                    fontsize=16)

plot_file_name="confed_g2_s_scores_box.jpg"
 
plt.ylim(0, 100)
confed_g2_s_scores_box.axes.set(ylabel='Score')

# save as jpeg
confed_g2_s_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
sp.stats.ttest_ind(confed_g2_d_scores_unmelt, confed_g2_s_scores_unmelt)

In [ ]:
sp.stats.ttest_ind(confed_g2_d_scores_unmelt, confed_g1_score['value'])

In [ ]:
sp.stats.ttest_ind(confed_g2_s_scores_unmelt, confed_g1_score['value'])

In [ ]:
sp.stats.ttest_ind(confed_g2_d_counts_unmelt, confed_g1_counts_unmelt)

In [ ]:
sp.stats.ttest_ind(confed_g2_s_counts_unmelt, confed_g1_counts_unmelt)

# JSD Analysis of the Combined Data

In [ ]:
soc_g1_counts

In [ ]:
soc_g2_counts_unmelt = soc_g2_d_counts_unmelt.append(soc_g2_s_counts_unmelt)

In [ ]:
soc_g2_counts_unmelt

In [ ]:
confed_g1_counts_unmelt

In [ ]:
confed_g2_counts_unmelt = confed_g2_d_counts_unmelt.append(confed_g2_s_counts_unmelt)

In [ ]:
confed_g2_counts_unmelt

# Maximizing vs Matching Social Condition

In [ ]:
m01_p1_g1_count

In [ ]:
# # Maximizing/Matching SOCIAL

# In[ ]:


from scipy import stats


# In[ ]:


#Optional: Idealized probability distributions to use for comparison. These assume a social strategy in which the probability
#distributions of both partners in a joint session are averaged (all holes reward at a 60% rate)

#social_max = [0, 0, 100]
#social_match = [30/180 * 100,60/180 * 100, 90/180 * 100]


# In[ ]:


#Calls jsd with the social distributions.
#def call_jsd_social(counts):
   # maxi = jsd(counts,social_max)
   # matchi = jsd(counts,social_match)
    #return maxi,matchi


# In[ ]:


j01_p1_g1_max,j01_p1_g1_match,j01_p1_g1_rand,j01_p1_g1_soc = call_jsd(j01_p1_g1_count)
j01_p1_g2_max,j01_p1_g2_match,j01_p1_g2_rand,j01_p1_g2_soc = call_jsd(j01_p1_g2_count)
j01_p2_g1_max,j01_p2_g1_match,j01_p2_g1_rand,j01_p2_g1_soc = call_jsd(j01_p2_g1_count)
j01_p2_g2_max,j01_p2_g2_match,j01_p2_g2_rand,j01_p2_g2_soc = call_jsd(j01_p2_g2_count)
j02_p1_g1_max,j02_p1_g1_match,j02_p1_g1_rand,j02_p1_g1_soc = call_jsd(j02_p1_g1_count)
j02_p1_g2_max,j02_p1_g2_match,j02_p1_g2_rand,j02_p1_g2_soc = call_jsd(j02_p1_g2_count)
j02_p2_g1_max,j02_p2_g1_match,j02_p2_g1_rand,j02_p2_g1_soc = call_jsd(j02_p2_g1_count)
j02_p2_g2_max,j02_p2_g2_match,j02_p2_g2_rand,j02_p2_g2_soc = call_jsd(j02_p2_g2_count)
j03_p1_g1_max,j03_p1_g1_match,j03_p1_g1_rand,j03_p1_g1_soc = call_jsd(j03_p1_g1_count)
j03_p1_g2_max,j03_p1_g2_match,j03_p1_g2_rand,j03_p1_g2_soc = call_jsd(j03_p1_g2_count)
j03_p2_g1_max,j03_p2_g1_match,j03_p2_g1_rand,j03_p2_g1_soc  = call_jsd(j03_p2_g1_count)
j03_p2_g2_max,j03_p2_g2_match,j03_p2_g2_rand,j03_p2_g2_soc = call_jsd(j03_p2_g2_count)
j04_p1_g1_max,j04_p1_g1_match,j04_p1_g1_rand,j04_p1_g1_soc = call_jsd(j04_p1_g1_count)
j04_p1_g2_max,j04_p1_g2_match,j04_p1_g2_rand,j04_p1_g2_soc = call_jsd(j04_p1_g2_count)
j04_p2_g1_max,j04_p2_g1_match,j04_p2_g1_rand,j04_p2_g1_soc  = call_jsd(j04_p2_g1_count)
j04_p2_g2_max,j04_p2_g2_match,j04_p2_g2_rand,j04_p2_g2_soc = call_jsd(j04_p2_g2_count)
j05_p1_g1_max,j05_p1_g1_match,j05_p1_g1_rand,j05_p1_g1_soc = call_jsd(j05_p1_g1_count)
j05_p1_g2_max,j05_p1_g2_match,j05_p1_g2_rand,j05_p1_g2_soc = call_jsd(j05_p1_g2_count)
j05_p2_g1_max,j05_p2_g1_match,j05_p2_g1_rand,j05_p2_g1_soc = call_jsd(j05_p2_g1_count)
j05_p2_g2_max,j05_p2_g2_match,j05_p2_g2_rand,j05_p2_g2_soc = call_jsd(j05_p2_g2_count)
j06_p1_g1_max,j06_p1_g1_match,j06_p1_g1_rand,j06_p1_g1_soc = call_jsd(j06_p1_g1_count)
j06_p1_g2_max,j06_p1_g2_match,j06_p1_g2_rand,j06_p1_g2_soc = call_jsd(j06_p1_g2_count)
j06_p2_g1_max,j06_p2_g1_match,j06_p2_g1_rand,j06_p2_g1_soc = call_jsd(j06_p2_g1_count)
j06_p2_g2_max,j06_p2_g2_match,j06_p2_g2_rand,j06_p2_g2_soc = call_jsd(j06_p2_g2_count)
j07_p1_g1_max,j07_p1_g1_match,j07_p1_g1_rand,j07_p1_g1_soc = call_jsd(j07_p1_g1_count)
j07_p1_g2_max,j07_p1_g2_match,j07_p1_g2_rand,j07_p1_g2_soc = call_jsd(j07_p1_g2_count)
j07_p2_g1_max,j07_p2_g1_match,j07_p2_g1_rand,j07_p2_g1_soc = call_jsd(j07_p2_g1_count)
j07_p2_g2_max,j07_p2_g2_match,j07_p2_g2_rand,j07_p2_g2_soc = call_jsd(j07_p2_g2_count)
j08_p1_g1_max,j08_p1_g1_match,j08_p1_g1_rand,j08_p1_g1_soc = call_jsd(j08_p1_g1_count)
j08_p1_g2_max,j08_p1_g2_match,j08_p1_g2_rand,j08_p1_g2_soc = call_jsd(j08_p1_g2_count)
j08_p2_g1_max,j08_p2_g1_match,j08_p2_g1_rand,j08_p2_g1_soc = call_jsd(j08_p2_g1_count)
j08_p2_g2_max,j08_p2_g2_match,j08_p2_g2_rand,j08_p2_g2_soc = call_jsd(j08_p2_g2_count)
j10_p1_g1_max,j10_p1_g1_match,j10_p1_g1_rand,j10_p1_g1_soc = call_jsd(j10_p1_g1_count)
j10_p1_g2_max,j10_p1_g2_match,j10_p1_g2_rand,j10_p1_g2_soc = call_jsd(j10_p1_g2_count)
j10_p2_g1_max,j10_p2_g1_match,j10_p2_g1_rand,j10_p2_g1_soc = call_jsd(j10_p2_g1_count)
j10_p2_g2_max,j10_p2_g2_match,j10_p2_g2_rand,j10_p2_g2_soc = call_jsd(j10_p2_g2_count)
j11_p1_g1_max,j11_p1_g1_match,j11_p1_g1_rand,j11_p1_g1_soc = call_jsd([0,40,60])
j11_p1_g2_max,j11_p1_g2_match,j11_p1_g2_rand,j11_p1_g2_soc = call_jsd([0,22,78])
j11_p2_g1_max,j11_p2_g1_match,j11_p2_g1_rand,j11_p2_g1_soc = call_jsd(j11_p2_g1_count)
j11_p2_g2_max,j11_p2_g2_match,j11_p2_g2_rand,j11_p2_g2_soc = call_jsd([0,1,99])
m01_p1_g1_max,m01_p1_g1_match,m01_p1_g1_rand,m01_p1_g1_soc = call_jsd(m01_p1_g1_count)
m01_p1_g2_max,m01_p1_g2_match,m01_p1_g2_rand,m01_p1_g2_soc = call_jsd(m01_p1_g2_count)
m01_p2_g1_max,m01_p2_g1_match,m01_p2_g1_rand,m01_p2_g1_soc = call_jsd(m01_p2_g1_count)
m01_p2_g2_max,m01_p2_g2_match,m01_p2_g2_rand,m01_p2_g2_soc = call_jsd(m01_p2_g2_count)
m03_p1_g1_max,m03_p1_g1_match,m03_p1_g1_rand,m03_p1_g1_soc = call_jsd(m03_p1_g1_count)
m03_p1_g2_max,m03_p1_g2_match,m03_p1_g2_rand,m03_p1_g2_soc = call_jsd(m03_p1_g2_count)
m03_p2_g1_max,m03_p2_g1_match,m03_p2_g1_rand,m03_p2_g1_soc = call_jsd(m03_p2_g1_count)
m03_p2_g2_max,m03_p2_g2_match,m03_p2_g2_rand,m03_p2_g2_soc = call_jsd(m03_p2_g2_count)
m05_p1_g1_max,m05_p1_g1_match,m05_p1_g1_rand,m05_p1_g1_soc = call_jsd(m05_p1_g1_count)
m05_p1_g2_max,m05_p1_g2_match,m05_p1_g2_rand,m05_p1_g2_soc = call_jsd(m05_p1_g2_count)
m05_p2_g1_max,m05_p2_g1_match,m05_p2_g1_rand,m05_p2_g1_soc = call_jsd(m05_p2_g1_count)
m05_p2_g2_max,m05_p2_g2_match,m05_p2_g2_rand,m05_p2_g2_soc = call_jsd(m05_p2_g2_count)
m06_p1_g1_max,m06_p1_g1_match,m06_p1_g1_rand,m06_p1_g1_soc = call_jsd(m06_p1_g1_count)
m06_p1_g2_max,m06_p1_g2_match,m06_p1_g2_rand,m06_p1_g2_soc = call_jsd(m06_p1_g2_count)
m06_p2_g1_max,m06_p2_g1_match,m06_p2_g1_rand,m06_p2_g1_soc = call_jsd(m06_p2_g1_count)
m06_p2_g2_max,m06_p2_g2_match,m06_p2_g2_rand,m06_p2_g2_soc = call_jsd(m06_p2_g2_count)
m08_p1_g1_max,m08_p1_g1_match,m08_p1_g1_rand,m08_p1_g1_soc = call_jsd(m08_p1_g1_count)
m08_p1_g2_max,m08_p1_g2_match,m08_p1_g2_rand,m08_p1_g2_soc = call_jsd(m08_p1_g2_count)
m08_p2_g1_max,m08_p2_g1_match,m08_p2_g1_rand,m08_p2_g1_soc = call_jsd(m08_p2_g1_count)
m08_p2_g2_max,m08_p2_g2_match,m08_p2_g2_rand,m08_p2_g2_soc = call_jsd(m08_p2_g2_count)
m10_p1_g1_max,m10_p1_g1_match,m10_p1_g1_rand,m10_p1_g1_soc = call_jsd(m10_p1_g1_count)
m10_p1_g2_max,m10_p1_g2_match,m10_p1_g2_rand,m10_p1_g2_soc = call_jsd(m10_p1_g2_count)
m10_p2_g1_max,m10_p2_g1_match,m10_p2_g1_rand,m10_p2_g1_soc = call_jsd(m10_p2_g1_count)
m10_p2_g2_max,m10_p2_g2_match,m10_p2_g2_rand,m10_p2_g2_soc = call_jsd(m10_p2_g2_count)
m11_p1_g1_max,m11_p1_g1_match,m11_p1_g1_rand,m11_p1_g1_soc = call_jsd(m11_p1_g1_count)
m11_p1_g2_max,m11_p1_g2_match,m11_p1_g2_rand,m11_p1_g2_soc = call_jsd([0,1,99])
m11_p2_g1_max,m11_p2_g1_match,m11_p2_g1_rand,m11_p2_g1_soc = call_jsd(m11_p2_g1_count)
m11_p2_g2_max,m11_p2_g2_match,m11_p2_g2_rand,m11_p2_g2_soc = call_jsd([0,0,100])
m13_p1_g1_max,m13_p1_g1_match,m13_p1_g1_rand,m13_p1_g1_soc = call_jsd(m13_p1_g1_count)
m13_p1_g2_max,m13_p1_g2_match,m13_p1_g2_rand,m13_p1_g2_soc = call_jsd(m13_p1_g2_count)
m13_p2_g1_max,m13_p2_g1_match,m13_p2_g1_rand,m13_p2_g1_soc = call_jsd(m13_p2_g1_count)
m13_p2_g2_max,m13_p2_g2_match,m13_p2_g2_rand,m13_p2_g2_soc = call_jsd(m13_p2_g2_count)
m14_p1_g1_max,m14_p1_g1_match,m14_p1_g1_rand,m14_p1_g1_soc = call_jsd(m14_p1_g1_count)
m14_p1_g2_max,m14_p1_g2_match,m14_p1_g2_rand,m14_p1_g2_soc = call_jsd(m14_p1_g2_count)
m14_p2_g1_max,m14_p2_g1_match,m14_p2_g1_rand,m14_p2_g1_soc = call_jsd(m14_p2_g1_count)
m14_p2_g2_max,m14_p2_g2_match,m14_p2_g2_rand,m14_p2_g2_soc = call_jsd(m14_p2_g2_count)

In [ ]:
#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
social_jsd_g1 = pd.DataFrame({'max':[j01_p1_g1_max,j01_p2_g1_max,j02_p1_g1_max,j02_p2_g1_max,j03_p1_g1_max,j03_p2_g1_max,
                             j04_p1_g1_max,j04_p2_g1_max,j05_p1_g1_max,j05_p2_g1_max,j06_p1_g1_max,j06_p2_g1_max,
                             j07_p1_g1_max,j07_p2_g1_max,j08_p1_g1_max,j08_p2_g1_max, j10_p1_g1_max, j10_p2_g1_max,
                               j11_p1_g1_max, j11_p2_g1_max, m01_p1_g1_max, m01_p2_g1_max, m03_p1_g1_max, m03_p2_g1_max,
                                     m05_p1_g1_max, m05_p2_g1_max,
                                   m08_p1_g1_max, m08_p2_g1_max, m10_p1_g1_max, m10_p2_g1_max, m11_p1_g1_max, m11_p2_g1_max,
                                    m13_p1_g1_max, m13_p2_g1_max, m14_p1_g1_max, m14_p2_g1_max],
                    'match':[j01_p1_g1_match,j01_p2_g1_match,j02_p1_g1_match, j02_p2_g1_match,j03_p1_g1_match,j03_p2_g1_match,
                               j04_p1_g1_match,j04_p2_g1_match,j05_p1_g1_match,
                               j05_p2_g1_match,j06_p1_g1_match,j06_p2_g1_match,
                               j07_p1_g1_match,j07_p2_g1_match,j08_p1_g1_match,j08_p2_g1_match, j10_p1_g1_match,
                               j10_p2_g1_match, j11_p1_g1_match, j11_p2_g1_match, m01_p1_g1_match, m01_p2_g1_match,
                             m03_p1_g1_match, m03_p2_g1_match, m05_p1_g1_match, m05_p2_g1_match,
                                m08_p1_g1_match, m08_p2_g1_match, m10_p1_g1_match,
                               m10_p2_g1_match, m11_p1_g1_match, m11_p2_g1_match, m13_p1_g1_match, m13_p2_g1_match,
                               m14_p1_g1_match, m14_p2_g1_match],
                    'rand':[j01_p1_g1_rand,j01_p2_g1_rand,j02_p1_g1_rand,j02_p2_g1_rand,j03_p1_g1_rand,j03_p2_g1_rand,
                             j04_p1_g1_rand,j04_p2_g1_rand,j05_p1_g1_rand,j05_p2_g1_rand,j06_p1_g1_rand,j06_p2_g1_rand,
                             j07_p1_g1_rand,j07_p2_g1_rand,j08_p1_g1_rand,j08_p2_g1_rand, j10_p1_g1_rand, j10_p2_g1_rand,
                               j11_p1_g1_rand, j11_p2_g1_rand, m01_p1_g1_rand, m01_p2_g1_rand, m03_p1_g1_rand, m03_p2_g1_rand,
                                m05_p1_g1_rand, m05_p2_g1_rand,
                                   m08_p1_g1_rand, m08_p2_g1_rand, m10_p1_g1_rand, m10_p2_g1_rand, m11_p1_g1_rand, m11_p2_g1_rand,
                                    m13_p1_g1_rand, m13_p2_g1_rand, m14_p1_g1_rand, m14_p2_g1_rand],
                    'soc':[j01_p1_g1_soc,j01_p2_g1_soc,j02_p1_g1_soc, j02_p2_g1_soc,j03_p1_g1_soc,j03_p2_g1_soc,
                               j04_p1_g1_soc,j04_p2_g1_soc,j05_p1_g1_soc,
                               j05_p2_g1_soc,j06_p1_g1_soc,j06_p2_g1_soc,
                               j07_p1_g1_soc,j07_p2_g1_soc,j08_p1_g1_soc,j08_p2_g1_soc, j10_p1_g1_soc,
                               j10_p2_g1_soc, j11_p1_g1_soc, j11_p2_g1_soc, m01_p1_g1_soc, m01_p2_g1_soc, 
                               m03_p1_g1_soc, m03_p2_g1_soc, m05_p1_g1_soc, m05_p2_g1_soc,
                               m08_p1_g1_soc, m08_p2_g1_soc, m10_p1_g1_soc,
                               m10_p2_g1_soc, m11_p1_g1_soc, m11_p2_g1_soc, m13_p1_g1_soc, m13_p2_g1_soc,
                               m14_p1_g1_soc, m14_p2_g1_soc]},
                         index=['j01_p1', 'j01_p2', 'j02_p1', 'j02_p2', 'j03_p1', 'j03_p2','j04_p1',
                                'j04_p2','j05_p1', 'j05_p2', 'j06_p1', 'j06_p2','j07_p1','j07_p1','j08_p1','j08_p2',
                                'j10_p1', 'j10_p2', 'j11_p1', 'j11_p2','m01_p1','m01_p2','m03_p1','m03_p2','m05_p1',
                                'm05_p2','m08_p1','m08_p2', 'm10_p1', 'm10_p2', 'm11_p1', 'm11_p2', 'm13_p1','m13_p2',
                                'm14_p1', 'm14_p2'])


# In[ ]:


#to print wothout truncation
pd.set_option('display.max_rows', None)


# In[ ]:


social_jsd_g1




# get the column name of min values in every row
strategy_social_jsd_g1 = social_jsd_g1.idxmin(axis=1)
 
print("min values of row are at following columns :")
print(strategy_social_jsd_g1)





strategy_social_jsd_g1.value_counts()


# In[ ]:


#social_types_g1 = check_type(social_jsd_g1)


# In[ ]:


#social_types_g1.drop([0])


# In[ ]:


#(social_types_g1 == 'maxi').sum()

In [ ]:
m01_p1_g1_rand

In [ ]:
social_jsd_g1

In [ ]:
sp.stats.ttest_ind(control_jsd_g1,social_jsd_g1)

In [ ]:
sp.stats.ttest_ind(control_jsd_g1,social_jsd_g1)

In [ ]:
#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
social_jsd_g2 = pd.DataFrame({'max':[j01_p1_g2_max,j01_p2_g2_max,j02_p1_g2_max,j02_p2_g2_max,j03_p1_g2_max,j03_p2_g2_max,
                             j04_p1_g2_max,j04_p2_g2_max,j05_p1_g2_max,j05_p2_g2_max,j06_p1_g2_max,j06_p2_g2_max,
                             j07_p1_g2_max,j07_p2_g2_max,j08_p1_g2_max,j08_p2_g2_max, j10_p1_g2_max, j10_p2_g2_max,
                               j11_p1_g2_max, j11_p2_g2_max, m01_p1_g2_max, m01_p2_g2_max, m03_p1_g2_max, m03_p2_g2_max,
                                     m05_p1_g2_max, m05_p2_g2_max, m08_p1_g2_max, m08_p2_g2_max, m10_p1_g2_max, m10_p2_g2_max, m11_p1_g2_max, m11_p2_g2_max,
                                    m13_p1_g2_max, m13_p2_g2_max, m14_p1_g2_max, m14_p2_g2_max],
                    'match':[j01_p1_g2_match,j01_p2_g2_match,j02_p1_g2_match, j02_p2_g2_match,j03_p1_g2_match,j03_p2_g2_match,
                               j04_p1_g2_match,j04_p2_g2_match,j05_p1_g2_match,
                               j05_p2_g2_match,j06_p1_g2_match,j06_p2_g2_match,
                               j07_p1_g2_match,j07_p2_g2_match,j08_p1_g2_match,j08_p2_g2_match, j10_p1_g2_match,
                               j10_p2_g2_match, j11_p1_g2_match, j11_p2_g2_match, m01_p1_g2_match, m01_p2_g2_match,m03_p1_g2_match, 
                               m03_p2_g2_match, m05_p1_g2_match, m05_p2_g2_match,
                               m08_p1_g2_match, m08_p2_g2_match, m10_p1_g2_match,
                               m10_p2_g2_match, m11_p1_g2_match, m11_p2_g2_match, m13_p1_g2_match, m13_p2_g2_match,
                               m14_p1_g2_match, m14_p2_g2_match],
                    'rand':[j01_p1_g2_rand,j01_p2_g2_rand,j02_p1_g2_rand,j02_p2_g2_rand,j03_p1_g2_rand,j03_p2_g2_rand,
                             j04_p1_g2_rand,j04_p2_g2_rand,j05_p1_g2_rand,j05_p2_g2_rand,j06_p1_g2_rand,j06_p2_g2_rand,
                             j07_p1_g2_rand,j07_p2_g2_rand,j08_p1_g2_rand,j08_p2_g2_rand, j10_p1_g2_rand, j10_p2_g2_rand,
                               j11_p1_g2_rand, j11_p2_g2_rand, m01_p1_g2_rand, m01_p2_g2_rand,m03_p1_g2_rand, m03_p2_g2_rand,
                            m05_p1_g2_rand, m05_p2_g2_rand,
                                   m08_p1_g2_rand, m08_p2_g2_rand, m10_p1_g2_rand, m10_p2_g2_rand, m11_p1_g2_rand, m11_p2_g2_rand,
                                    m13_p1_g2_rand, m13_p2_g2_rand, m14_p1_g2_rand, m14_p2_g2_rand],
                    'soc':[j01_p1_g2_soc,j01_p2_g2_soc,j02_p1_g2_soc, j02_p2_g2_soc,j03_p1_g2_soc,j03_p2_g2_soc,
                               j04_p1_g2_soc,j04_p2_g2_soc,j05_p1_g2_soc,
                               j05_p2_g2_soc,j06_p1_g2_soc,j06_p2_g2_soc,
                               j07_p1_g2_soc,j07_p2_g2_soc,j08_p1_g2_soc,j08_p2_g2_soc, j10_p1_g2_soc,
                               j10_p2_g2_soc, j11_p1_g2_soc, j11_p2_g2_soc, m01_p1_g2_soc, m01_p2_g2_soc,m03_p1_g2_soc, 
                           m03_p2_g2_soc,
                           m05_p1_g2_soc, m05_p2_g2_soc,
                               m08_p1_g2_soc, m08_p2_g2_soc, m10_p1_g2_soc,
                               m10_p2_g2_soc, m11_p1_g2_soc, m11_p2_g2_soc, m13_p1_g2_soc, m13_p2_g2_soc,
                               m14_p1_g2_soc, m14_p2_g2_soc]},
                         index=['j01_p1', 'j01_p2', 'j02_p1', 'j02_p2', 'j03_p1', 'j03_p2','j04_p1',
                                'j04_p2','j05_p1', 'j05_p2', 'j06_p1', 'j06_p2','j07_p1','j07_p1','j08_p1','j08_p2',
                                'j10_p1', 'j10_p2', 'j11_p1', 'j11_p2','m01_p1','m01_p2','m03_p1','m03_p2','m05_p1',
                                'm05_p2','m08_p1','m08_p2', 'm10_p1', 'm10_p2', 'm11_p1', 'm11_p2', 'm13_p1','m13_p2',
                                'm14_p1', 'm14_p2'])


strategy_social_jsd_g2 = social_jsd_g2.idxmin(axis=1)

print(strategy_social_jsd_g2)


strategy_social_jsd_g2.value_counts()

In [ ]:
#can use to see strategy development from g1 to g2
social_jsd_strategy = strategy_social_jsd_g1.append(strategy_social_jsd_g2)
social_jsd_strategy

In [ ]:
#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
social_jsd_g2d = pd.DataFrame({'max':[j01_p1_g2_max,j01_p2_g2_max,j02_p1_g2_max,j02_p2_g2_max,j03_p1_g2_max,j03_p2_g2_max,
                             j04_p1_g2_max,j04_p2_g2_max,j05_p1_g2_max,j05_p2_g2_max,j06_p1_g2_max,j06_p2_g2_max,
                             j07_p1_g2_max,j07_p2_g2_max,j08_p1_g2_max,j08_p2_g2_max,m01_p1_g2_max, m01_p2_g2_max, m03_p1_g2_max, m03_p2_g2_max,
                                     m05_p1_g2_max, m05_p2_g2_max],
                    'match':[j01_p1_g2_match,j01_p2_g2_match,j02_p1_g2_match, j02_p2_g2_match,j03_p1_g2_match,j03_p2_g2_match,
                               j04_p1_g2_match,j04_p2_g2_match,j05_p1_g2_match,
                               j05_p2_g2_match,j06_p1_g2_match,j06_p2_g2_match,
                               j07_p1_g2_match,j07_p2_g2_match,j08_p1_g2_match,j08_p2_g2_match, m01_p1_g2_match, m01_p2_g2_match,m03_p1_g2_match, 
                               m03_p2_g2_match, m05_p1_g2_match, m05_p2_g2_match],
                    'rand':[j01_p1_g2_rand,j01_p2_g2_rand,j02_p1_g2_rand,j02_p2_g2_rand,j03_p1_g2_rand,j03_p2_g2_rand,
                             j04_p1_g2_rand,j04_p2_g2_rand,j05_p1_g2_rand,j05_p2_g2_rand,j06_p1_g2_rand,j06_p2_g2_rand,
                             j07_p1_g2_rand,j07_p2_g2_rand,j08_p1_g2_rand,j08_p2_g2_rand, m01_p1_g2_rand, m01_p2_g2_rand,
                            m03_p1_g2_rand, m03_p2_g2_rand,
                            m05_p1_g2_rand, m05_p2_g2_rand],
                    'soc':[j01_p1_g2_soc,j01_p2_g2_soc,j02_p1_g2_soc, j02_p2_g2_soc,j03_p1_g2_soc,j03_p2_g2_soc,
                               j04_p1_g2_soc,j04_p2_g2_soc,j05_p1_g2_soc,
                               j05_p2_g2_soc,j06_p1_g2_soc,j06_p2_g2_soc,
                               j07_p1_g2_soc,j07_p2_g2_soc,j08_p1_g2_soc,j08_p2_g2_soc,
                           m01_p1_g2_soc, m01_p2_g2_soc,m03_p1_g2_soc, 
                           m03_p2_g2_soc,
                           m05_p1_g2_soc, m05_p2_g2_soc]},
                         index=['j01', 'j01', 'j02', 'j02', 'j03', 'j03','j04',
                                'j04','j05', 'j05', 'j06', 'j06','j07','j07','j08','j08',
                                'm01','m01','m03','m03','m05','m05'])

In [ ]:
#Create a dataframe for single player game 1 jsd scores, with a column for each strategy.'''
social_jsd_g2s = pd.DataFrame({'max':[j10_p1_g2_max, j10_p2_g2_max,
                               j11_p1_g2_max, j11_p2_g2_max, m08_p1_g2_max, m08_p2_g2_max, m10_p1_g2_max, 
                                      m10_p2_g2_max, m11_p1_g2_max, m11_p2_g2_max,
                                    m13_p1_g2_max, m13_p2_g2_max, m14_p1_g2_max, m14_p2_g2_max],
                    'match':[j10_p1_g2_match, j10_p2_g2_match, j11_p1_g2_match, j11_p2_g2_match, 
                               m08_p1_g2_match, m08_p2_g2_match, m10_p1_g2_match,
                               m10_p2_g2_match, m11_p1_g2_match, m11_p2_g2_match, m13_p1_g2_match, m13_p2_g2_match,
                               m14_p1_g2_match, m14_p2_g2_match],
                    'rand':[j10_p1_g2_rand, j10_p2_g2_rand,
                               j11_p1_g2_rand, j11_p2_g2_rand,
                                   m08_p1_g2_rand, m08_p2_g2_rand, m10_p1_g2_rand, m10_p2_g2_rand, m11_p1_g2_rand, m11_p2_g2_rand,
                                    m13_p1_g2_rand, m13_p2_g2_rand, m14_p1_g2_rand, m14_p2_g2_rand],
                    'soc':[j10_p1_g2_soc, j10_p2_g2_soc, j11_p1_g2_soc, j11_p2_g2_soc,
                               m08_p1_g2_soc, m08_p2_g2_soc, m10_p1_g2_soc,
                               m10_p2_g2_soc, m11_p1_g2_soc, m11_p2_g2_soc, m13_p1_g2_soc, m13_p2_g2_soc,
                               m14_p1_g2_soc, m14_p2_g2_soc]},
                         index=['j10', 'j10', 'j11', 'j11',
                                'm08', 'm08', 'm10', 'm10', 'm11', 'm11', 'm13','m13', 'm14', 'm14'])

In [ ]:
# get the column name of min values in every row
strategy_social_jsd_g2s = social_jsd_g2s.idxmin(axis=1)

strategy_social_jsd_g2d = social_jsd_g2d.idxmin(axis=1)

In [ ]:
strategy_social_jsd_g2s.value_counts()

In [ ]:
strategy_social_jsd_g2d.value_counts()

In [ ]:
sp.stats.ttest_ind(social_jsd_g1, social_jsd_g2)

In [ ]:
sp.stats.ttest_ind(social_jsd_g1['rand'], social_jsd_g2['rand'])

In [ ]:
sp.stats.ttest_ind(social_jsd_g1['match'],social_jsd_g2['match'])

In [ ]:
# In[ ]:


#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g1['max'], control_jsd_g1['match'], 1)

social_ego_jsd_g1_plot = social_jsd_g1.plot.scatter(x='max',y='match',s=50)
plt.title('Social Egocentric Strategies Game 1', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('social_ego_jsd_1.jpeg',format='jpeg')
plt.show()

In [ ]:
# In[ ]:


#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g1['max'], control_jsd_g1['match'], 1)

social_ego_jsd_g2d_plot = social_jsd_g2d.plot.scatter(x='max',y='match',s=50)
plt.title('Social Egocentric Strategies Game 2-Different', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('social_ego_jsd_2d.jpeg',format='jpeg')
plt.show()

In [ ]:
# In[ ]:


#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g1['max'], control_jsd_g1['match'], 1)

social_ego_jsd_g2s_plot = social_jsd_g2s.plot.scatter(x='max',y='match',s=50)
plt.title('Social Egocentric Strategies Game 2-Same', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('social_ego_jsd_2s.jpeg',format='jpeg')
plt.show()

In [ ]:
social_jsd_g2 = social_jsd_g2d.append(social_jsd_g2s)

In [ ]:
# In[ ]:


#under line = matching (closer to 0)
#plots of actual jsd values

m, b = np.polyfit(control_jsd_g1['max'], control_jsd_g1['match'], 1)

social_ego_jsd_g2_plot = social_jsd_g2.plot.scatter(x='max',y='match',s=50)
plt.title('Social Egocentric Strategies Game 2- Combined', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('social_ego_jsd_2.jpeg',format='jpeg')
plt.show()

In [ ]:
sp.stats.ttest_ind(social_jsd_g2s, social_jsd_g2d)

In [ ]:
sp.stats.ttest_ind(control_jsd_g1, social_jsd_g1)

In [ ]:
sp.stats.ttest_ind(control_jsd_g2, social_jsd_g2)

In [ ]:
sp.stats.ttest_rel(control_jsd_g1, control_jsd_g2)

In [ ]:
sp.stats.ttest_rel(social_jsd_g1, social_jsd_g2)

In [ ]:
social_jsd_g2['rand'].sum()

In [ ]:
social_jsd_g2['max'].sum()

In [ ]:
social_jsd_g2['match'].sum()

In [ ]:
social_jsd_g1['rand'].sum()

In [ ]:
social_jsd_g1['max'].sum()

In [ ]:
social_jsd_g1['match'].sum()

In [ ]:
social_jsd_g1.mean()

In [ ]:
social_jsd_g2.mean()

In [ ]:
confed_g2_counts = confed_g2_s_counts_unmelt.append(confed_g2_d_counts_unmelt)

In [ ]:
soc_g2_counts = soc_g2_s_counts_unmelt.append(soc_g2_d_counts_unmelt)

In [ ]:
sp.stats.ttest_ind(confed_g1_counts_unmelt,confed_g2_counts)

In [ ]:
sp.stats.ttest_ind(soc_g2_s_counts_unmelt,soc_g2_d_counts_unmelt)

In [ ]:
soc_g1_counts_unmelt.mean()

In [ ]:
soc_g2_counts.mean()

In [ ]:
sp.stats.ttest_ind(soc_g1_counts_unmelt,soc_g2_d_counts_unmelt)

In [ ]:
sp.stats.ttest_ind(soc_g1_counts_unmelt,soc_g2_s_counts_unmelt)

In [ ]:
sp.stats.ttest_ind(soc_g2_d_scores_unmelt, soc_g2_s_scores_unmelt)

In [ ]:
sp.stats.ttest_ind(soc_g1_score['value'], soc_g2_s_scores_unmelt)

In [ ]:
sp.stats.ttest_ind(soc_g1_score['value'], soc_g2_d_scores_unmelt)

In [ ]:
soc_g2_scores = soc_g2_d_scores_unmelt.append(soc_g2_s_scores_unmelt)

In [ ]:
sp.stats.ttest_ind(soc_g1_score['value'],soc_g2_scores)

In [ ]:
sp.stats.ttest_ind(confed_g2_d_scores_unmelt, confed_g2_s_scores_unmelt)

In [ ]:
confed_g2_scores = confed_g2_d_scores_unmelt.append(confed_g2_s_scores_unmelt)

In [ ]:
sp.stats.ttest_ind(confed_g1_score['value'], confed_g2_scores)

In [ ]:
sp.stats.ttest_ind(confed_g1_score['value'], soc_g1_score['value'])

In [ ]:
sp.stats.ttest_ind(confed_g2_scores, soc_g2_scores)

# Summing G1 and G2 scores

In [ ]:
confed_g1_score.sort_index()
confed_g2_scores.sort_index()

In [ ]:
confed_g1_score

In [ ]:
confed_combo_score = confed_g1_score['value'] + confed_g2_scores['score']

In [ ]:
confed_combo_score

In [ ]:
confed_combo_score.size

In [ ]:
soc_g1_score.sort_index()
soc_g2_scores.sort_index()

In [ ]:
soc_combo_score = soc_g1_score['value'] + soc_g2_scores['score_2']

In [ ]:
soc_combo_score

In [ ]:
soc_combo_score.size

In [ ]:
soc_combo_score_below_chance = soc_combo_score[soc_combo_score <= 120]
confed_combo_score_below_chance = confed_combo_score[confed_combo_score <= 120]

In [ ]:
soc_combo_score_below_chance

In [ ]:
confed_combo_score_below_chance

In [ ]:
confed_g1_score_below_chance = confed_g1_score[confed_g1_score['value'] <= 60]
confed_g2_score_below_chance = confed_g2_scores[confed_g2_scores['score'] <= 60]

In [ ]:
soc_g1_score_below_chance = soc_g1_score[soc_g1_score['value'] <= 60]
soc_g2_score_below_chance = soc_g2_scores[soc_g2_scores['score_2'] <= 60]

In [ ]:
confed_g1_score_below_chance

In [ ]:
soc_g1_score_below_chance

In [ ]:
confed_g2_score_below_chance

In [ ]:
soc_g2_score_below_chance

# Plotting combined g2 data

In [ ]:
def anova_label(df):
    #ANOVA for blocks, games, by condition
#add column to counts indicating whether it's game 1 or 2

#NEED: convert to function

    subjects = df.index.to_numpy()

    game = np.array([])
    
    #add participant column with subject code 
    participant = np.array([])

    condition = np.array([])
    
    experiment = np.array([])

    for i in subjects:
        print(i)
        if [i][0][8] == '1':
            game = np.append(game,'1')
        elif [i][0][8] == '2':
            game = np.append(game,'2')
            
        participant = np.append(participant, [i][0][0:6])
        
        if [i][0][0] == 's' or [i][0][0] == 'c':
            condition = np.append(condition,'control')
        elif [i][0][0] == 'm' or [i][0][0] == 'j':
            condition = np.append(condition,'social')
            
        if [i][0][0] == 'j' or [i][0][0] == 's':
            experiment = np.append(experiment,3)
        elif [i][0][0] == 'm' or [i][0][0] == 'c':
            experiment = np.append(experiment,4)

    df['game'] = game
    df['participant'] = participant
    df['condition'] = condition
    df['experiment'] = experiment
    
    df = df.reset_index(drop=True)
    
    return df

In [ ]:
soc_g2_scores_box = sns.boxplot( 
                 data=soc_g2_scores,
                 width=0.5,
                 palette="pastel")
soc_g2_scores_box.axes.set_title("Social Scores in Game 2-Combined",
                    fontsize=16)

plot_file_name="soc_g2_combo_scores_box.jpg"
 
plt.ylim(0, 100)
soc_g2_scores_box.axes.set(ylabel='Score')

# save as jpeg
soc_g2_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
soc_g2_counts_box = sns.boxplot( 
                 data=soc_g2_counts,
                 width=0.5,
                 palette="pastel")
soc_g2_counts_box.axes.set_title("Social Counts in Game 2-Combined",
                    fontsize=16)

plot_file_name="soc_g2_combo_combo_box.jpg"
 
plt.ylim(0, 100)
soc_g2_counts_box.axes.set(ylabel='Choice Distribution')

# save as jpeg
soc_g2_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
social_jsd_g2 = social_jsd_g2d.append(social_jsd_g2s)

In [ ]:
social_ego_jsd_g2_plot = social_jsd_g2.plot.scatter(x='max',y='match',s=50)
plt.title('Social Egocentric Strategies Game 2', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('social_ego_jsd_2.jpeg',format='jpeg')
plt.show()

In [ ]:
confed_g2_scores_box = sns.boxplot( 
                 data=confed_g2_scores,
                 width=0.5,
                 palette="pastel")
confed_g2_scores_box.axes.set_title("Confederate Scores in Game 2-Combined",
                    fontsize=16)

plot_file_name="confed_g2_combo_scores_box.jpg"
 
plt.ylim(0, 100)
confed_g2_scores_box.axes.set(ylabel='Score')

# save as jpeg
confed_g2_scores_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
confed_g2_counts_box = sns.boxplot( 
                 data=confed_g2_counts,
                 width=0.5,
                 palette="pastel")
confed_g2_counts_box.axes.set_title("Confederate Counts in Game 2-Combined",
                    fontsize=16)

plot_file_name="confed_g2_combo_combo_box.jpg"
 
plt.ylim(0, 100)
confed_g2_counts_box.axes.set(ylabel='Choice Distribution')

# save as jpeg
confed_g2_counts_box.figure.savefig(plot_file_name,
                    format='jpeg')

In [ ]:
#confed_jsd_g2 = confed_jsd_g2_d.append(confed_jsd_g2_s)

In [ ]:
#confed_ego_jsd_g2_plot = confed_jsd_g2.plot.scatter(x='max',y='match',s=50)
plt.title('Confederate Egocentric Strategies Game 2', fontsize=18, y=1.05)
plt.xticks(np.arange(0, 1, step=0.1))
plt.yticks(np.arange(0, 1, step=0.1))
#line1 = mlines.Line2D([0.2148, 0.2148],[-0.03, 0.5], color='red')
#line2 = mlines.Line2D([0,0.8],[0.2148,0.2148])

#plt.xlabel('Match')
#plt.ylabel('Maximize')

plt.ylim(-.1, .5)
plt.xlim(-.1, .5)

plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

#add legend (NEEDS TO BE FIXED)
from numpy.random import randn

z = randn(100)

#add line of slope 1
plt.plot([0,1],[0,1], transform=plt.gca().transAxes)

#s_jsd_g2_plot.add_line(line2)
plt.savefig('confed_ego_jsd_2.jpeg',format='jpeg')
plt.show()

# Splitting the data into three blocks 

In [ ]:
'''This function splits dataframes contained in a dictionary into separate dataframes that each contain 30 trials.
The first ten trials of a session are removed.
input: dictionary containing dataframes split by player and game
output: dictionary with the dataframes with separate blocks added and dictionary with just blocked data'''
def create_blocks (dicti):
    
    all_blocks = {}

    length = len(dicti)

    i = 0
    
    while i < length:
        key = get_nth_key(dicti, n=i)

        df = pd.DataFrame(dicti[key]).reset_index(drop=True)

        #edit these numbers if you want 
        blocks = [df[0:10], df[10:40], df[40:70], df[70:100]]
        block_names = [key + '_b0', key + '_b1', key + '_b2', key + '_b3']
        blocks = dict(zip(block_names, blocks))


        i = i + 1
    
        all_blocks.update(blocks)
        
    #dicti.update(all_blocks)
        
    return dicti, all_blocks

In [ ]:
'''Function called in choice_dist_and_avg to get the key of a dictionary based on the index'''
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 

In [ ]:
'''This function gets the weight of a hole based on its probabiliy and the number of times it was chosen in the block'''
def get_weight(df, val, dec):
    
    try:
        weight = df.loc[df['probability'] == val, 'count_column'].iloc[0] *  dec
        
    #this will raise an error when that hole was chosen 0 times in the block
    except IndexError: 
        weight = 0
        
    return weight

In [ ]:
'''This function creates a dataframe that contains the average choice distribution and weighted average of the choice
distribution for all of the dataframes contained in the dictionary that is inputted.
input: dictionary containing dataframes separated by game and player number and separated into 4 blocks using the 
create blocks function. 
output: the dictionary with the count_column added to each dataframe (NOT currently);
a dataframe containing the weighted average of the choice distribution of each block'''
def choice_dist_and_avg (dicti):
    
    index = []
    avgs = []
    choices = []

    length = len(dicti)

    i = 0
    
    while i < length:
        key = get_nth_key(dicti, n=i)
        
        df = pd.DataFrame(dicti[key]).reset_index(drop=True)
        
        index.append(key)
        
        count = df['probability'].value_counts(sort=False).sort_index()
        
        #convert to dictionary
        count_dict = count.to_dict()
        
        #add count column to df
        df['count_column'] = df['probability'].map(count_dict)
        
        dicti[key] = df
        
        low = get_weight(df, 30.0, 0.30)
        med = get_weight(df, 60.0, 0.60)
        high = get_weight(df, 90.0, 0.90)
            
        avg = low + med + high
        
        avgs.append(avg)
        choices.append(count)
        
        i = i + 1
        
    avgs_df_melt = pd.DataFrame({'avg': avgs}, index = index)
    
    avgs_df = unmelt_df(avgs_df_melt)
    
    choices_df = pd.DataFrame(choices, index = index)
    choices_df = choices_df.fillna(0)
    
    return dicti, avgs_df_melt, avgs_df, choices_df

In [ ]:
'''This function unmelts the avgs dataframe so that there is 1 column row subject and 1 column per block.'''
#add column to counts indicating whether it's game 1 or 2
def unmelt_df (df):
    subjects = df.index.to_numpy()

    b0 = np.array([])
    b1 = np.array([])
    b2 = np.array([])
    b3 = np.array([])
    new_subjects = np.array([])

    ii = 1;
    for i in subjects:
        if [i][0][11] == '0':
            b0 = np.append(b0,df.loc[i])
        elif [i][0][11] == '1':
            b1 = np.append(b1,df.loc[i])
        elif [i][0][11] == '2':
            b2 = np.append(b2,df.loc[i])
        elif [i][0][11] == '3':
            b3 = np.append(b3,df.loc[i])
            
        #create new array for subjects with every 4th element from the subjects array
        if ii % 4 == 0:
            #remove last 3 characters from subject (ex. '_b0')
            new_subjects = np.append(new_subjects, i[:-3])
        ii = ii + 1;

    df_new = pd.DataFrame({'participant': new_subjects, 'b0': b0, 'b1': b1, 'b2': b2, 'b3': b3})

    return df_new

In [ ]:
#create a list of the dataframes
soc_g1 = [m01_p1_g1, m01_p2_g1, m03_p1_g1, m03_p2_g1, m05_p1_g1, m05_p2_g1, m08_p1_g1, m08_p2_g1, m10_p1_g1,
         m10_p2_g1, m11_p1_g1, m11_p2_g1, m13_p1_g1, m13_p2_g1, m14_p1_g1, m14_p2_g1, j01_p1_g1, j01_p2_g1, j02_p1_g1,
        j02_p2_g1,
         j03_p1_g1, j03_p2_g1, j04_p1_g1, j04_p2_g1, j05_p1_g1, j05_p2_g1, j06_p1_g1, j06_p2_g1, j07_p1_g1, j07_p2_g1, j08_p1_g1, j08_p2_g1,
         j10_p1_g1, j10_p2_g1, j11_p1_g1, j11_p2_g1]

#convert list to dictionary so dataframes can be accessed by their name
soc_g1 = {"{}".format(get_df_name(df)): dict(df) for i, df in enumerate(soc_g1, start=1)}

#call function to split df's into 3 blocks 
soc_g1, soc_g1_blocks = create_blocks(soc_g1)

#call function to calculate choice distributions and weighted avgs
soc_g1_blocks, soc_g1_blocks_avgs_melt, soc_g1_blocks_avgs, soc_g1_blocks_choices = choice_dist_and_avg(soc_g1_blocks)

In [ ]:
soc_g1_blocks_avgs.set_index('participant', inplace=True, drop=True)

In [ ]:
#repeat for soc_g2, confed_g1, and confed_g2
#create a list of the dataframes
soc_g2 = [m01_p1_g2, m01_p2_g2, m03_p1_g2, m03_p2_g2, m05_p1_g2, m05_p2_g2, m08_p1_g2, m08_p2_g2, m10_p1_g2,
         m10_p2_g2, m11_p1_g2, m11_p2_g2, m13_p1_g2, m13_p2_g2, m14_p1_g2, m14_p2_g2, j01_p1_g2, j01_p2_g2, j02_p1_g2,
        j02_p2_g2,
         j03_p1_g2, j03_p2_g2, j04_p1_g2, j04_p2_g2, j05_p1_g2, j05_p2_g2, j06_p1_g2, j06_p2_g2, j07_p1_g2, j07_p2_g2, j08_p1_g2, j08_p2_g2,
         j10_p1_g2, j10_p2_g2, j11_p1_g2, j11_p2_g2]

#convert list to dictionary so dataframes can be accessed by their name
soc_g2 = {"{}".format(get_df_name(df)): dict(df) for i, df in enumerate(soc_g2, start=1)}

#call function to split df's into 3 blocks 
soc_g2, soc_g2_blocks = create_blocks(soc_g2)

#call function to calculate choice distributions and weighted avgs
soc_g2_blocks, soc_g2_blocks_avgs_melt, soc_g2_blocks_avgs, soc_g2_blocks_choices = choice_dist_and_avg(soc_g2_blocks)

In [ ]:
soc_g2_blocks_avgs.set_index('participant', inplace=True, drop=True)
soc_g2_blocks_avgs_trans = soc_g2_blocks_avgs.transpose()

In [ ]:
#repeat for soc_g2, confed_g1, and confed_g2
#create a list of the dataframes
confed_g1 = [c01_p1_g1, c02_p1_g1, c03_p1_g1, c05_p1_g1, c06_p1_g1, c07_p1_g1, c09_p1_g1, c10_p1_g1, 
             c11_p1_g1, c12_p1_g1, c13_p1_g1, c14_p1_g1, c15_p1_g1, c16_p1_g1, c17_p1_g1, c18_p1_g1, s01_p1_g1, 
             s03_p1_g1, s05_p1_g1, s06_p1_g1, s07_p1_g1, s08_p1_g1, s09_p1_g1, s11_p1_g1, s12_p1_g1, s14_p1_g1, 
             s15_p1_g1, s16_p1_g1, s17_p1_g1, s19_p1_g1, s20_p1_g1]

#convert list to dictionary so dataframes can be accessed by their name
confed_g1 = {"{}".format(get_df_name(df)): dict(df) for i, df in enumerate(confed_g1, start=1)}

#call function to split df's into 3 blocks 
confed_g1, confed_g1_blocks = create_blocks(confed_g1)

#call function to calculate choice distributions and weighted avgs
confed_g1_blocks, confed_g1_blocks_avgs_melt, confed_g1_blocks_avgs, confed_g1_blocks_choices = choice_dist_and_avg(confed_g1_blocks)

In [ ]:
confed_g1_blocks_avgs.set_index('participant', inplace=True, drop=True)
confed_g1_blocks_avgs_trans = confed_g1_blocks_avgs.transpose()

In [ ]:
#repeat for soc_g2, confed_g1, and confed_g2
#create a list of the dataframes
confed_g2 = [c01_p1_g2, c02_p1_g2, c03_p1_g2, c05_p1_g2, c06_p1_g2, c07_p1_g2, c09_p1_g2, c10_p1_g2, 
             c11_p1_g2, c12_p1_g2, c13_p1_g2, c14_p1_g2, c15_p1_g2, c16_p1_g2, c17_p1_g2, c18_p1_g2, s01_p1_g2, 
             s03_p1_g2, s05_p1_g2, s06_p1_g2, s07_p1_g2, s08_p1_g2, s09_p1_g2, s11_p1_g2, s12_p1_g2, s14_p1_g2, 
             s15_p1_g2, s16_p1_g2, s17_p1_g2, s19_p1_g2, s20_p1_g2]

#convert list to dictionary so dataframes can be accessed by their name
confed_g2 = {"{}".format(get_df_name(df)): dict(df) for i, df in enumerate(confed_g2, start=1)}

#call function to split df's into 3 blocks 
confed_g2, confed_g2_blocks = create_blocks(confed_g2)

#call function to calculate choice distributions and weighted avgs
confed_g2_blocks, confed_g2_blocks_avgs_melt, confed_g2_blocks_avgs, confed_g2_blocks_choices = choice_dist_and_avg(confed_g2_blocks)

In [ ]:
confed_g2_blocks_avgs.set_index('participant', inplace=True, drop=True)
confed_g2_blocks_avgs_trans = confed_g2_blocks_avgs.transpose()

In [ ]:
def transpose(df):
    df_trans = df.transpose()
    return df_trans

In [ ]:
soc_g1_blocks_avgs_winsor = soc_g1_blocks_avgs.apply(winsorize_column,axis=0)
soc_g2_blocks_avgs_winsor = soc_g2_blocks_avgs.apply(winsorize_column,axis=0)
confed_g1_blocks_avgs_winsor = confed_g1_blocks_avgs.apply(winsorize_column,axis=0)
confed_g2_blocks_avgs_winsor = confed_g2_blocks_avgs.apply(winsorize_column,axis=0)

In [ ]:
confed_g1_blocks_avgs_winsor

In [ ]:
soc_g1_blocks_avgs_winsor_trans = transpose(soc_g1_blocks_avgs_winsor)
soc_g2_blocks_avgs_winsor_trans = transpose(soc_g2_blocks_avgs_winsor)
confed_g1_blocks_avgs_winsor_trans = transpose(confed_g1_blocks_avgs_winsor)
confed_g2_blocks_avgs_winsor_trans = transpose(confed_g2_blocks_avgs_winsor)

In [ ]:
sp.stats.ttest_ind(soc_g1_blocks_avgs_winsor,confed_g1_blocks_avgs_winsor)

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_avgs_winsor['b1'],confed_g1_blocks_avgs_winsor['b2'])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_avgs_winsor['b2'],confed_g1_blocks_avgs_winsor['b3'])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_avgs_winsor['b1'],confed_g1_blocks_avgs_winsor['b3'])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_avgs_winsor['b1'],soc_g1_blocks_avgs_winsor['b2'])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_avgs_winsor['b2'],soc_g1_blocks_avgs_winsor['b3'])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_avgs_winsor['b1'],soc_g1_blocks_avgs_winsor['b3'])

In [ ]:
#outlier 1.5 * IQR
sns.boxplot(data=soc_g1_blocks_avgs)

In [ ]:
sns.boxplot(data=soc_g1_blocks_avgs_winsor)

In [ ]:
sns.boxplot(data=soc_g2_blocks_avgs)

In [ ]:
sns.boxplot(data=confed_g1_blocks_avgs)

In [ ]:
sns.boxplot(data=confed_g2_blocks_avgs)

In [ ]:
soc_g1_blocks_avgs_trans = soc_g1_blocks_avgs.transpose()

In [ ]:
sns.lineplot(data=soc_g1_blocks_avgs_trans, dashes=False)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
#mean with 95% confidence interval
#try fig, ax
soc_g1_blocks_avgs = soc_g1_blocks_avgs.melt(ignore_index = False)
sns.lineplot(data=soc_g1_blocks_avgs, x='variable',y='value')

In [ ]:
sns.lineplot(data=soc_g2_blocks_avgs_trans, dashes=False)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
soc_g2_blocks_avgs = soc_g2_blocks_avgs.melt(ignore_index = False)
sns.lineplot(data=soc_g2_blocks_avgs,x='variable',y='value')

In [ ]:
sns.lineplot(data=confed_g1_blocks_avgs_trans, dashes=False, legend = False)

In [ ]:
confed_g1_blocks_avgs = confed_g1_blocks_avgs.melt(ignore_index = False)
sns.lineplot(data=confed_g1_blocks_avgs,x='variable',y='value')

In [ ]:
confed_g1_blocks_avgs_winsor = confed_g1_blocks_avgs_winsor.melt(ignore_index = False)
sns.lineplot(data=confed_g1_blocks_avgs_winsor,x='variable',y='value')

In [ ]:
soc_g1_blocks_avgs_winsor = soc_g1_blocks_avgs_winsor.melt(ignore_index = False)
sns.lineplot(data=soc_g1_blocks_avgs_winsor,x='variable',y='value')

In [ ]:
soc_g2_blocks_avgs_winsor = soc_g2_blocks_avgs_winsor.melt(ignore_index = False)
sns.lineplot(data=soc_g2_blocks_avgs_winsor,x='variable',y='value')

In [ ]:
sns.lineplot(data=confed_g2_blocks_avgs_trans, dashes=False, legend = False)

In [ ]:
confed_g2_blocks_avgs = confed_g2_blocks_avgs.melt(ignore_index = False)
sns.lineplot(data=confed_g2_blocks_avgs,x='variable',y='value')

In [ ]:
confed_g2_blocks_avgs_winsor = confed_g2_blocks_avgs_winsor.melt(ignore_index = False)
sns.lineplot(data=confed_g2_blocks_avgs_winsor,x='variable',y='value')

In [ ]:
#greater difference b/w g1 and g2 in social condition than control condition

In [ ]:
#BUT interaction is not significant in ANOVA

In [ ]:
confed_avgs = confed_g1_blocks_avgs.append(confed_g2_blocks_avgs)
soc_avgs = soc_g1_blocks_avgs.append(soc_g2_blocks_avgs)
avgs = confed_avgs.append(soc_avgs)

In [ ]:
confed_avgs_winsor = confed_g1_blocks_avgs_winsor.append(confed_g2_blocks_avgs_winsor)
soc_avgs_winsor = soc_g1_blocks_avgs_winsor.append(soc_g2_blocks_avgs_winsor)
avgs_winsor = confed_avgs_winsor.append(soc_avgs_winsor)

In [ ]:
sns.lineplot(data=confed_g1_blocks_avgs_winsor, x='variable', y='value', label='Confederate Game 1')
sns.lineplot(data=soc_g1_blocks_avgs_winsor, x='variable', y='value', label='Social Game 1')

In [ ]:
avgs_winsor_g1 = confed_g1_blocks_avgs_winsor.append(soc_g1_blocks_avgs_winsor)
avgs_winsor_g1_nob0 = avgs_winsor_g1[avgs_winsor_g1.variable != 'b0']

In [ ]:
avgs_winsor_g1_nob0 = anova_label(avgs_winsor_g1_nob0)

In [ ]:
avgs_winsor_g1_nob0

In [ ]:
pg.mixed_anova(data=avgs_winsor_g1_nob0, within='variable',between='condition',dv='value',subject='participant')

In [ ]:
avgs = anova_label(avgs)

In [ ]:
avgs_winsor = anova_label(avgs_winsor)

In [ ]:
avgs['group'] = 'game ' + avgs['game'] + ' ' + avgs['condition']
avgs

In [ ]:
avgs_winsor['group'] = 'game ' + avgs_winsor['game'] + ' ' + avgs_winsor['condition']
avgs_winsor

In [ ]:
avgs

In [ ]:
avgs_trans = avgs.transpose()

In [ ]:
avgs_trans

In [ ]:
avgs

In [ ]:
sns.lineplot(data=avgs,x='variable',y='value', hue = 'group')

In [ ]:
avgs_winsor

In [ ]:
sns.lineplot(data=avgs_winsor,x='variable',y='value', hue = 'group')

In [ ]:
sns.lineplot(data=avgs[avgs.variable != 'b0'],x='variable',y='value', hue = 'group', estimator = np.median)
plt.ylim(16, 27)

In [ ]:
#break up df by block
avgs_b1 = avgs.loc[avgs.variable=='b1']
avgs_b1 = avgs_b1.reset_index()

#break up df by block
avgs_b2 = avgs.loc[avgs.variable=='b2']
avgs_b2 = avgs_b2.reset_index()

#break up df by block
avgs_b3 = avgs.loc[avgs.variable=='b3']
avgs_b3 = avgs_b3.reset_index()

In [ ]:
pg.mixed_anova(data=avgs_b1, dv='value', between='condition', within='game', subject='participant')

In [ ]:
pg.mixed_anova(data=avgs_b2, dv='value', between='condition', within='game', subject='participant')

In [ ]:
pg.mixed_anova(data=avgs_b3, dv='value', between='condition', within='game', subject='participant')

In [ ]:
#significant interaction in b2
#may be reduced bc there is more information on hole 60

In [ ]:
confed_g1_blocks_avgs.mean()

In [ ]:
soc_g1_blocks_avgs.mean()

In [ ]:
confed_g2_blocks_avgs.mean()

In [ ]:
soc_g2_blocks_avgs.mean()

In [ ]:
#learn to ignore participant's choices in g2

In [ ]:
#check that subjects are above chance ?

In [ ]:
soc_g2_blocks_avgs_trans

In [ ]:
sns.lineplot(data=soc_g1_blocks_avgs_trans.drop('b0'), dashes=False, legend = False)
plt.ylim(9, 30)

In [ ]:
sns.lineplot(data=soc_g2_blocks_avgs_trans.drop('b0'), dashes=False, legend = False)
plt.ylim(9, 30)

In [ ]:
avgs_b3

In [ ]:
avgs_no_b0 = avgs[avgs.variable != 'b0']

In [ ]:
avgs_no_b0_g1 = avgs_no_b0[avgs_no_b0.game == '1']

In [ ]:
avgs_no_b0_g1

In [ ]:
#3x2 mixed ANOVA with block (exluding b0) as within measure and condition as between measure
pg.mixed_anova(data=avgs_no_b0_g1, dv='value', within='variable', between='condition', subject = 'participant')

# Choice Behavior Across the Blocks

In [ ]:
#1/3 to DO: 
#get weighted avg for entire session and compare to overall score
soc_g1_blocks_avgs

In [ ]:
confed_g1_blocks

In [ ]:
#create dictionary of choices dfs
blocks_choices = {'confed_g1': confed_g1_blocks_choices, 'confed_g2': confed_g2_blocks_choices, 
                  'soc_g1': soc_g1_blocks_choices, 'soc_g2': soc_g2_blocks_choices}

In [ ]:
confed_g2_blocks_choices

In [ ]:
def separate_blocks_choices(df):
    subjects = df.index.to_numpy()

    block = np.array([])

    new_subjects = np.array([])

    for i in subjects:
        if [i][0][11] == '0':
            block = np.append(block, '0')
        elif [i][0][11] == '1':
            block = np.append(block, '1')
        elif [i][0][11] == '2':
            block = np.append(block, '2')
        elif [i][0][11] == '3':
            block = np.append(block, '3')

        new_subjects = np.append(new_subjects, i[:-3])

    df['block'] = block
    df['participant'] = new_subjects
    df.set_index('participant')
    
    #separate by block
    grouped = df.groupby(df.block)
    df_b0_choices = grouped.get_group('0')
    df_b1_choices = grouped.get_group('1')
    df_b2_choices = grouped.get_group('2')
    df_b3_choices = grouped.get_group('3')
    
    
    return df_b0_choices, df_b1_choices, df_b2_choices, df_b3_choices

In [ ]:
confed_g1_b0_choices, confed_g1_b1_choices, confed_g1_b2_choices, confed_g1_b3_choices = separate_blocks_choices(confed_g1_blocks_choices)
confed_g2_b0_choices, confed_g2_b1_choices, confed_g2_b2_choices, confed_g2_b3_choices = separate_blocks_choices(confed_g2_blocks_choices)
soc_g1_b0_choices, soc_g1_b1_choices, soc_g1_b2_choices, soc_g1_b3_choices = separate_blocks_choices(soc_g1_blocks_choices)
soc_g2_b0_choices, soc_g2_b1_choices, soc_g2_b2_choices, soc_g2_b3_choices = separate_blocks_choices(soc_g2_blocks_choices)

In [ ]:
confed_g1_b2_choices

In [ ]:
sns.boxplot(data=confed_g1_b2_choices.drop(['block','participant'],axis=1))

In [ ]:
sns.boxplot(data=soc_g1_b2_choices.drop(['block','participant'],axis=1))

In [ ]:
confed_b2_choices = confed_g1_b2_choices.append(confed_g2_b2_choices)
soc_b2_choices = soc_g1_b2_choices.append(soc_g2_b2_choices)
b2_choices = confed_b2_choices.append(soc_b2_choices)

In [ ]:
b2_choices = anova_label(b2_choices)

In [ ]:
sp.stats.ttest_ind(confed_g1_b2_choices[30.0], soc_g1_b2_choices[30.0])

In [ ]:
sp.stats.ttest_ind(confed_g1_b2_choices[90.0], soc_g1_b2_choices[90.0])

In [ ]:
sp.stats.ttest_ind(confed_g1_b3_choices[30.0], soc_g1_b3_choices[30.0])

In [ ]:
sp.stats.ttest_ind(confed_g1_b3_choices[60.0], soc_g1_b3_choices[60.0])

In [ ]:
sp.stats.ttest_ind(confed_g1_b3_choices[90.0], soc_g1_b3_choices[90.0])

In [ ]:
soc_g1_b2_choices[30.0].mean(), confed_g1_b2_choices[30.0].mean()

In [ ]:
soc_g1_b2_choices[30.0].std(), confed_g1_b2_choices[30.0].std()

In [ ]:
sp.stats.ttest_ind(confed_g2_b1_choices[30.0], soc_g2_b1_choices[30.0])

In [ ]:
def jsd_for_blocks(dicti):
    
    new_dicti = {}
    
    length = len(dicti)
    print(length)
    
    i = 0
    
    while i < length:
        key = get_nth_key(dicti, n=i)
        
        df = pd.DataFrame(dicti[key])
        
        df = df.set_index('participant',drop=True)

        df = df.drop(columns = ['block'])

        subjects = df.index.to_numpy()

        maxis = np.array([])
        matchi= np.array([])
        randi = np.array([])
        soci = np.array([])

        for subject in subjects:
            maxi, match, rand, soc = call_jsd(df.loc[subject])
            maxis = np.append(maxis, maxi)
            matchi = np.append(matchi, match)
            randi = np.append(randi, rand)
            soci = np.append(soci, soc)

        df['maxi'] = maxis
        df['matchi'] = matchi
        df['randi'] = randi
        df['soci'] = soci
        
        #set lowest jsd value as strategy
        df['strategy'] = df.drop([30.0,60.0,90.0],axis=1).idxmin(axis=1)
        
        new_dicti[key] = df
        
        i = i + 1
    
    return new_dicti

In [ ]:
#create dictionary of dataframes so that functions can be called on them more efficiently
block_choices = [confed_g1_b0_choices, confed_g1_b1_choices, confed_g1_b2_choices, confed_g1_b3_choices,
                confed_g2_b0_choices, confed_g2_b1_choices, confed_g2_b2_choices, confed_g2_b3_choices,
                soc_g1_b0_choices, soc_g1_b1_choices, soc_g1_b2_choices, soc_g1_b3_choices,
                soc_g2_b0_choices, soc_g2_b1_choices, soc_g2_b2_choices, soc_g2_b3_choices]
#convert list to dictionary so dataframes can be accessed by their name
block_choices = {"{}".format(get_df_name(df)): dict(df) for i, df in enumerate(block_choices, start=1)}

In [ ]:
block_choices = jsd_for_blocks(block_choices)

In [ ]:
block_choices['confed_g1_b1_choices'].mean()

In [ ]:
block_choices['confed_g1_b2_choices'].mean()

In [ ]:
block_choices['confed_g1_b3_choices'].mean()

In [ ]:
block_choices['confed_g2_b2_choices'].mean()

In [ ]:
block_choices['soc_g1_b2_choices'].mean()

In [ ]:
block_choices['soc_g2_b2_choices'].mean()

In [ ]:
#linear model to predict weighted avg from block
#linear hypothesis test

In [ ]:
#idea: look at how ppl reported the reward probs in the holes they chose and their overall reward count as measure of engagement (compare conditions)

In [ ]:
#one subject said they chose the hole their partner did NOT choose --> soc condition has 2x info on holes 

# maximizing vs matching across blocks

In [ ]:
block_choices['soc_g1_b2_choices']

In [ ]:
block_choices['soc_g1_b3_choices']

# disagregate game 2 by same and different and compare block 0

In [ ]:
confed_g2_blocks_avgs

In [ ]:
confed_g2_blocks_avgs['struct'] = ['s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s','s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s','s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s','s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s']

In [ ]:
confed_g2_blocks_avgs

In [ ]:
confed_g2s_blocks_avgs = confed_g2_blocks_avgs[confed_g2_blocks_avgs['struct']=='s']
confed_g2d_blocks_avgs = confed_g2_blocks_avgs[confed_g2_blocks_avgs['struct']=='d']

In [ ]:
sp.stats.ttest_ind(confed_g2s_blocks_avgs['value'],confed_g2d_blocks_avgs['value'])

In [ ]:
confed_g2s_blocks_avgs_b0 = confed_g2s_blocks_avgs[confed_g2s_blocks_avgs['variable']=='b0']
confed_g2d_blocks_avgs_b0 = confed_g2d_blocks_avgs[confed_g2d_blocks_avgs['variable']=='b0']

In [ ]:
sp.stats.ttest_ind(confed_g2s_blocks_avgs_b0['value'],confed_g2d_blocks_avgs_b0['value'])

In [ ]:
confed_g2s_blocks_avgs_b0['value'].mean()

In [ ]:
confed_g2d_blocks_avgs_b0['value'].mean()

In [ ]:
soc_g2_blocks_avgs

In [ ]:
soc_g2_blocks_avgs['struct'] = ['d','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',
                               'd','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',
                               'd','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',
                               'd','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',]

In [ ]:
soc_g2_blocks_avgs

In [ ]:
soc_g2s_blocks_avgs = soc_g2_blocks_avgs[soc_g2_blocks_avgs['struct']=='s']
soc_g2d_blocks_avgs = soc_g2_blocks_avgs[soc_g2_blocks_avgs['struct']=='d']

In [ ]:
soc_g2s_blocks_avgs_b0 = soc_g2s_blocks_avgs[soc_g2s_blocks_avgs['variable']=='b0']
soc_g2d_blocks_avgs_b0 = soc_g2d_blocks_avgs[soc_g2d_blocks_avgs['variable']=='b0']

In [ ]:
sp.stats.ttest_ind(soc_g2s_blocks_avgs_b0['value'],soc_g2d_blocks_avgs_b0['value'])

In [ ]:
soc_g2s_blocks_avgs_b0['value'].mean()

In [ ]:
soc_g2d_blocks_avgs_b0['value'].mean()

In [ ]:
soc_g2d_blocks_avgs_b0

In [ ]:
sns.boxplot(data=soc_g2s_blocks_avgs_b0['value'])

In [ ]:
sns.boxplot(data=soc_g2d_blocks_avgs_b0['value'])

In [ ]:
sns.boxplot(data=confed_g2s_blocks_avgs_b0['value'])

In [ ]:
sns.boxplot(data=confed_g2d_blocks_avgs_b0['value'])

In [ ]:
confed_g2d_blocks_avgs

In [ ]:
confed_g2s_blocks_avgs_b1 = confed_g2s_blocks_avgs[confed_g2s_blocks_avgs['variable']=='b1']
confed_g2d_blocks_avgs_b1 = confed_g2d_blocks_avgs[confed_g2d_blocks_avgs['variable']=='b1']
confed_g2s_blocks_avgs_b2 = confed_g2s_blocks_avgs[confed_g2s_blocks_avgs['variable']=='b2']
confed_g2d_blocks_avgs_b2 = confed_g2d_blocks_avgs[confed_g2d_blocks_avgs['variable']=='b2']
confed_g2s_blocks_avgs_b3 = confed_g2s_blocks_avgs[confed_g2s_blocks_avgs['variable']=='b3']
confed_g2d_blocks_avgs_b3 = confed_g2d_blocks_avgs[confed_g2d_blocks_avgs['variable']=='b3']

In [ ]:
soc_g2s_blocks_avgs_b1 = soc_g2s_blocks_avgs[soc_g2s_blocks_avgs['variable']=='b1']
soc_g2d_blocks_avgs_b1 = soc_g2d_blocks_avgs[soc_g2d_blocks_avgs['variable']=='b1']
soc_g2s_blocks_avgs_b2 = soc_g2s_blocks_avgs[soc_g2s_blocks_avgs['variable']=='b2']
soc_g2d_blocks_avgs_b2 = soc_g2d_blocks_avgs[soc_g2d_blocks_avgs['variable']=='b2']
soc_g2s_blocks_avgs_b3 = soc_g2s_blocks_avgs[soc_g2s_blocks_avgs['variable']=='b3']
soc_g2d_blocks_avgs_b3 = soc_g2d_blocks_avgs[soc_g2d_blocks_avgs['variable']=='b3']

In [ ]:
sp.stats.ttest_ind(soc_g2d_blocks_avgs_b1['value'],soc_g2s_blocks_avgs_b1['value'])

In [ ]:
sp.stats.ttest_ind(soc_g2d_blocks_avgs_b2['value'],soc_g2s_blocks_avgs_b2['value'])

In [ ]:
sp.stats.ttest_ind(soc_g2d_blocks_avgs_b3['value'],soc_g2s_blocks_avgs_b3['value'])

In [ ]:
sp.stats.ttest_ind(confed_g2d_blocks_avgs_b1['value'],confed_g2s_blocks_avgs_b1['value'])

In [ ]:
sp.stats.ttest_ind(confed_g2d_blocks_avgs_b2['value'],confed_g2s_blocks_avgs_b2['value'])

In [ ]:
sp.stats.ttest_ind(confed_g2d_blocks_avgs_b3['value'],confed_g2s_blocks_avgs_b3['value'])

# count number of 30 and 90 choices per block

In [ ]:
soc_g1_blocks_choices.set_index('participant', inplace=True, drop=True)
soc_g2_blocks_choices.set_index('participant', inplace=True, drop=True)
confed_g1_blocks_choices.set_index('participant', inplace=True, drop=True)
confed_g2_blocks_choices.set_index('participant', inplace=True, drop=True)

In [ ]:
soc_g1_blocks_90choices = soc_g1_blocks_choices.drop([30.0,60.0],axis=1)
soc_g2_blocks_90choices = soc_g2_blocks_choices.drop([30.0,60.0],axis=1)
confed_g1_blocks_90choices = confed_g1_blocks_choices.drop([30.0,60.0],axis=1)
confed_g2_blocks_90choices = confed_g2_blocks_choices.drop([30.0,60.0],axis=1)

In [ ]:
soc_g1_blocks_30choices = soc_g1_blocks_choices.drop([90.0,60.0],axis=1)
soc_g2_blocks_30choices = soc_g2_blocks_choices.drop([90.0,60.0],axis=1)
confed_g1_blocks_30choices = confed_g1_blocks_choices.drop([90.0,60.0],axis=1)
confed_g2_blocks_30choices = confed_g2_blocks_choices.drop([90.0,60.0],axis=1)

In [ ]:
soc_g1_blocks_90choices_pivot = soc_g1_blocks_90choices.pivot(columns='block',values=90.0)
soc_g2_blocks_90choices_pivot = soc_g2_blocks_90choices.pivot(columns='block',values=90.0)
confed_g1_blocks_90choices_pivot = confed_g1_blocks_90choices.pivot(columns='block',values=90.0)
confed_g2_blocks_90choices_pivot = confed_g2_blocks_90choices.pivot(columns='block',values=90.0)

In [ ]:
soc_g1_blocks_30choices_pivot = soc_g1_blocks_30choices.pivot(columns='block',values=30.0)
soc_g2_blocks_30choices_pivot = soc_g2_blocks_30choices.pivot(columns='block',values=30.0)
confed_g1_blocks_30choices_pivot = confed_g1_blocks_30choices.pivot(columns='block',values=30.0)
confed_g2_blocks_30choices_pivot = confed_g2_blocks_30choices.pivot(columns='block',values=30.0)

In [ ]:
sns.boxplot(data=confed_g1_blocks_30choices_pivot)

In [ ]:
sns.boxplot(data=soc_g1_blocks_30choices_pivot)

In [ ]:
confed_g1_blocks_30choices_pivot.describe()

In [ ]:
soc_g1_blocks_30choices_pivot.describe()

In [ ]:
sns.boxplot(data=soc_g1_blocks_90choices_pivot)

In [ ]:
sns.boxplot(data=confed_g1_blocks_90choices_pivot)

In [ ]:
sns.boxplot(data=soc_g2_blocks_90choices_pivot)

In [ ]:
sns.boxplot(data=confed_g2_blocks_90choices_pivot)

In [ ]:
sp.stats.ttest_ind(soc_g1_blocks_90choices_pivot,confed_g1_blocks_90choices_pivot)

In [ ]:
#HERE TODAY
sp.stats.ttest_ind(soc_g1_blocks_30choices_pivot,confed_g1_blocks_30choices_pivot)

In [ ]:
#HERE TODAY
sp.stats.ttest_ind(soc_g2_blocks_30choices_pivot,confed_g2_blocks_30choices_pivot)

In [ ]:
soc_g1_blocks_90choices_pivot_winsor = soc_g1_blocks_90choices_pivot.apply(winsorize_column,axis=0)
soc_g2_blocks_90choices_pivot_winsor = soc_g2_blocks_90choices_pivot.apply(winsorize_column,axis=0)
confed_g1_blocks_90choices_pivot_winsor = confed_g1_blocks_90choices_pivot.apply(winsorize_column,axis=0)
confed_g2_blocks_90choices_pivot_winsor = confed_g2_blocks_90choices_pivot.apply(winsorize_column,axis=0)

In [ ]:
sns.boxplot(data=soc_g1_blocks_90choices_pivot_winsor)

In [ ]:
sns.boxplot(data=confed_g1_blocks_90choices_pivot_winsor)

In [ ]:
sp.stats.ttest_ind(confed_g1_blocks_90choices_pivot_winsor,soc_g1_blocks_90choices_pivot_winsor)

In [ ]:
sp.stats.ttest_ind(soc_g2_blocks_90choices_pivot_winsor,confed_g2_blocks_90choices_pivot_winsor)

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_90choices_pivot_winsor['1'],soc_g1_blocks_90choices_pivot_winsor['2'])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_90choices_pivot_winsor['2'],soc_g1_blocks_90choices_pivot_winsor['3'])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_90choices_pivot_winsor['1'],soc_g1_blocks_90choices_pivot_winsor['3'])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_90choices_pivot_winsor['1'],confed_g1_blocks_90choices_pivot_winsor['2'])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_90choices_pivot_winsor['2'],confed_g1_blocks_90choices_pivot_winsor['3'])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_90choices_pivot_winsor['1'],confed_g1_blocks_90choices_pivot_winsor['3'])

In [ ]:
confed_g1_blocks_90choices_pivot_winsor.describe()

In [ ]:
soc_g1_blocks_90choices_pivot_winsor.describe()

In [ ]:
confed_g1_blocks_90choices_pivot.describe()

In [ ]:
soc_g1_blocks_90choices_pivot.describe()

In [ ]:
sns.boxplot(data=confed_g1_blocks_90choices_pivot)

In [ ]:
sns.boxplot(data=confed_g1_blocks_90choices_pivot_winsor)

In [ ]:
sns.boxplot(data=soc_g1_blocks_90choices_pivot_winsor)

In [ ]:
sns.boxplot(data=soc_g1_blocks_90choices_pivot)

In [ ]:
confed_blocks_90choices_winsor = confed_g1_blocks_90choices_pivot_winsor.append(confed_g2_blocks_90choices_pivot_winsor)
soc_blocks_90choices_winsor = soc_g1_blocks_90choices_pivot_winsor.append(soc_g2_blocks_90choices_pivot_winsor)
blocks_90choices_winsor = confed_blocks_90choices_winsor.append(soc_blocks_90choices_winsor)

In [ ]:
blocks_90choices_winsor = blocks_90choices_winsor.melt(ignore_index=False)

In [ ]:
blocks_90choices_winsor = anova_label(blocks_90choices_winsor)

In [ ]:
blocks_90choices_winsor['group'] = 'game ' + blocks_90choices_winsor['game'] + ' ' + blocks_90choices_winsor['condition']
blocks_90choices_winsor_nob0 = blocks_90choices_winsor[blocks_90choices_winsor.block != '0']

In [ ]:
sns.lineplot(data=blocks_90choices_winsor,x='block',y='value', hue = 'group')

In [ ]:
#HERE TODAY
sns.lineplot(data=soc_g1_blocks_30choices,x='block',y=30.0, label= 'Social Game 1')
sns.lineplot(data=confed_g1_blocks_30choices,x='block',y=30.0, label= 'Confederate Game 1')
#sns.lineplot(data=soc_g2_blocks_30choices,x='block',y=30.0, label= 'Social Game 2')
#sns.lineplot(data=confed_g2_blocks_30choices,x='block',y=30.0, label= 'Confederate Game 2')

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '1'][30.0],confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '3'][30.0])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '1'][30.0],confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '2'][30.0])

In [ ]:
sp.stats.ttest_rel(confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '2'][30.0],confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '3'][30.0])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '1'][30.0],soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '2'][30.0])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '2'][30.0],soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '3'][30.0])

In [ ]:
sp.stats.ttest_rel(soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '1'][30.0],soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '3'][30.0])

In [ ]:
sp.stats.ttest_ind(confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '2'][30.0],soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '2'][30.0])

In [ ]:
sp.stats.ttest_ind(confed_g1_blocks_30choices[confed_g1_blocks_30choices.block == '1'][30.0],soc_g1_blocks_30choices[soc_g1_blocks_30choices.block == '1'][30.0])

In [ ]:
confed_blocks_30choices = confed_g1_blocks_30choices.append(confed_g2_blocks_30choices)
soc_blocks_30choices = soc_g1_blocks_30choices.append(soc_g2_blocks_30choices)
blocks_30choices = confed_blocks_30choices.append(soc_blocks_30choices)

In [ ]:
confed_g1_blocks_30choices

In [ ]:
blocks_30choices

In [ ]:
blocks_30choices = anova_label(blocks_30choices)

In [ ]:
blocks_30choices_nob0 = blocks_30choices[blocks_30choices.block != '0']

In [ ]:
blocks_30choices_nob0_g1 = blocks_30choices_nob0[blocks_30choices_nob0.game == '1']

In [ ]:
blocks_30choices_nob0_g1.reset_index(drop=True)

In [ ]:
#3x2 mixed ANOVA with block (exluding b0) as within measure and condition as between measure
pg.mixed_anova(data=blocks_30choices_nob0_g1, dv=30.0, within='block', between='condition', subject = 'participant')

In [ ]:
pg.pairwise_ttests(data=blocks_30choices_nob0_g1, dv=30.0, within='block', between='condition', subject = 'participant')

In [ ]:
blocks_90choices_winsor_nob0 = blocks_90choices_winsor[blocks_90choices_winsor.block != '0']
blocks_90choices_winsor_nob0_g1 = blocks_90choices_winsor_nob0[blocks_90choices_winsor_nob0.game == '1']
blocks_90choices_winsor_nob0_g1

In [ ]:
blocks_90choices_winsor_nob0_g1.shape

In [ ]:
#3x2 mixed ANOVA with block (exluding b0) as within measure and condition as between measure
pg.mixed_anova(data=blocks_90choices_winsor_nob0_g1, dv='value', within='block', between='condition', subject = 'participant')

In [ ]:
pg.pairwise_ttests(dv='value', within='block', between='condition', subject='participant', data=blocks_90choices_winsor_nob0_g1)

# plotting 90 choices across game

In [ ]:
#loop through trials
#if probability == 90.0, add to 90_count

In [ ]:
'''This function creates a dataframe that contains the average choice distribution and weighted average of the choice
distribution for all of the dataframes contained in the dictionary that is inputted.
input: dictionary containing dataframes separated by game and player number and separated into 4 blocks using the 
create blocks function. 
output: the dictionary with the count_column added to each dataframe (NOT currently);
a dataframe containing the weighted average of the choice distribution of each block'''
def count_by_trial (dicti):
    
    length = len(dicti)

    i = 0
    
    while i < length:
        
        high_count = []
        med_count = []
        low_count = []
    
        key = get_nth_key(dicti, n=i)
        
        df = pd.DataFrame(dicti[key]).reset_index(drop=True)
        
        for index, row in df.iterrows():
            counts = df[:index]['probability'].value_counts(sort=False).reindex(df.probability.unique(),fill_value=0).sort_index()
            
            low = get_count(counts,30.0)
            med = get_count(counts,60.0)
            high = get_count(counts,90.0)
            
            low_count.append(low)
            med_count.append(med)
            high_count.append(high)
        
        df['low_count'] = low_count
        df['med_count'] = med_count
        df['high_count'] = high_count
        
        dicti[key] = df

        i = i + 1
    
    return dicti

In [ ]:
'''Function to deal with KeyError if participant chose one hole 0 times.'''
def get_count(counts, prob):
    try:
        return counts[prob]
    #if the hole was never chosen and the denominator is 0
    except KeyError:
        return 0

In [ ]:
confed_g1 = count_by_trial(confed_g1)
confed_g2 = count_by_trial(confed_g2)
soc_g1 = count_by_trial(soc_g1)
soc_g2 = count_by_trial(soc_g2)

In [ ]:
s16_p1_g1 = confed_g1['s16_p1_g1']

In [ ]:
#sns.lineplot(data=s17_p1_g1,x=s17_p1_g1.index,y=s17_p1_g1['high_count'])
#sns.lineplot(data=s16_p1_g1,x=s16_p1_g1.index,y=s16_p1_g1['high_count'])

In [ ]:
def plot_count(dicti):
    length = len(dicti)

    i = 0

    while i < length:
        key = get_nth_key(dicti, n=i)
        plot = sns.lineplot(data=dicti[key],x=dicti[key].index,y=dicti[key]['high_count'])
        i = i + 1
        
    return plot

In [ ]:
plot_count(confed_g1)

In [ ]:
plot_count(soc_g1)

In [ ]:
plot_count(confed_g2)

In [ ]:
pd.merge(confed_g1['c01_p1_g1'][['low_count','med_count','high_count']],confed_g1['c02_p1_g1'][['low_count','med_count','high_count']])

In [ ]:
suffixes = ['_c01_p1_g1','_c02_p1_g1','_c03_p1_g1']
dfs_array = [confed_g1['c01_p1_g1']['low_count'],confed_g1['c02_p1_g1']['low_count'],confed_g1['c03_p1_g1']['low_count']]

In [ ]:
c01_p1_g1_high = pd.DataFrame(confed_g1['c01_p1_g1']['high_count'])

In [ ]:
c02_p1_g1_high = pd.DataFrame(confed_g1['c02_p1_g1']['high_count'])

In [ ]:
c01_p1_g1_high

In [ ]:
def combine_counts(dicti,hole):
    length = len(dicti)

    i = 0
    dfs_array = []
    suffixes = []

    while i < length:
        key = get_nth_key(dicti, n=i)
        #can be changed to do for low and medium too
        dfs_array.append(dicti[key][hole])
        suffixes.append(key)
        i = i + 1
    
    #turn into function and do to entire dictionary
    #combo_count = reduce(lambda left,right: pd.merge(left,right), dfs_array)
    #combo_count = reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True,suffixes=suffixes), dfs_array)
    #pd.merge(dfs_array,left_index=True,right_index=True,suffixes=(suffixes))
    combo_count = pd.concat(dfs_array, join='outer', axis=1)
    combo_count.columns = suffixes
    
    return combo_count

In [ ]:
from functools import reduce

In [ ]:
confed_g1_high = combine_counts(confed_g1,'high_count')
soc_g1_high = combine_counts(soc_g1,'high_count')
confed_g2_high = combine_counts(confed_g2,'high_count')
soc_g2_high = combine_counts(soc_g2,'high_count')
confed_g1_high

In [ ]:
confed_g1_low = combine_counts(confed_g1,'low_count')
soc_g1_low = combine_counts(soc_g1,'low_count')
confed_g2_low = combine_counts(confed_g2,'low_count')
soc_g2_low = combine_counts(soc_g2,'low_count')
confed_g1_low

In [ ]:
confed_g1_med = combine_counts(confed_g1,'med_count')
soc_g1_med = combine_counts(soc_g1,'med_count')
confed_g2_med = combine_counts(confed_g2,'med_count')
soc_g2_med = combine_counts(soc_g2,'med_count')
confed_g1_med

In [ ]:
confed_g1_high.drop(confed_g1_high.tail(1).index,inplace=True)
confed_g2_high.drop(confed_g2_high.tail(1).index,inplace=True)
confed_g1_low.drop(confed_g1_low.tail(1).index,inplace=True)
confed_g2_low.drop(confed_g2_low.tail(1).index,inplace=True)
confed_g1_med.drop(confed_g1_med.tail(1).index,inplace=True)
confed_g2_med.drop(confed_g2_med.tail(1).index,inplace=True)


In [ ]:
confed_g1_high.transpose().describe()

In [ ]:
confed_g1_high.transpose().describe()

In [ ]:
confed_g1_low.transpose().describe()

In [ ]:
soc_g1_low.transpose().describe()

In [ ]:
confed_g1_med.transpose().describe()

In [ ]:
confed_g1_high = confed_g1_high.melt(ignore_index= False)
soc_g1_high = soc_g1_high.melt(ignore_index= False)
confed_g2_high = confed_g2_high.melt(ignore_index= False)
soc_g2_high = soc_g2_high.melt(ignore_index= False)

In [ ]:
confed_g1_low = confed_g1_low.melt(ignore_index= False)
soc_g1_low = soc_g1_low.melt(ignore_index= False)
confed_g2_low = confed_g2_low.melt(ignore_index= False)
soc_g2_low = soc_g2_low.melt(ignore_index= False)

In [ ]:
confed_g1_med = confed_g1_med.melt(ignore_index= False)
soc_g1_med = soc_g1_med.melt(ignore_index= False)
confed_g2_med = confed_g2_med.melt(ignore_index= False)
soc_g2_med = soc_g2_med.melt(ignore_index= False)

In [ ]:
confed_g1_high['trial'] = confed_g1_high.index
confed_g1_high = confed_g1_high.reset_index(drop=True)

soc_g1_high['trial'] = soc_g1_high.index
soc_g1_high = soc_g1_high.reset_index(drop=True)

confed_g2_high['trial'] = confed_g2_high.index
confed_g2_high = confed_g2_high.reset_index(drop=True)

soc_g2_high['trial'] = soc_g2_high.index
soc_g2_high = soc_g2_high.reset_index(drop=True)

In [ ]:
confed_g1_low['trial'] = confed_g1_low.index
confed_g1_low = confed_g1_low.reset_index(drop=True)

soc_g1_low['trial'] = soc_g1_low.index
soc_g1_low = soc_g1_low.reset_index(drop=True)

confed_g2_low['trial'] = confed_g2_low.index
confed_g2_low = confed_g2_low.reset_index(drop=True)

soc_g2_low['trial'] = soc_g2_low.index
soc_g2_low = soc_g2_low.reset_index(drop=True)

In [ ]:
confed_g1_med['trial'] = confed_g1_med.index
confed_g1_med = confed_g1_med.reset_index(drop=True)

soc_g1_med['trial'] = soc_g1_med.index
soc_g1_med = soc_g1_med.reset_index(drop=True)

confed_g2_med['trial'] = confed_g2_med.index
confed_g2_med = confed_g2_med.reset_index(drop=True)

soc_g2_med['trial'] = soc_g2_med.index
soc_g2_med = soc_g2_med.reset_index(drop=True)

In [ ]:
confed_g1_high

In [ ]:
sns.lineplot(data=confed_g1_high, x='trial', y='value', label='Confederate Game 1')
sns.lineplot(data=soc_g1_high, x='trial', y='value', label='Social Game 1')
sns.lineplot(data=confed_g2_high, x='trial', y='value', label='Confederate Game 2')
sns.lineplot(data=soc_g2_high, x='trial', y='value', label='Social Game 2')
plt.legend()

In [ ]:
sns.lineplot(data=confed_g1_low, x='trial', y='value', label='Confederate Game 1')
sns.lineplot(data=soc_g1_low, x='trial', y='value', label='Social Game 1')
sns.lineplot(data=confed_g2_low, x='trial', y='value', label='Confederate Game 2')
sns.lineplot(data=soc_g2_low, x='trial', y='value', label='Social Game 2')
plt.legend()

In [ ]:
sns.lineplot(data=confed_g1_med, x='trial', y='value', label='Confederate Game 1')
sns.lineplot(data=soc_g1_med, x='trial', y='value', label='Social Game 1')
sns.lineplot(data=confed_g2_med, x='trial', y='value', label='Confederate Game 2')
sns.lineplot(data=soc_g2_med, x='trial', y='value', label='Social Game 2')
plt.legend()

In [ ]:
confed_g2_high

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
confed_g2_high_trial = np.array(confed_g2_high['trial'])

In [ ]:
confed_g2_high_trial = confed_g2_high_trial.reshape(-1, 1)

In [ ]:
confed_g2_high_val = np.array(confed_g2_high.value)

In [ ]:
confed_g2_high_trial

In [ ]:
reg = LinearRegression().fit(confed_g2_high_trial,confed_g2_high_val)

In [ ]:
reg.intercept_

In [ ]:
reg.score(confed_g2_high_trial, confed_g2_high_val)

In [ ]:
reg.coef_

In [ ]:
confed_high = confed_g1_high.append(confed_g2_high)
soc_high = soc_g1_high.append(soc_g2_high)
high = confed_high.append(soc_high)

In [ ]:
confed_low = confed_g1_low.append(confed_g2_low)
soc_low = soc_g1_low.append(soc_g2_low)
low = confed_low.append(soc_low)

In [ ]:
confed_med = confed_g1_med.append(confed_g2_med)
soc_med = soc_g1_med.append(soc_g2_med)
med = confed_med.append(soc_med)

In [ ]:
high = high.rename(columns={"variable":"participant"})
high = high.set_index('participant')

In [ ]:
low = low.rename(columns={"variable":"participant"})
low = low.set_index('participant')

In [ ]:
med = med.rename(columns={"variable":"participant"})
med = med.set_index('participant')

In [ ]:
high = anova_label(high)

In [ ]:
low = anova_label(low)

In [ ]:
med = anova_label(med)

In [ ]:
high_g1 = high[high.game == '1']

In [ ]:
low_g1 = low[low.game == '1']

In [ ]:
med_g1 = med[med.game == '1']

In [ ]:
med_g1

In [ ]:
pg.mixed_anova(data=high_g1,dv='value',within='trial',between='condition',subject='participant')

In [ ]:
pg.mixed_anova(data=med_g1,dv='value',within='trial',between='condition',subject='participant')

In [ ]:
#takes a long time to run
#pg.pairwise_ttests(data=high_g1,dv='value',within='trial',between='condition',subject='participant')

In [ ]:
#remove c07 (missing 2 trials)
low_g1 = low_g1[low_g1.participant != 'c07_p1']

In [ ]:
pg.mixed_anova(data=low_g1,dv='value',within='trial',between='condition',subject='participant')

In [ ]:
#takes a long time to run
#pg.pairwise_ttests(dv='value', within='trial', between='condition', subject='participant', data=low_g1)

# disagregate game 2 by same and different and compare block 0 using choice behavior

In [ ]:
struct_soc = ['d','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',
                               'd','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',
                               'd','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s',
                               'd','d','d','d','d','d','s','s','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','d','d','d','d','d','d','d','d','d','s','s','s','s']

In [ ]:
struct_confed = ['s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s','s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s','s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s','s','d','d','d','d','d','d','d','s','s','s','s','s','s','s','s','d','d','d','d',
                                   'd','d','d','s','s','s','s','s','s','s','s']

In [ ]:
struct_confed = struct_confed[0:31]

In [ ]:
struct_soc = struct_soc[0:36]

In [ ]:
confed_g2_blocks_90choices_b0 = confed_g2_blocks_90choices[confed_g2_blocks_90choices.block == '0']
confed_g2_blocks_90choices_b1 = confed_g2_blocks_90choices[confed_g2_blocks_90choices.block == '1']

In [32]:
confed_g2_blocks_30choices_b0 = confed_g2_blocks_30choices[confed_g2_blocks_30choices.block == '0']
confed_g2_blocks_30choices_b1 = confed_g2_blocks_30choices[confed_g2_blocks_30choices.block == '1']

NameError: name 'confed_g2_blocks_30choices' is not defined

In [ ]:
confed_g2_blocks_90choices_b0['struct'] = struct_confed
confed_g2_blocks_30choices_b0['struct'] = struct_confed

In [ ]:
confed_g2_blocks_90choices_b1['struct'] = struct_confed
confed_g2_blocks_30choices_b1['struct'] = struct_confed

In [ ]:
soc_g2_blocks_30choices_b0 = soc_g2_blocks_30choices[soc_g2_blocks_30choices.block == '0']

In [ ]:
soc_g2_blocks_90choices_b0 = soc_g2_blocks_90choices[soc_g2_blocks_90choices.block == '0']

In [ ]:
soc_g2_blocks_90choices_b0['struct'] = struct_soc
soc_g2_blocks_30choices_b0['struct'] = struct_soc

In [ ]:
confed_g2s_blocks_90choices_b0 = confed_g2_blocks_90choices_b0[confed_g2_blocks_90choices_b0['struct']=='s']
confed_g2d_blocks_90choices_b0 = confed_g2_blocks_90choices_b0[confed_g2_blocks_90choices_b0['struct']=='d']

In [ ]:
confed_g2s_blocks_30choices_b0 = confed_g2_blocks_30choices_b0[confed_g2_blocks_30choices_b0['struct']=='s']
confed_g2d_blocks_30choices_b0 = confed_g2_blocks_30choices_b0[confed_g2_blocks_30choices_b0['struct']=='d']

In [ ]:
confed_g2s_blocks_90choices_b1 = confed_g2_blocks_90choices_b1[confed_g2_blocks_90choices_b1['struct']=='s']
confed_g2d_blocks_90choices_b1 = confed_g2_blocks_90choices_b1[confed_g2_blocks_90choices_b1['struct']=='d']

In [ ]:
soc_g2s_blocks_90choices_b0 = soc_g2_blocks_90choices_b0[soc_g2_blocks_90choices_b0.struct == 's']
soc_g2d_blocks_90choices_b0 = soc_g2_blocks_90choices_b0[soc_g2_blocks_90choices_b0.struct == 'd']
soc_g2s_blocks_30choices_b0 = soc_g2_blocks_30choices_b0[soc_g2_blocks_30choices_b0.struct == 's']
soc_g2d_blocks_30choices_b0 = soc_g2_blocks_30choices_b0[soc_g2_blocks_30choices_b0.struct == 'd']

In [ ]:
sp.stats.ttest_ind(confed_g2s_blocks_90choices_b0[90.0],confed_g2d_blocks_90choices_b0[90.0])

In [ ]:
confed_g2s_blocks_90choices_b0.describe()

In [ ]:
confed_g2d_blocks_90choices_b0.describe()

In [ ]:
sp.stats.ttest_ind(confed_g2s_blocks_90choices_b0[90.0],confed_g2d_blocks_90choices_b0[90.0])

In [ ]:
sp.stats.ttest_ind(confed_g2s_blocks_30choices_b0[30.0],confed_g2d_blocks_30choices_b0[30.0])

In [ ]:
sp.stats.ttest_ind(soc_g2s_blocks_30choices_b0[30.0],soc_g2d_blocks_30choices_b0[30.0])

In [ ]:
sp.stats.ttest_ind(soc_g2s_blocks_90choices_b0[90.0],soc_g2d_blocks_90choices_b0[90.0])

In [ ]:
soc_g2d_blocks_90choices_b0.describe()

In [ ]:
soc_g2s_blocks_90choices_b0.describe()

In [ ]:
block_choices['soc_g1_b3_choices']['struct'] = struct_soc

In [ ]:
block_choices['confed_g1_b3_choices']['struct'] = struct_confed

In [ ]:
block_choices['soc_g1_b3_choices_d'] = block_choices['soc_g1_b3_choices'][block_choices['soc_g1_b3_choices'].struct == 'd']

In [ ]:
block_choices['soc_g1_b3_choices_s'] = block_choices['soc_g1_b3_choices'][block_choices['soc_g1_b3_choices'].struct == 's']

In [ ]:
block_choices['confed_g1_b3_choices_d'] = block_choices['confed_g1_b3_choices'][block_choices['confed_g1_b3_choices'].struct == 'd']

In [ ]:
block_choices['confed_g1_b3_choices_s'] = block_choices['confed_g1_b3_choices'][block_choices['confed_g1_b3_choices'].struct == 's']

In [ ]:
len(block_choices['confed_g1_b3_choices_s'])

In [ ]:
soc_jsd_g1_b3_d = np.array(block_choices['soc_g1_b3_choices_d']['matchi'])

In [ ]:
confed_jsd_g1_b3_d = np.array(block_choices['confed_g1_b3_choices_d']['matchi'])

In [ ]:
confed_jsd_g1_b3_s = np.array(block_choices['confed_g1_b3_choices_s']['matchi'])

In [ ]:
block_choices['soc_g1_b3_choices_s'] = block_choices['soc_g1_b3_choices'][block_choices['soc_g1_b3_choices'].struct == 's']

In [ ]:
soc_jsd_g1_b3_s = np.array(block_choices['soc_g1_b3_choices_s']['matchi'])

In [ ]:
len(soc_jsd_g1_b3_s)

In [ ]:
len(np.array(soc_g2s_blocks_90choices_b0[90.0]))

In [ ]:
soc_jsd_in_g1_vs_g2_diff = pd.DataFrame({'jsd_g1_b3_match_d': soc_jsd_g1_b3_d,'g2d_b0_90_choices': np.array(soc_g2d_blocks_90choices_b0[90.0])})


In [ ]:
soc_jsd_in_g1_vs_g2_same = pd.DataFrame({'jsd_g1_b3_match_s': soc_jsd_g1_b3_s,'g2s_b0_90_choices': np.array(soc_g2s_blocks_90choices_b0[90.0])})


In [ ]:
confed_jsd_in_g1_vs_g2_same = pd.DataFrame({'jsd_g1_b3_match_s': confed_jsd_g1_b3_s,'g2s_b0_90_choices': np.array(confed_g2s_blocks_90choices_b0[90.0])})


In [ ]:
confed_jsd_in_g1_vs_g2_diff = pd.DataFrame({'jsd_g1_b3_match_d': confed_jsd_g1_b3_d,'g2d_b0_90_choices': np.array(confed_g2d_blocks_90choices_b0[90.0])})


In [ ]:
soc_jsd_in_g1_vs_g2_diff

In [ ]:
confed_jsd_in_g1_vs_g2_same

In [ ]:
sns.scatterplot(data=soc_jsd_in_g1_vs_g2_diff,x='jsd_g1_b3_match_d',y='g2d_b0_90_choices')

In [ ]:
sns.scatterplot(data=soc_jsd_in_g1_vs_g2_same,x='jsd_g1_b3_match_s',y='g2s_b0_90_choices')

In [ ]:
sns.scatterplot(data=confed_jsd_in_g1_vs_g2_diff,x='jsd_g1_b3_match_d',y='g2d_b0_90_choices')

In [ ]:
sns.scatterplot(data=confed_jsd_in_g1_vs_g2_same,x='jsd_g1_b3_match_s',y='g2s_b0_90_choices')